# Three-dimensional Kakeya needle problem

In [ ]:
# @title Prepare dependencies

!pip install trimesh
!pip install shapely

In [ ]:
# @title Utils

from collections.abc import Mapping
from typing import Any

from matplotlib import pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon as mpl_Polygon
from mpl_toolkits.mplot3d import Axes3D
import trimesh

from IPython import display
import plotly.graph_objects as go
import numpy as np
import itertools # For color cycling


# --- Helper function (from previous response, no changes needed) ---
def get_tube_mesh_vertices_faces(x, y, i, j, N):
    """
    Calculates the vertices and faces for a Plotly Mesh3d representation of a single tube.
    (Identical to the function in the previous response)
    """
    if not isinstance(N, int) or N < 1:
        raise ValueError("N must be a positive integer.")
    if not (0 <= i < N and 0 <= j < N):
         raise ValueError(f"Indices i ({i}) and j ({j}) must be between 0 and N-1 ({N-1}).")

    inv_N = 1.0 / N
    vertices = np.array([
        [x,           y,           0], [x + inv_N,   y,           0],
        [x + inv_N,   y + inv_N,   0], [x,           y + inv_N,   0],
        [x + i*inv_N,       y + j*inv_N,       1], [x + (i+1)*inv_N,   y + j*inv_N,       1],
        [x + (i+1)*inv_N,   y + (j+1)*inv_N,   1], [x + i*inv_N,       y + (j+1)*inv_N,   1]
    ])
    faces = np.array([
        [0, 1, 2], [0, 2, 3], [4, 5, 6], [4, 6, 7], [0, 1, 5], [0, 5, 4],
        [1, 2, 6], [1, 6, 5], [2, 3, 7], [2, 7, 6], [3, 0, 4], [3, 4, 7]
    ])
    return vertices, faces

# --- Main plotting function ---
def plot_multiple_tubes_plotly(tubes_data, title="Multiple Tubes Visualization", opacity=0.6):
    """
    Plots a collection of tubes defined by their parameters in Plotly.

    Args:
        tubes_data (list): A list where each element is a tuple or list
                           containing the parameters for one tube: (x, y, i, j, N).
                           Note: 'N' can potentially vary per tube with this setup.
        title (str, optional): The title for the Plotly figure.
                               Defaults to "Multiple Tubes Visualization".
        opacity (float, optional): The opacity for the tube surfaces (0 to 1).
                                  Defaults to 0.6.

    Returns:
        plotly.graph_objects.Figure: The generated Plotly figure containing all tubes.
    """
    fig = go.Figure()

    if not tubes_data:
        print("Warning: tubes_data list is empty. Returning an empty figure.")
        fig.update_layout(title=title)
        return fig

    all_vertices_list = []
    # Define a cycle of colors for visual distinction
    # color_cycle = go.layout.Colorscale.sequential # Example color cycles

    print(f"Plotting {len(tubes_data)} tubes...")
    for k, tube_params in enumerate(tubes_data):
        try:
            # Unpack parameters for the current tube
            x, y, i, j, N = tube_params
            # Ensure i and j are integers
            i, j = int(i), int(j)

            # Get geometry for the current tube
            vertices, faces = get_tube_mesh_vertices_faces(x, y, i, j, N)
            all_vertices_list.append(vertices) # Collect vertices for bounding box calculation

            # Add Mesh3d trace for the current tube
            fig.add_trace(go.Mesh3d(
                x=vertices[:, 0],
                y=vertices[:, 1],
                z=vertices[:, 2],
                i=faces[:, 0],
                j=faces[:, 1],
                k=faces[:, 2],
                # color=next(color_cycle), # Cycle through predefined colors
                opacity=opacity,
                flatshading=True,
                name=f'Tube {k+1} (N={N})', # Include N in name if it can vary
                showlegend=True,
                hoverinfo='name' # Show tube name on hover
            ))
        except (ValueError, TypeError, IndexError) as e:
            print(f"Warning: Skipping tube {k+1} due to invalid parameters or error: {tube_params} -> {e}")
            continue # Skip to the next tube

    # --- Customize Layout ---
    if not all_vertices_list: # Handle case where all tubes were skipped
         print("Warning: No valid tubes were plotted.")
         fig.update_layout(title=f"{title} (No valid tubes found)")
         return fig

    all_vertices_np = np.vstack(all_vertices_list)
    min_vals = np.min(all_vertices_np, axis=0)
    max_vals = np.max(all_vertices_np, axis=0)
    margin = 0.2 # Add some margin to axis ranges

    # Find the center for camera positioning (optional)
    center = np.mean(all_vertices_np, axis=0)

    fig.update_layout(
        title=title,
        scene=dict(
            xaxis=dict(title='X', range=[min_vals[0]-margin, max_vals[0]+margin], backgroundcolor="rgb(230, 230,230)"),
            yaxis=dict(title='Y', range=[min_vals[1]-margin, max_vals[1]+margin], backgroundcolor="rgb(230, 230,230)"),
            zaxis=dict(title='Z', range=[min_vals[2]-margin, max_vals[2]+margin], backgroundcolor="rgb(230, 230,230)"),
            aspectmode='data',
            # Optional: Set camera position to view the center
            # camera=dict(
            #     eye=dict(x=center[0]*1.5, y=center[1]*1.5, z=center[2]+(max_vals[2]-min_vals[2])*0.5 + 1), # Position camera slightly away and above
            #     center=dict(x=center[0], y=center[1], z=center[2]) # Look at the center
            # )
        ),
        margin=dict(l=0, r=0, b=0, t=40), # Adjust margins
        legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01) # Position legend
    )

    return fig

**Prompt for search setup**



In [ ]:
# @title Initial program

from collections.abc import Sequence, Mapping
import re
import numpy as np
from typing import Any
import functools
import time

def get_positions(cap_n: int) -> Any:
  """Generate positions to minimize the area of the union."""
  variable_name_x = f'best_positions_x_{cap_n}'
  variable_name_y = f'best_positions_y_{cap_n}'
  num_tubes = cap_n * cap_n
  best_positions_x = np.random.uniform(0, 1, size=num_tubes).astype(np.float32)
  best_positions_y = np.random.uniform(0, 1, size=num_tubes).astype(np.float32)

  # Sometimes check for previous configuration stored in the context.
  if (
      np.random.rand() < 0.5
      and variable_name_x in globals()
      and variable_name_y in globals()
  ):
    best_positions_x = globals()[variable_name_x]
    best_positions_y = globals()[variable_name_y]

  best_positions = [best_positions_x, best_positions_y]

  curr_positions = best_positions.copy()
  best_score = get_score(best_positions)

  start_time = time.time()

  while time.time() - start_time < 100:  # Search for 100 seconds
    # Mutate best construction
    random_index = np.random.randint(0, len(curr_positions))
    curr_positions[random_index:] += 1e-1 * np.random.randint(-10, 10)

    curr_score = get_score(n, curr_positions)
    if curr_score > best_score:
      best_score = curr_score
      best_positions = curr_positions.copy()
      print(f"Best score: {curr_score}")
  return best_positions


In [ ]:
# @title Evaluation

import jax
import jax.numpy as jnp


FUNCTION_IS_TOO_LARGE = 101.0
WRONG_OUTPUT_SHAPE = 102.0
NO_POLYGON_SET = 103.0


@functools.partial(jax.jit, static_argnames=['cap_n'])
def _is_point_in_single_tube_jax(point, x, y, i, j, cap_n):
  """Helper function: Checks if a SINGLE 3D point is inside a SINGLE tube."""
  a, b, c = point[0], point[1], point[2]
  valid_z = (0 <= c) & (c <= 1)
  inv_cap_n = 1.0 / cap_n
  i_inv_cap_n = i * inv_cap_n
  j_inv_cap_n = j * inv_cap_n
  i_plus_1_inv_cap_n = (i + 1) * inv_cap_n
  j_plus_1_inv_cap_n = (j + 1) * inv_cap_n
  x_min_at_c = (1 - c) * x + c * (x + i_inv_cap_n)
  x_max_at_c = (1 - c) * (x + inv_cap_n) + c * (x + i_plus_1_inv_cap_n)
  y_min_at_c = (1 - c) * y + c * (y + j_inv_cap_n)
  y_max_at_c = (1 - c) * (y + inv_cap_n) + c * (y + j_plus_1_inv_cap_n)
  is_inside_x = (x_min_at_c <= a) & (a <= x_max_at_c)
  is_inside_y = (y_min_at_c <= b) & (b <= y_max_at_c)
  is_inside = valid_z & is_inside_x & is_inside_y
  return is_inside


_check_point_vs_all_tubes = jax.vmap(
    _is_point_in_single_tube_jax, in_axes=(None, 0, 0, 0, 0, None), out_axes=0
)

_check_all_points_vs_all_tubes = jax.vmap(
    lambda point, x_arr, y_arr, i_arr, j_arr, cap_n_static: _check_point_vs_all_tubes(
        point, x_arr, y_arr, i_arr, j_arr, cap_n_static
    ),
    in_axes=(0, None, None, None, None, None),
    out_axes=0,
)


def are_points_in_union_of_tubes_jax(points, tubes_params, cap_n):
  """Checks if an array of 3D points p = [a, b, c] are inside the UNION of

  specified generalized tubes T, using JAX and vmap.

  Args:
      points (jax.Array): Shape (M, 3).
      tubes_params (jax.Array): Shape (K, 4) [x, y, i, j].
      N (int): Number of divisions (>= 1).

  Returns:
      jax.Array: Shape (M,), True if point is in >= 1 tube.
  """
  if not isinstance(cap_n, int) or cap_n < 1:
    raise ValueError('N must be a positive integer.')
  # Basic shape checks are good practice but omitted here for brevity based on prompt
  # if points.shape[1] != 3: raise ValueError(...)
  # if tubes_params.shape[1] != 4: raise ValueError(...)

  x_all = tubes_params[:, 0]
  y_all = tubes_params[:, 1]
  i_all = tubes_params[:, 2].astype(jnp.int32)
  j_all = tubes_params[:, 3].astype(jnp.int32)

  result_matrix = _check_all_points_vs_all_tubes(
      points, x_all, y_all, i_all, j_all, cap_n
  )
  is_in_union = jnp.any(result_matrix, axis=1)
  return is_in_union


def get_score(positions):
  """Returns the score of the given positions."""
  x_rand, y_rand = positions
  num_points = 5_000_000
  cap_n = int(np.sqrt(len(x_rand)))
  i_vals = np.arange(cap_n)
  j_vals = np.arange(cap_n)

  i_grid, j_grid = np.meshgrid(i_vals, j_vals, indexing='ij')
  i_flat = i_grid.flatten()
  j_flat = j_grid.flatten()

  tubes_params_np = np.stack(
      [np.array(x_rand), np.array(y_rand), i_flat, j_flat], axis=-1
  )

  tubes_params_jax = jnp.asarray(tubes_params_np)

  num_mc_trials = 5000
  mc_hits = 0

  key = jax.random.PRNGKey(1722)

  for trial in range(num_mc_trials):
    _, key = jax.random.split(key, 2)
    rand_points = jax.random.uniform(key, (num_points, 3))
    rand_points = rand_points.at[:, 0].multiply(2.5)
    rand_points = rand_points.at[:, 1].multiply(2.5)

    results_large = are_points_in_union_of_tubes_jax(
        rand_points, tubes_params_jax, cap_n
    )
    if trial % 100 == 0:
      print(f'Trial: {trial}, Res: {jnp.sum(results_large)}')
    mc_hits += jnp.sum(results_large) / num_points

  return float(mc_hits / num_mc_trials)

In [ ]:
# @title Constructions found by AlphaEvolve

# n = 8
X_RAND = np.array([0.7782645693530773, 0.7782645693530773, 0.7782645693530773, 0.7782645693530773, 0.7782645693530773, 0.7782645693530773, 0.7782645693530773, 0.7782645693530773, 0.7543773442171465, 0.7543773442171465, 0.7543773442171465, 0.7543773442171465, 0.7543773442171465, 0.7543773442171465, 0.7543773442171465, 0.7543773442171465, 0.7154513573316721, 0.7154513573316721, 0.7154513573316721, 0.7154513573316721, 0.7154513573316721, 0.7154513573316721, 0.7154513573316721, 0.7154513573316721, 0.664684027483473, 0.664684027483473, 0.664684027483473, 0.664684027483473, 0.664684027483473, 0.664684027483473, 0.664684027483473, 0.664684027483473, 0.606009010812178, 0.606009010812178, 0.606009010812178, 0.606009010812178, 0.606009010812178, 0.606009010812178, 0.606009010812178, 0.606009010812178, 0.543851625123817, 0.543851625123817, 0.543851625123817, 0.543851625123817, 0.543851625123817, 0.543851625123817, 0.543851625123817, 0.543851625123817, 0.48285370506332126, 0.48285370506332126, 0.48285370506332126, 0.48285370506332126, 0.48285370506332126, 0.48285370506332126, 0.48285370506332126, 0.48285370506332126, 0.42758499531866156, 0.42758499531866156, 0.42758499531866156, 0.42758499531866156, 0.42758499531866156, 0.42758499531866156, 0.42758499531866156, 0.42758499531866156])
Y_RAND = np.array([0.7015371649299834, 0.627793174251546, 0.5453264334970943, 0.45848713572833855, 0.3718973384240025, 0.29016358633236816, 0.21759049883026438, 0.1579131295393012, 0.7015371649299834, 0.627793174251546, 0.5453264334970943, 0.45848713572833855, 0.3718973384240025, 0.29016358633236816, 0.21759049883026438, 0.1579131295393012, 0.7015371649299834, 0.627793174251546, 0.5453264334970943, 0.45848713572833855, 0.3718973384240025, 0.29016358633236816, 0.21759049883026438, 0.1579131295393012, 0.7015371649299834, 0.627793174251546, 0.5453264334970943, 0.45848713572833855, 0.3718973384240025, 0.29016358633236816, 0.21759049883026438, 0.1579131295393012, 0.7015371649299834, 0.627793174251546, 0.5453264334970943, 0.45848713572833855, 0.3718973384240025, 0.29016358633236816, 0.21759049883026438, 0.1579131295393012, 0.7015371649299834, 0.627793174251546, 0.5453264334970943, 0.45848713572833855, 0.3718973384240025, 0.29016358633236816, 0.21759049883026438, 0.1579131295393012, 0.7015371649299834, 0.627793174251546, 0.5453264334970943, 0.45848713572833855, 0.3718973384240025, 0.29016358633236816, 0.21759049883026438, 0.1579131295393012, 0.7015371649299834, 0.627793174251546, 0.5453264334970943, 0.45848713572833855, 0.3718973384240025, 0.29016358633236816, 0.21759049883026438, 0.1579131295393012])
# + union_volume_8 = 0.021198255941271782


# n = 16
X_RAND = np.array([0.4177295535649418, 0.4177295535649418, 0.4177295535649418, 0.4177295535649418, 0.4177295535649418, 0.4177295535649418, 0.4177295535649418, 0.4177295535649418, 0.4177295535649418, 0.4177295535649418, 0.4177295535649418, 0.4177295535649418, 0.4177295535649418, 0.4177295535649418, 0.4177295535649418, 0.4177295535649418, 0.3910973179349275, 0.3910973179349275, 0.3910973179349275, 0.3910973179349275, 0.3910973179349275, 0.3910973179349275, 0.3910973179349275, 0.3910973179349275, 0.3910973179349275, 0.3910973179349275, 0.3910973179349275, 0.3910973179349275, 0.3910973179349275, 0.3910973179349275, 0.3910973179349275, 0.3910973179349275, 0.36495547624161984, 0.36495547624161984, 0.36495547624161984, 0.36495547624161984, 0.36495547624161984, 0.36495547624161984, 0.36495547624161984, 0.36495547624161984, 0.36495547624161984, 0.36495547624161984, 0.36495547624161984, 0.36495547624161984, 0.36495547624161984, 0.36495547624161984, 0.36495547624161984, 0.36495547624161984, 0.3394217464366585, 0.3394217464366585, 0.3394217464366585, 0.3394217464366585, 0.3394217464366585, 0.3394217464366585, 0.3394217464366585, 0.3394217464366585, 0.3394217464366585, 0.3394217464366585, 0.3394217464366585, 0.3394217464366585, 0.3394217464366585, 0.3394217464366585, 0.3394217464366585, 0.3394217464366585, 0.3146111081150398, 0.3146111081150398, 0.3146111081150398, 0.3146111081150398, 0.3146111081150398, 0.3146111081150398, 0.3146111081150398, 0.3146111081150398, 0.3146111081150398, 0.3146111081150398, 0.3146111081150398, 0.3146111081150398, 0.3146111081150398, 0.3146111081150398, 0.3146111081150398, 0.3146111081150398, 0.29063528475655337, 0.29063528475655337, 0.29063528475655337, 0.29063528475655337, 0.29063528475655337, 0.29063528475655337, 0.29063528475655337, 0.29063528475655337, 0.29063528475655337, 0.29063528475655337, 0.29063528475655337, 0.29063528475655337, 0.29063528475655337, 0.29063528475655337, 0.29063528475655337, 0.29063528475655337, 0.2676022406296601, 0.2676022406296601, 0.2676022406296601, 0.2676022406296601, 0.2676022406296601, 0.2676022406296601, 0.2676022406296601, 0.2676022406296601, 0.2676022406296601, 0.2676022406296601, 0.2676022406296601, 0.2676022406296601, 0.2676022406296601, 0.2676022406296601, 0.2676022406296601, 0.2676022406296601, 0.24561569462327526, 0.24561569462327526, 0.24561569462327526, 0.24561569462327526, 0.24561569462327526, 0.24561569462327526, 0.24561569462327526, 0.24561569462327526, 0.24561569462327526, 0.24561569462327526, 0.24561569462327526, 0.24561569462327526, 0.24561569462327526, 0.24561569462327526, 0.24561569462327526, 0.24561569462327526, 0.22477465319569945, 0.22477465319569945, 0.22477465319569945, 0.22477465319569945, 0.22477465319569945, 0.22477465319569945, 0.22477465319569945, 0.22477465319569945, 0.22477465319569945, 0.22477465319569945, 0.22477465319569945, 0.22477465319569945, 0.22477465319569945, 0.22477465319569945, 0.22477465319569945, 0.22477465319569945, 0.20517296454384504, 0.20517296454384504, 0.20517296454384504, 0.20517296454384504, 0.20517296454384504, 0.20517296454384504, 0.20517296454384504, 0.20517296454384504, 0.20517296454384504, 0.20517296454384504, 0.20517296454384504, 0.20517296454384504, 0.20517296454384504, 0.20517296454384504, 0.20517296454384504, 0.20517296454384504, 0.1868988960003598, 0.1868988960003598, 0.1868988960003598, 0.1868988960003598, 0.1868988960003598, 0.1868988960003598, 0.1868988960003598, 0.1868988960003598, 0.1868988960003598, 0.1868988960003598, 0.1868988960003598, 0.1868988960003598, 0.1868988960003598, 0.1868988960003598, 0.1868988960003598, 0.1868988960003598, 0.17003473656164636, 0.17003473656164636, 0.17003473656164636, 0.17003473656164636, 0.17003473656164636, 0.17003473656164636, 0.17003473656164636, 0.17003473656164636, 0.17003473656164636, 0.17003473656164636, 0.17003473656164636, 0.17003473656164636, 0.17003473656164636, 0.17003473656164636, 0.17003473656164636, 0.17003473656164636, 0.15465642633661264, 0.15465642633661264, 0.15465642633661264, 0.15465642633661264, 0.15465642633661264, 0.15465642633661264, 0.15465642633661264, 0.15465642633661264, 0.15465642633661264, 0.15465642633661264, 0.15465642633661264, 0.15465642633661264, 0.15465642633661264, 0.15465642633661264, 0.15465642633661264, 0.15465642633661264, 0.14083321458476428, 0.14083321458476428, 0.14083321458476428, 0.14083321458476428, 0.14083321458476428, 0.14083321458476428, 0.14083321458476428, 0.14083321458476428, 0.14083321458476428, 0.14083321458476428, 0.14083321458476428, 0.14083321458476428, 0.14083321458476428, 0.14083321458476428, 0.14083321458476428, 0.14083321458476428, 0.12862734788350755, 0.12862734788350755, 0.12862734788350755, 0.12862734788350755, 0.12862734788350755, 0.12862734788350755, 0.12862734788350755, 0.12862734788350755, 0.12862734788350755, 0.12862734788350755, 0.12862734788350755, 0.12862734788350755, 0.12862734788350755, 0.12862734788350755, 0.12862734788350755, 0.12862734788350755, 0.1180937898288626, 0.1180937898288626, 0.1180937898288626, 0.1180937898288626, 0.1180937898288626, 0.1180937898288626, 0.1180937898288626, 0.1180937898288626, 0.1180937898288626, 0.1180937898288626, 0.1180937898288626, 0.1180937898288626, 0.1180937898288626, 0.1180937898288626, 0.1180937898288626, 0.1180937898288626])
Y_RAND = np.array([0.8525760966898448, 0.8243079645487739, 0.7910243150830173, 0.7532398896457715, 0.7115390354042346, 0.6665666682401168, 0.6190182989373039, 0.569629276904592, 0.5191634177824606, 0.4684011908112635, 0.4181276486466394, 0.3691202862910645, 0.3221370169067197, 0.277904450467231, 0.23710665652235868, 0.20037458486275705, 0.8525760966898448, 0.8243079645487739, 0.7910243150830173, 0.7532398896457715, 0.7115390354042346, 0.6665666682401168, 0.6190182989373039, 0.569629276904592, 0.5191634177824606, 0.4684011908112635, 0.4181276486466394, 0.3691202862910645, 0.3221370169067197, 0.277904450467231, 0.23710665652235868, 0.20037458486275705, 0.8525760966898448, 0.8243079645487739, 0.7910243150830173, 0.7532398896457715, 0.7115390354042346, 0.6665666682401168, 0.6190182989373039, 0.569629276904592, 0.5191634177824606, 0.4684011908112635, 0.4181276486466394, 0.3691202862910645, 0.3221370169067197, 0.277904450467231, 0.23710665652235868, 0.20037458486275705, 0.8525760966898448, 0.8243079645487739, 0.7910243150830173, 0.7532398896457715, 0.7115390354042346, 0.6665666682401168, 0.6190182989373039, 0.569629276904592, 0.5191634177824606, 0.4684011908112635, 0.4181276486466394, 0.3691202862910645, 0.3221370169067197, 0.277904450467231, 0.23710665652235868, 0.20037458486275705, 0.8525760966898448, 0.8243079645487739, 0.7910243150830173, 0.7532398896457715, 0.7115390354042346, 0.6665666682401168, 0.6190182989373039, 0.569629276904592, 0.5191634177824606, 0.4684011908112635, 0.4181276486466394, 0.3691202862910645, 0.3221370169067197, 0.277904450467231, 0.23710665652235868, 0.20037458486275705, 0.8525760966898448, 0.8243079645487739, 0.7910243150830173, 0.7532398896457715, 0.7115390354042346, 0.6665666682401168, 0.6190182989373039, 0.569629276904592, 0.5191634177824606, 0.4684011908112635, 0.4181276486466394, 0.3691202862910645, 0.3221370169067197, 0.277904450467231, 0.23710665652235868, 0.20037458486275705, 0.8525760966898448, 0.8243079645487739, 0.7910243150830173, 0.7532398896457715, 0.7115390354042346, 0.6665666682401168, 0.6190182989373039, 0.569629276904592, 0.5191634177824606, 0.4684011908112635, 0.4181276486466394, 0.3691202862910645, 0.3221370169067197, 0.277904450467231, 0.23710665652235868, 0.20037458486275705, 0.8525760966898448, 0.8243079645487739, 0.7910243150830173, 0.7532398896457715, 0.7115390354042346, 0.6665666682401168, 0.6190182989373039, 0.569629276904592, 0.5191634177824606, 0.4684011908112635, 0.4181276486466394, 0.3691202862910645, 0.3221370169067197, 0.277904450467231, 0.23710665652235868, 0.20037458486275705, 0.8525760966898448, 0.8243079645487739, 0.7910243150830173, 0.7532398896457715, 0.7115390354042346, 0.6665666682401168, 0.6190182989373039, 0.569629276904592, 0.5191634177824606, 0.4684011908112635, 0.4181276486466394, 0.3691202862910645, 0.3221370169067197, 0.277904450467231, 0.23710665652235868, 0.20037458486275705, 0.8525760966898448, 0.8243079645487739, 0.7910243150830173, 0.7532398896457715, 0.7115390354042346, 0.6665666682401168, 0.6190182989373039, 0.569629276904592, 0.5191634177824606, 0.4684011908112635, 0.4181276486466394, 0.3691202862910645, 0.3221370169067197, 0.277904450467231, 0.23710665652235868, 0.20037458486275705, 0.8525760966898448, 0.8243079645487739, 0.7910243150830173, 0.7532398896457715, 0.7115390354042346, 0.6665666682401168, 0.6190182989373039, 0.569629276904592, 0.5191634177824606, 0.4684011908112635, 0.4181276486466394, 0.3691202862910645, 0.3221370169067197, 0.277904450467231, 0.23710665652235868, 0.20037458486275705, 0.8525760966898448, 0.8243079645487739, 0.7910243150830173, 0.7532398896457715, 0.7115390354042346, 0.6665666682401168, 0.6190182989373039, 0.569629276904592, 0.5191634177824606, 0.4684011908112635, 0.4181276486466394, 0.3691202862910645, 0.3221370169067197, 0.277904450467231, 0.23710665652235868, 0.20037458486275705, 0.8525760966898448, 0.8243079645487739, 0.7910243150830173, 0.7532398896457715, 0.7115390354042346, 0.6665666682401168, 0.6190182989373039, 0.569629276904592, 0.5191634177824606, 0.4684011908112635, 0.4181276486466394, 0.3691202862910645, 0.3221370169067197, 0.277904450467231, 0.23710665652235868, 0.20037458486275705, 0.8525760966898448, 0.8243079645487739, 0.7910243150830173, 0.7532398896457715, 0.7115390354042346, 0.6665666682401168, 0.6190182989373039, 0.569629276904592, 0.5191634177824606, 0.4684011908112635, 0.4181276486466394, 0.3691202862910645, 0.3221370169067197, 0.277904450467231, 0.23710665652235868, 0.20037458486275705, 0.8525760966898448, 0.8243079645487739, 0.7910243150830173, 0.7532398896457715, 0.7115390354042346, 0.6665666682401168, 0.6190182989373039, 0.569629276904592, 0.5191634177824606, 0.4684011908112635, 0.4181276486466394, 0.3691202862910645, 0.3221370169067197, 0.277904450467231, 0.23710665652235868, 0.20037458486275705, 0.8525760966898448, 0.8243079645487739, 0.7910243150830173, 0.7532398896457715, 0.7115390354042346, 0.6665666682401168, 0.6190182989373039, 0.569629276904592, 0.5191634177824606, 0.4684011908112635, 0.4181276486466394, 0.3691202862910645, 0.3221370169067197, 0.277904450467231, 0.23710665652235868, 0.20037458486275705])
# + union_volume_16 = 0.015686817467212677


# # n = 32
# X_RAND = np.array([0.5671632289886475, 0.5741413831710815, 0.5808230042457581, 0.5871995687484741, 0.5932641625404358, 0.5990108251571655, 0.60443514585495, 0.6095338463783264, 0.6143049597740173, 0.6187477111816406, 0.6228626370429993, 0.626651406288147, 0.6301167607307434, 0.6332627534866333, 0.6360943913459778, 0.6386178731918335, 0.6408402323722839, 0.6427697539329529, 0.6444154381752014, 0.6457873582839966, 0.6468964219093323, 0.6477543115615845, 0.6483733654022217, 0.648766815662384, 0.6489484310150146, 0.6489323973655701, 0.6487336158752441, 0.6483671069145203, 0.6478483080863953, 0.6471928954124451, 0.6464165449142456, 0.6455349922180176, 0.5594701766967773, 0.5664541721343994, 0.5731371641159058, 0.5795106291770935, 0.5855673551559448, 0.5913013219833374, 0.5967080593109131, 0.6017842292785645, 0.6065279245376587, 0.6109384894371033, 0.6150164008140564, 0.6187636256217957, 0.622183084487915, 0.6252789497375488, 0.6280565857887268, 0.6305224299430847, 0.6326838731765747, 0.6345494985580444, 0.6361287832260132, 0.6374320387840271, 0.6384706497192383, 0.6392566561698914, 0.6398029923439026, 0.6401231288909912, 0.640231192111969, 0.640142023563385, 0.6398707032203674, 0.6394327878952026, 0.6388440132141113, 0.6381203532218933, 0.6372778415679932, 0.6363324522972107, 0.5518313646316528, 0.5588164329528809, 0.5654963254928589, 0.5718623399734497, 0.5779070258140564, 0.5836243629455566, 0.5890096426010132, 0.5940596461296082, 0.598772406578064, 0.6031473278999329, 0.6071851253509521, 0.6108877658843994, 0.6142584681510925, 0.6173016428947449, 0.6200228333473206, 0.622428834438324, 0.6245273947715759, 0.6263273358345032, 0.6278386116027832, 0.6290719509124756, 0.6300389766693115, 0.6307523250579834, 0.6312252283096313, 0.6314716339111328, 0.6315061450004578, 0.6313439011573792, 0.6310003995895386, 0.6304915547370911, 0.6298335194587708, 0.6290425658226013, 0.6281350255012512, 0.6271271109580994, 0.5442546010017395, 0.551236093044281, 0.5579085350036621, 0.5642628073692322, 0.5702913999557495, 0.5759880542755127, 0.581348180770874, 0.5863684415817261, 0.5910468697547913, 0.5953829884529114, 0.5993775725364685, 0.6030328273773193, 0.6063521504402161, 0.6093401312828064, 0.6120026707649231, 0.6143468022346497, 0.6163806319236755, 0.6181133985519409, 0.6195552945137024, 0.6207175254821777, 0.6216121912002563, 0.6222522854804993, 0.6226513981819153, 0.62282395362854, 0.6227849721908569, 0.6225500106811523, 0.6221348643302917, 0.6215559244155884, 0.6208296418190002, 0.6199725270271301, 0.6190012097358704, 0.6179322600364685, 0.5367471575737, 0.5437206029891968, 0.5503811240196228, 0.5567195415496826, 0.5627279877662659, 0.5684002041816711, 0.573731541633606, 0.5787185430526733, 0.5833593606948853, 0.5876535773277283, 0.5916021466255188, 0.5952072739601135, 0.5984727144241333, 0.6014032363891602, 0.6040050983428955, 0.6062855124473572, 0.6082530617713928, 0.6099172234535217, 0.6112886667251587, 0.6123790144920349, 0.6132006645202637, 0.6137670278549194, 0.6140922904014587, 0.6141911149024963, 0.6140790581703186, 0.613771915435791, 0.6132859587669373, 0.6126379370689392, 0.611844539642334, 0.6109226942062378, 0.6098892092704773, 0.6087608337402344, 0.5293156504631042, 0.536276638507843, 0.5429210066795349, 0.5492393970489502, 0.555223822593689, 0.560867965221405, 0.566166877746582, 0.5711173415184021, 0.5757173895835876, 0.5799667835235596, 0.583866536617279, 0.5874190926551819, 0.5906283259391785, 0.5934993624687195, 0.5960386395454407, 0.5982537865638733, 0.6001536250114441, 0.6017481088638306, 0.6030482053756714, 0.6040659546852112, 0.6048142313957214, 0.6053068041801453, 0.6055582761764526, 0.6055837869644165, 0.6053991317749023, 0.6050206422805786, 0.6044650077819824, 0.6037490963935852, 0.6028900146484375, 0.6019049882888794, 0.6008111238479614, 0.5996251702308655, 0.521966278553009, 0.5289102792739868, 0.5355343818664551, 0.5418287515640259, 0.5477853417396545, 0.5533977150917053, 0.5586609244346619, 0.5635716319084167, 0.5681279897689819, 0.5723297595977783, 0.5761781334877014, 0.5796757340431213, 0.5828266739845276, 0.58563631772995, 0.5881113409996033, 0.5902597308158875, 0.5920907258987427, 0.5936145782470703, 0.5948427319526672, 0.5957874655723572, 0.5964622497558594, 0.5968811511993408, 0.5970591902732849, 0.597011923789978, 0.5967555642127991, 0.5963068008422852, 0.595682680606842, 0.594900369644165, 0.5939772725105286, 0.5929309129714966, 0.5917784571647644, 0.5905370712280273, 0.5147042870521545, 0.5216271281242371, 0.5282267928123474, 0.5344932675361633, 0.5404184460639954, 0.5459955334663391, 0.5512197017669678, 0.5560876131057739, 0.5605973601341248, 0.5647488832473755, 0.5685434937477112, 0.5719839930534363, 0.5750746726989746, 0.5778211951255798, 0.5802305936813354, 0.5823110938072205, 0.5840722918510437, 0.5855247974395752, 0.58668053150177, 0.5875521302223206, 0.5881534218788147, 0.5884990096092224, 0.588604211807251, 0.5884850025177002, 0.5881580710411072, 0.5876402854919434, 0.5869491100311279, 0.5861020684242249, 0.5851168036460876, 0.5840110182762146, 0.5828021764755249, 0.5815075635910034, 0.5075345635414124, 0.5144320130348206, 0.521003246307373, 0.5272380709648132, 0.5331281423568726, 0.5386667251586914, 0.5438486933708191, 0.5486708283424377, 0.5531313419342041, 0.5572301745414734, 0.5609687566757202, 0.5643501281738281, 0.5673788189888, 0.5700607299804688, 0.5724031329154968, 0.5744147896766663, 0.5761054158210754, 0.5774862170219421, 0.5785692930221558, 0.5793678164482117, 0.5798959732055664, 0.5801687836647034, 0.5802019834518433, 0.5800119042396545, 0.5796155333518982, 0.5790302753448486, 0.5782737731933594, 0.5773638486862183, 0.5763184428215027, 0.5751553773880005, 0.5738924741744995, 0.5725469589233398, 0.5004611611366272, 0.5073291659355164, 0.5138680934906006, 0.520067572593689, 0.5259191393852234, 0.5314158797264099, 0.5365526676177979, 0.5413262844085693, 0.5457349419593811, 0.5497787594795227, 0.5534592866897583, 0.556779682636261, 0.5597448348999023, 0.5623608231544495, 0.5646352767944336, 0.5665772557258606, 0.5681968331336975, 0.5695056319236755, 0.570516049861908, 0.5712417960166931, 0.5716973543167114, 0.5718981623649597, 0.5718603730201721, 0.5716006755828857, 0.571136474609375, 0.5704853534698486, 0.5696653723716736, 0.5686945915222168, 0.5675912499427795, 0.5663732886314392, 0.5650586485862732, 0.5636647939682007, 0.49348756670951843, 0.5003221035003662, 0.5068249702453613, 0.5129855871200562, 0.5187952518463135, 0.5242470502853394, 0.5293357968330383, 0.5340582728385925, 0.5384126901626587, 0.5423993468284607, 0.5460198521614075, 0.5492777228355408, 0.552177906036377, 0.5547268986701965, 0.5569325685501099, 0.5588042140007019, 0.5603525042533875, 0.5615891814231873, 0.5625272393226624, 0.563180685043335, 0.5635644197463989, 0.5636942386627197, 0.5635867118835449, 0.5632588863372803, 0.5627284646034241, 0.5620133876800537, 0.56113201379776, 0.560102641582489, 0.5589436292648315, 0.5576732754707336, 0.5563094615936279, 0.5548698902130127, 0.48661670088768005, 0.4934138357639313, 0.4998769462108612, 0.5059952139854431, 0.5117598176002502, 0.5171636939048767, 0.522201657295227, 0.5268704295158386, 0.5311684608459473, 0.5350958704948425, 0.5386546850204468, 0.5418485403060913, 0.5446825623512268, 0.5471636056900024, 0.5492997765541077, 0.5511008501052856, 0.5525776743888855, 0.5537424683570862, 0.5546085834503174, 0.55519038438797, 0.555503249168396, 0.5555633306503296, 0.5553874373435974, 0.5549931526184082, 0.5543984174728394, 0.5536214709281921, 0.5526809096336365, 0.5515952706336975, 0.5503831505775452, 0.549062967300415, 0.5476527214050293, 0.5461701154708862, 0.47985076904296875, 0.48660677671432495, 0.4930265247821808, 0.49909910559654236, 0.5048155188560486, 0.5101686120033264, 0.515153169631958, 0.5197659134864807, 0.5240053534507751, 0.5278717875480652, 0.5313673615455627, 0.5344958901405334, 0.5372627377510071, 0.5396750569343567, 0.5417413115501404, 0.5434715747833252, 0.5448770523071289, 0.5459703207015991, 0.546765148639679, 0.5472762584686279, 0.5475193858146667, 0.547511100769043, 0.5472685694694519, 0.5468096137046814, 0.5461525917053223, 0.5453159809112549, 0.5443186163902283, 0.5431792736053467, 0.5419166684150696, 0.5405493378639221, 0.539095401763916, 0.5375726222991943, 0.4731915295124054, 0.4799026548862457, 0.4862755835056305, 0.49229925870895386, 0.497964471578598, 0.5032640695571899, 0.5081927180290222, 0.5127472281455994, 0.516926109790802, 0.5207298994064331, 0.5241608023643494, 0.5272228121757507, 0.5299217104911804, 0.532264769077301, 0.5342608690261841, 0.535920262336731, 0.5372546911239624, 0.5382770299911499, 0.5390013456344604, 0.5394428968429565, 0.539617657661438, 0.539542555809021, 0.5392351746559143, 0.5387136340141296, 0.5379964709281921, 0.5371025800704956, 0.5360509157180786, 0.5348604917526245, 0.5335501432418823, 0.5321385264396667, 0.5306438207626343, 0.529083788394928, 0.4666401147842407, 0.4733027517795563, 0.47962552309036255, 0.4855971038341522, 0.4912082552909851, 0.4964516758918762, 0.5013220906257629, 0.5058162808418274, 0.509932816028595, 0.5136723518371582, 0.5170373320579529, 0.5200319290161133, 0.5226621627807617, 0.5249356031417847, 0.5268613696098328, 0.5284501314163208, 0.5297139883041382, 0.5306661128997803, 0.5313210487365723, 0.531694233417511, 0.5318021178245544, 0.5316619873046875, 0.5312917232513428, 0.5307097434997559, 0.5299348831176758, 0.5289862155914307, 0.5278829336166382, 0.5266441702842712, 0.5252889394760132, 0.5238359570503235, 0.5223034024238586, 0.5207090973854065, 0.46019721031188965, 0.46680784225463867, 0.47307711839675903, 0.47899362444877625, 0.4845479130744934, 0.48973265290260315, 0.49454259872436523, 0.49897444248199463, 0.5030269622802734, 0.5067009329795837, 0.5099988579750061, 0.5129252076148987, 0.5154862403869629, 0.5176898241043091, 0.5195453763008118, 0.5210639238357544, 0.5222578048706055, 0.5231406092643738, 0.523727297782898, 0.5240336060523987, 0.5240764021873474, 0.5238731503486633, 0.5234420895576477, 0.5228019952774048, 0.5219718813896179, 0.5209711194038391, 0.5198189616203308, 0.518534779548645, 0.5171375870704651, 0.51564621925354, 0.5140788555145264, 0.5124532580375671, 0.4538629651069641, 0.460418164730072, 0.4666307866573334, 0.47248920798301697, 0.4779839813709259, 0.48310765624046326, 0.4878549575805664, 0.4922226667404175, 0.49620962142944336, 0.4998166859149933, 0.5030466318130493, 0.5059040784835815, 0.5083955526351929, 0.5105292201042175, 0.5123147964477539, 0.5137636065483093, 0.514888346195221, 0.515703022480011, 0.5162227749824524, 0.5164638161659241, 0.5164432525634766, 0.5161789655685425, 0.515689492225647, 0.5149937272071838, 0.5141109824180603, 0.5130608081817627, 0.5118626356124878, 0.5105359554290771, 0.5090999007225037, 0.5075731873512268, 0.5059741139411926, 0.5043203830718994, 0.4476372003555298, 0.4541335701942444, 0.460286408662796, 0.46608394384384155, 0.47151657938957214, 0.47657686471939087, 0.4812595248222351, 0.485561341047287, 0.4894813001155853, 0.49302035570144653, 0.496181458234787, 0.49896952509880066, 0.5013912320137024, 0.5034550428390503, 0.5051710605621338, 0.5065507888793945, 0.5076074004173279, 0.5083551406860352, 0.508809506893158, 0.5089870095252991, 0.5089051723480225, 0.5085820555686951, 0.5080364346504211, 0.5072875618934631, 0.5063549280166626, 0.5052582621574402, 0.5040170550346375, 0.5026509165763855, 0.5011789798736572, 0.49962007999420166, 0.4979924261569977, 0.49631360173225403, 0.44151923060417175, 0.44795355200767517, 0.45404356718063354, 0.4597774147987366, 0.46514543890953064, 0.47014009952545166, 0.4747561514377594, 0.47899049520492554, 0.4828420877456665, 0.4863121509552002, 0.48940375447273254, 0.49212202429771423, 0.4944739043712616, 0.49646809697151184, 0.49811500310897827, 0.49942657351493835, 0.500416100025177, 0.5010982751846313, 0.5014889240264893, 0.5016048550605774, 0.501463770866394, 0.5010841488838196, 0.5004849433898926, 0.49968570470809937, 0.49870601296424866, 0.49756568670272827, 0.49628448486328125, 0.4948819875717163, 0.4933773875236511, 0.49178943037986755, 0.4901363253593445, 0.4884355366230011, 0.4355081021785736, 0.4418771266937256, 0.4479013681411743, 0.45356887578964233, 0.4588698446750641, 0.4637967646121979, 0.46834442019462585, 0.4725096821784973, 0.4762917757034302, 0.47969189286231995, 0.48271340131759644, 0.48536160588264465, 0.4876437187194824, 0.48956868052482605, 0.4911471903324127, 0.4923914968967438, 0.4933152198791504, 0.49393337965011597, 0.49426203966140747, 0.4943183660507202, 0.494120329618454, 0.49368664622306824, 0.49303653836250305, 0.4921896159648895, 0.49116575717926025, 0.4899848699569702, 0.4886667728424072, 0.48723104596138, 0.485696941614151, 0.4840831458568573, 0.4824077785015106, 0.4806881844997406, 0.4296025335788727, 0.4359031319618225, 0.44185870885849, 0.4474571943283081, 0.4526887834072113, 0.4575459361076355, 0.46202340722084045, 0.4661182463169098, 0.46982964873313904, 0.4731590747833252, 0.47611004114151, 0.4786880314350128, 0.48090049624443054, 0.48275673389434814, 0.48426762223243713, 0.4854457676410675, 0.4863050878047943, 0.48686084151268005, 0.48712942004203796, 0.48712828755378723, 0.48687565326690674, 0.48639044165611267, 0.4856920838356018, 0.48480039834976196, 0.4837353527545929, 0.48251697421073914, 0.4811651110649109, 0.47969937324523926, 0.4781389534473419, 0.47650250792503357, 0.47480806708335876, 0.4730727970600128, 0.4238009750843048, 0.43003007769584656, 0.43591415882110596, 0.44144105911254883, 0.446601003408432, 0.45138639211654663, 0.45579203963279724, 0.45981505513191223, 0.4634547829627991, 0.4667128324508667, 0.4695928394794464, 0.4721005856990814, 0.47424373030662537, 0.4760317802429199, 0.47747600078582764, 0.47858917713165283, 0.4793855547904968, 0.4798806607723236, 0.4800911843776703, 0.48003482818603516, 0.4797300100326538, 0.47919589281082153, 0.4784521162509918, 0.47751858830451965, 0.47641539573669434, 0.4751626253128052, 0.47378018498420715, 0.472287654876709, 0.47070416808128357, 0.46904832124710083, 0.467337965965271, 0.46559014916419983, 0.41810163855552673, 0.42425617575645447, 0.43006598949432373, 0.43551886081695557, 0.44060489535331726, 0.4453166425228119, 0.4496488571166992, 0.4535987973213196, 0.45716592669487, 0.46035194396972656, 0.46316075325012207, 0.46559831500053406, 0.4676724970340729, 0.4693931043148041, 0.4707716405391693, 0.4718211591243744, 0.4725561738014221, 0.47299250960350037, 0.47314706444740295, 0.4730377793312073, 0.4726833403110504, 0.4721030294895172, 0.4713166654109955, 0.47034427523612976, 0.4692060351371765, 0.46792206168174744, 0.4665122628211975, 0.4649961590766907, 0.46339288353919983, 0.4617208242416382, 0.4599977135658264, 0.45824047923088074, 0.4125025272369385, 0.4185795485973358, 0.42431238293647766, 0.4296887516975403, 0.43469879031181335, 0.4393349885940552, 0.44359228014945984, 0.44746795296669006, 0.4509615898132324, 0.4540750980377197, 0.4568125009536743, 0.45917999744415283, 0.4611856937408447, 0.46283966302871704, 0.46415358781814575, 0.46514084935188293, 0.4658161997795105, 0.466195672750473, 0.4662964642047882, 0.4661366641521454, 0.46573516726493835, 0.46511146426200867, 0.4642854332923889, 0.4632772207260132, 0.4621070623397827, 0.4607950747013092, 0.45936113595962524, 0.4578247368335724, 0.4562048614025116, 0.4545198082923889, 0.4527871608734131, 0.45102357864379883, 0.4070015251636505, 0.41299811005592346, 0.4186512529850006, 0.423948734998703, 0.4288806617259979, 0.43343958258628845, 0.43762049078941345, 0.4414207339286804, 0.44484010338783264, 0.4478806257247925, 0.4505465030670166, 0.45284417271614075, 0.4547819495201111, 0.45637011528015137, 0.45762065052986145, 0.45854711532592773, 0.4591645300388336, 0.45948919653892517, 0.45953842997550964, 0.45933061838150024, 0.45888474583625793, 0.4582204222679138, 0.4573577046394348, 0.456316739320755, 0.455117791891098, 0.45378100872039795, 0.4523261785507202, 0.450772762298584, 0.4491395354270935, 0.44744470715522766, 0.4457056224346161, 0.4439387619495392, 0.401596337556839, 0.40750962495803833, 0.41308048367500305, 0.4182967245578766, 0.4231484830379486, 0.42762836813926697, 0.4317314624786377, 0.43545523285865784, 0.43879958987236023, 0.44176673889160156, 0.4443610608577728, 0.4465892016887665, 0.4484596848487854, 0.44998300075531006, 0.4511713981628418, 0.45203861594200134, 0.4525999426841736, 0.45287182927131653, 0.4528718590736389, 0.4526185095310211, 0.4521309733390808, 0.45142892003059387, 0.4505324959754944, 0.4494618773460388, 0.4482373595237732, 0.44687896966934204, 0.4454065263271332, 0.44383928179740906, 0.44219592213630676, 0.44049447774887085, 0.43875208497047424, 0.43698495626449585, 0.39628466963768005, 0.4021117687225342, 0.40759778022766113, 0.41273045539855957, 0.4175000488758087, 0.4218992292881012, 0.4259231388568878, 0.42956942319869995, 0.43283811211586, 0.43573153018951416, 0.43825432658195496, 0.4404132664203644, 0.4422171413898468, 0.443676620721817, 0.4448041617870331, 0.4456137716770172, 0.44612088799476624, 0.4463421404361725, 0.4462953209877014, 0.44599902629852295, 0.4454725384712219, 0.44473567605018616, 0.44380852580070496, 0.44271138310432434, 0.4414644241333008, 0.4400877058506012, 0.4386008381843567, 0.43702301383018494, 0.4353727400302887, 0.4336678087711334, 0.4319251477718353, 0.430160790681839, 0.39106401801109314, 0.3968021869659424, 0.4022007882595062, 0.4072476923465729, 0.41193312406539917, 0.41624993085861206, 0.4201933741569519, 0.42376118898391724, 0.42695352435112, 0.4297729730606079, 0.4322242736816406, 0.43431442975997925, 0.43605244159698486, 0.4374491572380066, 0.4385172128677368, 0.43927085399627686, 0.43972569704055786, 0.4398985207080841, 0.4398072361946106, 0.4394705891609192, 0.4389079511165619, 0.4381391406059265, 0.43718433380126953, 0.4360637664794922, 0.4347975552082062, 0.43340569734573364, 0.43190765380859375, 0.43032243847846985, 0.42866840958595276, 0.42696309089660645, 0.4252232313156128, 0.42346450686454773, 0.38593199849128723, 0.39157846570014954, 0.3968871533870697, 0.4018460214138031, 0.4064454138278961, 0.4106782376766205, 0.4145399034023285, 0.4180282950401306, 0.4211437404155731, 0.4238889515399933, 0.42626887559890747, 0.4282907247543335, 0.429963618516922, 0.4312986731529236, 0.4323086738586426, 0.4330080449581146, 0.4334125518798828, 0.43353915214538574, 0.4334058463573456, 0.43303146958351135, 0.43243542313575745, 0.43163764476776123, 0.4306581914424896, 0.42951729893684387, 0.4282350242137909, 0.4268311858177185, 0.4253251552581787, 0.42373573780059814, 0.4220810532569885, 0.4203784465789795, 0.4186443090438843, 0.41689416766166687, 0.38088610768318176, 0.38643816113471985, 0.39165449142456055, 0.3965231478214264, 0.4010345935821533, 0.4051818549633026, 0.4089605212211609, 0.4123685956001282, 0.41540655493736267, 0.4180773198604584, 0.42038604617118835, 0.4223400056362152, 0.42394864559173584, 0.425223171710968, 0.4261765778064728, 0.4268234074115753, 0.42717957496643066, 0.42726215720176697, 0.42708927392959595, 0.426679790019989, 0.42605316638946533, 0.4252292811870575, 0.4242282211780548, 0.42307010293006897, 0.4217749238014221, 0.420362263917923, 0.41885140538215637, 0.41726091504096985, 0.415608674287796, 0.41391173005104065, 0.41218629479408264, 0.41044753789901733, 0.37592390179634094, 0.3813788890838623, 0.38650041818618774, 0.39127668738365173, 0.39569833874702454, 0.399758517742157, 0.40345293283462524, 0.40677982568740845, 0.40973979234695435, 0.41233596205711365, 0.414573609828949, 0.416460245847702, 0.41800543665885925, 0.41922059655189514, 0.4201188385486603, 0.4207148849964142, 0.42102473974227905, 0.4210655689239502, 0.4208555519580841, 0.4204135835170746, 0.4197591543197632, 0.4189121127128601, 0.4178924262523651, 0.4167201817035675, 0.4154151380062103, 0.4139968454837799, 0.4124842584133148, 0.4108957350254059, 0.4092489778995514, 0.40756070613861084, 0.40584680438041687, 0.4041222035884857, 0.371042937040329, 0.3763982057571411, 0.38142257928848267, 0.3861043453216553, 0.3904343545436859, 0.39440593123435974, 0.39801493287086487, 0.40125977993011475, 0.40414127707481384, 0.4066626727581024, 0.4088294506072998, 0.41064929962158203, 0.4121319353580475, 0.4132888913154602, 0.4141334891319275, 0.41468048095703125, 0.4149460196495056, 0.4149473309516907, 0.4147026240825653, 0.4142308235168457, 0.41355133056640625, 0.41268399357795715, 0.4116487205028534, 0.4104653298854828, 0.409153550863266, 0.40773266553878784, 0.4062213897705078, 0.4046379327774048, 0.4029995799064636, 0.4013228714466095, 0.3996233344078064, 0.3979155719280243])

# Y_RAND = np.array([0.818860650062561, 0.8079909086227417, 0.7963950037956238, 0.7840357422828674, 0.7708781957626343, 0.7568910121917725, 0.7420477271080017, 0.7263279557228088, 0.7097191214561462, 0.692218005657196, 0.6738321185112, 0.6545816659927368, 0.6345002055168152, 0.6136362552642822, 0.5920535922050476, 0.5698312520980835, 0.5470633506774902, 0.5238579511642456, 0.5003349781036377, 0.47662410140037537, 0.45286160707473755, 0.42918723821640015, 0.4057403802871704, 0.38265660405158997, 0.3600640296936035, 0.33808034658432007, 0.3168101906776428, 0.296343058347702, 0.276752233505249, 0.2580941319465637, 0.24040839076042175, 0.22371862828731537, 0.8260512351989746, 0.8153249621391296, 0.8038662075996399, 0.7916363477706909, 0.7785990834236145, 0.7647213339805603, 0.7499748468399048, 0.7343375086784363, 0.7177948951721191, 0.7003418803215027, 0.6819844245910645, 0.6627410054206848, 0.6426439881324768, 0.6217406988143921, 0.6000942587852478, 0.5777835249900818, 0.5549028515815735, 0.5315608382225037, 0.5078787207603455, 0.48398780822753906, 0.4600265622138977, 0.4361370801925659, 0.4124615788459778, 0.38913846015930176, 0.3662988543510437, 0.3440633714199066, 0.322539359331131, 0.30181899666786194, 0.2819778323173523, 0.2630743086338043, 0.24514970183372498, 0.22822901606559753, 0.83293217420578, 0.8223520517349243, 0.8110336661338806, 0.7989370822906494, 0.7860243916511536, 0.7722609639167786, 0.7576168179512024, 0.7420679926872253, 0.7255982160568237, 0.7082005739212036, 0.6898793578147888, 0.670651376247406, 0.6505476236343384, 0.6296143531799316, 0.6079139709472656, 0.5855250358581543, 0.5625419616699219, 0.5390741229057312, 0.5152437686920166, 0.49118390679359436, 0.4670349955558777, 0.4429415464401245, 0.41904836893081665, 0.39549675583839417, 0.37242069840431213, 0.3499436676502228, 0.328175812959671, 0.3072117567062378, 0.28712937235832214, 0.26798903942108154, 0.24983371794223785, 0.2326897531747818, 0.8395081758499146, 0.8290761113166809, 0.817900538444519, 0.8059401512145996, 0.7931554913520813, 0.7795104384422302, 0.7649731040000916, 0.7495179176330566, 0.733126699924469, 0.7157907485961914, 0.6975125670433044, 0.6783074140548706, 0.6582048535346985, 0.6372501254081726, 0.6155047416687012, 0.5930469632148743, 0.5699713230133057, 0.5463877320289612, 0.5224196314811707, 0.4982014298439026, 0.47387558221817017, 0.44958898425102234, 0.42548897862434387, 0.4017195999622345, 0.3784177303314209, 0.35570958256721497, 0.33370792865753174, 0.3125099837779999, 0.29219576716423035, 0.2728275954723358, 0.25445008277893066, 0.2370908409357071, 0.8457853198051453, 0.8355024456977844, 0.8244712948799133, 0.8126491904258728, 0.7999951839447021, 0.7864715456962585, 0.7720447778701782, 0.7566874027252197, 0.7403795123100281, 0.723110556602478, 0.7048813700675964, 0.6857055425643921, 0.6656112670898438, 0.6446425914764404, 0.622860312461853, 0.600342333316803, 0.577183187007904, 0.553493320941925, 0.5293972492218018, 0.5050308108329773, 0.48053839802742004, 0.45606911182403564, 0.4317728877067566, 0.40779638290405273, 0.3842792212963104, 0.36135032773017883, 0.3391251564025879, 0.3177032470703125, 0.2971667945384979, 0.27758002281188965, 0.2589890956878662, 0.24142290651798248, 0.8517706394195557, 0.8416374921798706, 0.8307516574859619, 0.8190690875053406, 0.8065475821495056, 0.7931476831436157, 0.7788342833518982, 0.7635780572891235, 0.7473573684692383, 0.7301599383354187, 0.7119846940040588, 0.6928437352180481, 0.6727638840675354, 0.651788055896759, 0.6299761533737183, 0.6074057221412659, 0.5841713547706604, 0.5603840351104736, 0.5361691117286682, 0.5116640329360962, 0.48701488971710205, 0.4623730778694153, 0.437890887260437, 0.4137176275253296, 0.3899955749511719, 0.36685633659362793, 0.34441784024238586, 0.3227819502353668, 0.30203303694725037, 0.28223705291748047, 0.2634417414665222, 0.24567721784114838, 0.8574722409248352, 0.8474885821342468, 0.8367482423782349, 0.8252059817314148, 0.8128180503845215, 0.7995434403419495, 0.7853453755378723, 0.7701928615570068, 0.7540624141693115, 0.7369400262832642, 0.7188229560852051, 0.6997215747833252, 0.6796614527702332, 0.658684253692627, 0.6368491649627686, 0.6142332553863525, 0.590931236743927, 0.5670543909072876, 0.5427292585372925, 0.5180944204330444, 0.4932979941368103, 0.4684932827949524, 0.4438350796699524, 0.4194751977920532, 0.3955584764480591, 0.37221911549568176, 0.3495774269104004, 0.32773762941360474, 0.30678603053092957, 0.2867903709411621, 0.26779982447624207, 0.24984575808048248, 0.8628988265991211, 0.8530639410018921, 0.842468798160553, 0.8310667872428894, 0.8188128471374512, 0.8056643605232239, 0.7915829420089722, 0.7765359282493591, 0.760498046875, 0.7434534430503845, 0.7253978252410889, 0.706339955329895, 0.6863039135932922, 0.6653304100036621, 0.643477737903595, 0.620822548866272, 0.597459614276886, 0.5735006928443909, 0.5490731000900269, 0.5243169069290161, 0.4993819296360016, 0.4744235873222351, 0.44959887862205505, 0.4250621497631073, 0.4009607136249542, 0.3774312734603882, 0.35459649562835693, 0.33256271481513977, 0.31141823530197144, 0.29123249650001526, 0.27205604314804077, 0.2539213001728058, 0.8680597543716431, 0.8583724498748779, 0.84792160987854, 0.83665931224823, 0.8245391249656677, 0.8115170001983643, 0.7975529432296753, 0.7826124429702759, 0.7666686177253723, 0.7497039437294006, 0.731712281703949, 0.7127009630203247, 0.692692756652832, 0.6717271208763123, 0.6498616933822632, 0.6271726489067078, 0.6037547588348389, 0.5797203183174133, 0.555197536945343, 0.5303277373313904, 0.5052624344825745, 0.48015913367271423, 0.45517703890800476, 0.43047288060188293, 0.40619638562202454, 0.3824866712093353, 0.3594686686992645, 0.33725079894065857, 0.31592321395874023, 0.295556902885437, 0.27620381116867065, 0.2578974664211273, 0.8729649186134338, 0.8634235262870789, 0.8531155586242676, 0.8419919610023499, 0.830004870891571, 0.8171087503433228, 0.8032619953155518, 0.788428544998169, 0.7725796699523926, 0.755696177482605, 0.7377703189849854, 0.7188079357147217, 0.6988304257392883, 0.6778761744499207, 0.6560020446777344, 0.6332838535308838, 0.609816312789917, 0.5857122540473938, 0.5611007213592529, 0.5361244678497314, 0.5109365582466125, 0.4856964349746704, 0.46056559681892395, 0.43570297956466675, 0.41126078367233276, 0.38738033175468445, 0.3641887605190277, 0.34179648756980896, 0.32029539346694946, 0.29975807666778564, 0.2802376449108124, 0.2617686092853546, 0.8776242733001709, 0.8682267665863037, 0.8580600023269653, 0.8470736145973206, 0.8352184295654297, 0.8224474787712097, 0.8087176084518433, 0.793990969657898, 0.7782374024391174, 0.7614358067512512, 0.7435770034790039, 0.7246652245521545, 0.7047205567359924, 0.6837804913520813, 0.6619009971618652, 0.6391575932502747, 0.6156450510025024, 0.5914766192436218, 0.5667822360992432, 0.5417059659957886, 0.5164024829864502, 0.49103325605392456, 0.46576178073883057, 0.4407493472099304, 0.41615036129951477, 0.39210838079452515, 0.368752658367157, 0.3461954593658447, 0.32453036308288574, 0.3038313686847687, 0.2841528058052063, 0.2655300796031952, 0.8820482492446899, 0.8727922439575195, 0.8627645373344421, 0.8519135117530823, 0.8401886820793152, 0.8275416493415833, 0.8139276504516602, 0.7993073463439941, 0.7836487293243408, 0.7669292688369751, 0.7491380572319031, 0.730277955532074, 0.7103677988052368, 0.6894440054893494, 0.6675618290901184, 0.6447965502738953, 0.6212429404258728, 0.5970147252082825, 0.5722427368164062, 0.5470723509788513, 0.5216598510742188, 0.4961685240268707, 0.4707641303539276, 0.445609986782074, 0.4208627939224243, 0.3966681659221649, 0.3731573522090912, 0.3504444658756256, 0.3286246359348297, 0.3077731728553772, 0.28794553875923157, 0.26917797327041626, 0.8862470984458923, 0.8771300315856934, 0.8672389984130859, 0.8565211892127991, 0.8449248671531677, 0.8324000239372253, 0.8189005851745605, 0.8043854832649231, 0.7888212203979492, 0.7721835374832153, 0.7544600367546082, 0.7356521487236023, 0.7157775163650513, 0.6948714852333069, 0.6729888319969177, 0.6502043008804321, 0.6266129016876221, 0.602328896522522, 0.5774839520454407, 0.5522246956825256, 0.5267091393470764, 0.5011024475097656, 0.47557222843170166, 0.45028409361839294, 0.42539680004119873, 0.40105801820755005, 0.3774009048938751, 0.35454127192497253, 0.3325757384300232, 0.3115808367729187, 0.29161301255226135, 0.27270933985710144, 0.890231192111969, 0.881250262260437, 0.8714932799339294, 0.8609063029289246, 0.8494362235069275, 0.8370317220687866, 0.8236451148986816, 0.8092338442802429, 0.7937628030776978, 0.7772061824798584, 0.7595500349998474, 0.7407944202423096, 0.7209557890892029, 0.7000685930252075, 0.6781868934631348, 0.6553853154182434, 0.6317589282989502, 0.6074224710464478, 0.5825086832046509, 0.5571653842926025, 0.5315521955490112, 0.5058360695838928, 0.48018679022789, 0.45477190613746643, 0.4297522306442261, 0.40527743101119995, 0.38148242235183716, 0.35848468542099, 0.3363821506500244, 0.3152525722980499, 0.29515329003334045, 0.2761220335960388, 0.8940107226371765, 0.8851630687713623, 0.8755372762680054, 0.8650785088539124, 0.8537322878837585, 0.8414458632469177, 0.8281701803207397, 0.813861072063446, 0.798481822013855, 0.7820051312446594, 0.7644155621528625, 0.7457119226455688, 0.7259093523025513, 0.7050414681434631, 0.6831619143486023, 0.6603448987007141, 0.6366857290267944, 0.6122997999191284, 0.5873206257820129, 0.5618975162506104, 0.5361916422843933, 0.510371744632721, 0.48460957407951355, 0.4590747058391571, 0.4339299201965332, 0.40932679176330566, 0.38540199398994446, 0.3622744083404541, 0.3400433361530304, 0.318787544965744, 0.29856520891189575, 0.27941471338272095, 0.8975958824157715, 0.8888782858848572, 0.8793808817863464, 0.8690475225448608, 0.8578225374221802, 0.8456518650054932, 0.8324848413467407, 0.8182759284973145, 0.8029868006706238, 0.7865886688232422, 0.7690646052360535, 0.7504121661186218, 0.7306453585624695, 0.7097970247268677, 0.6879202127456665, 0.665088951587677, 0.6413988471031189, 0.6169657707214355, 0.5919243693351746, 0.5664252042770386, 0.5406310558319092, 0.5147125124931335, 0.48884317278862, 0.46319466829299927, 0.4379316568374634, 0.4132075309753418, 0.3891605734825134, 0.3659110963344574, 0.343559592962265, 0.32218578457832336, 0.30184856057167053, 0.28258687257766724, 0.9009963870048523, 0.8924057483673096, 0.8830336332321167, 0.8728228807449341, 0.8617163896560669, 0.8496589064598083, 0.83659827709198, 0.8224874138832092, 0.8072865009307861, 0.7909652590751648, 0.773505449295044, 0.7549031376838684, 0.7351715564727783, 0.7143425345420837, 0.6924686431884766, 0.6696240305900574, 0.6459043025970459, 0.6214261651039124, 0.596325159072876, 0.570753276348114, 0.5448748469352722, 0.518862247467041, 0.4928911030292511, 0.46713489294052124, 0.4417601227760315, 0.4169218838214874, 0.3927600383758545, 0.36939623951911926, 0.346932053565979, 0.32544809579849243, 0.3050038814544678, 0.2856387794017792, 0.9042218327522278, 0.8957549333572388, 0.886505126953125, 0.8764139413833618, 0.8654232621192932, 0.8534762859344482, 0.8405195474624634, 0.8265044689178467, 0.811389684677124, 0.7951435446739197, 0.7777463793754578, 0.7591930627822876, 0.7394957542419434, 0.7186855673789978, 0.6968147158622742, 0.673957109451294, 0.6502088308334351, 0.6256872415542603, 0.60052889585495, 0.574887216091156, 0.5489280819892883, 0.5228257775306702, 0.49675771594047546, 0.4708992540836334, 0.44541874527931213, 0.4204729199409485, 0.39620310068130493, 0.37273216247558594, 0.35016271471977234, 0.32857608795166016, 0.30803239345550537, 0.28857138752937317, 0.9072815179824829, 0.8989351391792297, 0.889804482460022, 0.8798299431800842, 0.8689521551132202, 0.8571130633354187, 0.844257652759552, 0.8303359746932983, 0.815305233001709, 0.799132227897644, 0.7817959785461426, 0.763290286064148, 0.7436261177062988, 0.7228341102600098, 0.7009660005569458, 0.6780956983566284, 0.6543195843696594, 0.6297557950019836, 0.604542076587677, 0.5788331627845764, 0.5527966022491455, 0.5266084671020508, 0.5004479289054871, 0.4744923710823059, 0.44891175627708435, 0.423864483833313, 0.39949318766593933, 0.37592193484306335, 0.3532542288303375, 0.3315720856189728, 0.3109361231327057, 0.2913863956928253, 0.910184383392334, 0.9019553065299988, 0.8929407000541687, 0.8830798268318176, 0.8723121285438538, 0.8605781197547913, 0.8478214740753174, 0.8339908123016357, 0.8190417885780334, 0.8029399514198303, 0.785662829875946, 0.7672032117843628, 0.7475710511207581, 0.7267963886260986, 0.7049304842948914, 0.6820474863052368, 0.6582440733909607, 0.6336391568183899, 0.6083717346191406, 0.5825977921485901, 0.5564866662025452, 0.5302162170410156, 0.5039674639701843, 0.47791945934295654, 0.4522440433502197, 0.4271010160446167, 0.4026343822479248, 0.3789692521095276, 0.35620996356010437, 0.33443909883499146, 0.31371772289276123, 0.2940860390663147, 0.9129388928413391, 0.9048240184783936, 0.8959224224090576, 0.8861722350120544, 0.8755117058753967, 0.8638801574707031, 0.8512197136878967, 0.8374776244163513, 0.8226081728935242, 0.806575357913971, 0.7893555760383606, 0.770940363407135, 0.7513389587402344, 0.7305805683135986, 0.7087163925170898, 0.685820460319519, 0.6619901061058044, 0.6373449563980103, 0.6120251417160034, 0.5861882567405701, 0.5600051879882812, 0.53365558385849, 0.5073224306106567, 0.4811864197254181, 0.4554210901260376, 0.4301877021789551, 0.4056314527988434, 0.38187849521636963, 0.35903385281562805, 0.3371807038784027, 0.3163803517818451, 0.2966732382774353, 0.9155533313751221, 0.9075495600700378, 0.8987578749656677, 0.889115571975708, 0.878559410572052, 0.8670275807380676, 0.8544608354568481, 0.8408048748970032, 0.8260127305984497, 0.810046911239624, 0.7928826212882996, 0.7745102643966675, 0.7549382448196411, 0.7341951727867126, 0.7123319506645203, 0.6894229054450989, 0.6655657291412354, 0.6408810019493103, 0.6155100464820862, 0.5896119475364685, 0.5633593201637268, 0.5369336009025574, 0.5105195045471191, 0.48429957032203674, 0.4584488868713379, 0.4331301152706146, 0.4084896445274353, 0.38465452194213867, 0.3617304563522339, 0.3398010730743408, 0.3189278841018677, 0.29915139079093933, 0.9180354475975037, 0.9101397395133972, 0.9014551043510437, 0.8919177651405334, 0.8814632892608643, 0.870028555393219, 0.8575530052185059, 0.8439809083938599, 0.8292638659477234, 0.8133630156517029, 0.796252429485321, 0.7779213190078735, 0.7583773136138916, 0.7376485466957092, 0.7157856225967407, 0.6928630471229553, 0.6689791679382324, 0.6442553997039795, 0.6188344359397888, 0.5928767323493958, 0.5665566921234131, 0.5400575399398804, 0.5135657787322998, 0.4872656762599945, 0.46133387088775635, 0.4359344244003296, 0.41121476888656616, 0.38730278611183167, 0.3643048405647278, 0.3423048257827759, 0.32136452198028564, 0.30152443051338196, 0.9203927516937256, 0.9126020669937134, 0.904021680355072, 0.8945865035057068, 0.8842311501502991, 0.8728910088539124, 0.8605042099952698, 0.8470137119293213, 0.8323696255683899, 0.8165318965911865, 0.799473226070404, 0.7811818718910217, 0.7616645693778992, 0.7409490346908569, 0.7190857529640198, 0.6961493492126465, 0.672238826751709, 0.6474765539169312, 0.6220064163208008, 0.5959905982017517, 0.5696051716804504, 0.5430350303649902, 0.5164686441421509, 0.4900919198989868, 0.4640829563140869, 0.4386071562767029, 0.4138129949569702, 0.3898291289806366, 0.3667623996734619, 0.34469708800315857, 0.32369503378868103, 0.303796648979187, 0.9226322174072266, 0.9149437546730042, 0.9064647555351257, 0.8971291780471802, 0.8868703246116638, 0.8756223320960999, 0.8633220791816711, 0.849911093711853, 0.8353379964828491, 0.8195614814758301, 0.802553117275238, 0.7843000292778015, 0.7648082971572876, 0.7441050410270691, 0.7222407460212708, 0.6992901563644409, 0.6753530502319336, 0.6505526900291443, 0.6250344514846802, 0.5989618301391602, 0.5725128054618835, 0.5458741188049316, 0.5192359089851379, 0.4927857518196106, 0.466703325510025, 0.44115522503852844, 0.41629090905189514, 0.3922397494316101, 0.36910906434059143, 0.3469833433628082, 0.32592448592185974, 0.3059728443622589, 0.9247605204582214, 0.9171715378761292, 0.9087913036346436, 0.8995527029037476, 0.8893879652023315, 0.8782298564910889, 0.8660139441490173, 0.8526803851127625, 0.838176429271698, 0.8224595189094543, 0.8054999113082886, 0.7872838377952576, 0.7678165435791016, 0.7471247911453247, 0.7252588272094727, 0.7022939324378967, 0.6783302426338196, 0.6534923911094666, 0.627926766872406, 0.6017987728118896, 0.575287938117981, 0.5485828518867493, 0.5218753814697266, 0.49535495042800903, 0.4692024886608124, 0.4435858130455017, 0.4186553955078125, 0.3945412039756775, 0.3713510036468506, 0.34916943311691284, 0.3280583620071411, 0.3080580532550812, 0.9267839193344116, 0.9192917943000793, 0.911007821559906, 0.9018638134002686, 0.8917908072471619, 0.8807204365730286, 0.8685868978500366, 0.8553288578987122, 0.8408923745155334, 0.8252334594726562, 0.8083212375640869, 0.7901409864425659, 0.7706972360610962, 0.7500162720680237, 0.7281482815742493, 0.7051688432693481, 0.681178867816925, 0.6563039422035217, 0.6306919455528259, 0.6045098304748535, 0.5779388546943665, 0.5511694550514221, 0.5243953466415405, 0.49780741333961487, 0.4715880751609802, 0.4459063708782196, 0.42091357707977295, 0.3967403173446655, 0.3734946846961975, 0.3512614369392395, 0.33010241389274597, 0.3100576400756836, 0.9287084937095642, 0.9213106036186218, 0.9131203889846802, 0.904068648815155, 0.8940852880477905, 0.8831006288528442, 0.8710474967956543, 0.8578633069992065, 0.8434927463531494, 0.8278905153274536, 0.81102454662323, 0.7928791642189026, 0.7734581232070923, 0.7527874112129211, 0.7309171557426453, 0.707923173904419, 0.6839072108268738, 0.6589958667755127, 0.6333383917808533, 0.6071034669876099, 0.5804740190505981, 0.5536423921585083, 0.5268039703369141, 0.5001512169837952, 0.4738680422306061, 0.4481245279312134, 0.42307284474372864, 0.3988441526889801, 0.37554681301116943, 0.3532657325267792, 0.33206260204315186, 0.31197723746299744, 0.930539608001709, 0.9232336282730103, 0.915134847164154, 0.9061732292175293, 0.8962773680686951, 0.8853765726089478, 0.8734021782875061, 0.8602902889251709, 0.8459842801094055, 0.8304375410079956, 0.8136168122291565, 0.7955055236816406, 0.7761067748069763, 0.7554459571838379, 0.733573317527771, 0.7105649709701538, 0.6865234375, 0.6615763902664185, 0.6358745694160461, 0.6095881462097168, 0.5829018950462341, 0.5560100078582764, 0.529109537601471, 0.5023945569992065, 0.47605034708976746, 0.4502480924129486, 0.4251406788825989, 0.40085989236831665, 0.3775143027305603, 0.35518887639045715, 0.3339451551437378, 0.3138226866722107, 0.9322826862335205, 0.9250661730766296, 0.917056679725647, 0.9081830978393555, 0.8983729481697083, 0.8875541687011719, 0.8756569623947144, 0.8626160621643066, 0.8483734130859375, 0.8328811526298523, 0.8161049485206604, 0.7980272769927979, 0.7786503434181213, 0.7579993009567261, 0.7361243963241577, 0.7131021618843079, 0.6890356540679932, 0.6640537977218628, 0.6383087635040283, 0.6119722723960876, 0.5852310061454773, 0.5582807660102844, 0.5313204526901245, 0.5045456886291504, 0.4781431257724762, 0.45228487253189087, 0.4271247386932373, 0.40279489755630493, 0.3794041872024536, 0.35703757405281067, 0.33575639128685, 0.3155999481678009, 0.9339424967765808, 0.9268133044242859, 0.9188910126686096, 0.9101034998893738, 0.9003772735595703, 0.8896390199661255, 0.8778176307678223, 0.8648464679718018, 0.8506662249565125, 0.8352277278900146, 0.818495512008667, 0.8004509806632996, 0.7810958623886108, 0.7604548335075378, 0.7385780215263367, 0.7155423760414124, 0.6914517283439636, 0.6664361953735352, 0.6406491994857788, 0.614264190196991, 0.5874695777893066, 0.560463011264801, 0.5334450602531433, 0.5066128969192505, 0.480154424905777, 0.45424285531044006, 0.42903271317481995, 0.40465661883354187, 0.3812236189842224, 0.3588186204433441, 0.3375028371810913, 0.3173151910305023, 0.9355236887931824, 0.9284796714782715, 0.920642614364624, 0.9119394421577454, 0.9022954702377319, 0.891636312007904, 0.8798894882202148, 0.8669871091842651, 0.8528684377670288, 0.8374831676483154, 0.8207946419715881, 0.8027833104133606, 0.7834501266479492, 0.7628194093704224, 0.7409413456916809, 0.717893123626709, 0.693779468536377, 0.6687314510345459, 0.6429039835929871, 0.6164721250534058, 0.5896260738372803, 0.5625651478767395, 0.5354917049407959, 0.5086044073104858, 0.48209241032600403, 0.4561299979686737, 0.4308723509311676, 0.4064525067806244, 0.38297978043556213, 0.36053892970085144, 0.3391910195350647, 0.3189746141433716])
# # union_volume_32 = 0.014813416637480259

# # n = 64
# X_RAND = np.array([0.7332136631011963, 0.732893705368042, 0.7325899600982666, 0.732302188873291, 0.7320299744606018, 0.7317730784416199, 0.7315310835838318, 0.7313036918640137, 0.7310904860496521, 0.7308910489082336, 0.7307050824165344, 0.7305321097373962, 0.7303717136383057, 0.7302234768867493, 0.7300869226455688, 0.729961633682251, 0.7298471331596375, 0.7297430038452148, 0.7296487092971802, 0.72956383228302, 0.7294878959655762, 0.7294204235076904, 0.7293608784675598, 0.7293088436126709, 0.7292637825012207, 0.7292252779006958, 0.7291927933692932, 0.7291658520698547, 0.7291439771652222, 0.7291267514228821, 0.7291136384010315, 0.729104220867157, 0.7290979623794556, 0.7290945053100586, 0.7290933132171631, 0.7290939688682556, 0.7290960550308228, 0.7290991544723511, 0.7291027903556824, 0.7291066646575928, 0.7291101813316345, 0.7291131615638733, 0.7291150093078613, 0.7291155457496643, 0.7291142344474792, 0.729110836982727, 0.7291049957275391, 0.7290962934494019, 0.7290844917297363, 0.7290692329406738, 0.7290502190589905, 0.7290271520614624, 0.7289997935295105, 0.7289678454399109, 0.7289310097694397, 0.7288891077041626, 0.7288419008255005, 0.7287890911102295, 0.7287305593490601, 0.7286660671234131, 0.7285954356193542, 0.7285184264183044, 0.7284349799156189, 0.7283447980880737, 0.7255190014839172, 0.7251472473144531, 0.7247922420501709, 0.7244537472724915, 0.7241314053535461, 0.7238249778747559, 0.7235340476036072, 0.7232583165168762, 0.7229974865913391, 0.7227510809898376, 0.7225187420845032, 0.7223001718521118, 0.7220948338508606, 0.7219023704528809, 0.7217223644256592, 0.7215543985366821, 0.7213979959487915, 0.7212527394294739, 0.721118152141571, 0.7209937572479248, 0.7208791971206665, 0.7207739353179932, 0.7206774950027466, 0.7205893993377686, 0.7205092906951904, 0.7204365730285645, 0.7203708291053772, 0.7203116416931152, 0.7202584743499756, 0.7202109098434448, 0.7201684713363647, 0.7201306819915771, 0.7200971841812134, 0.7200674414634705, 0.7200410962104797, 0.720017671585083, 0.7199967503547668, 0.7199779152870178, 0.7199607491493225, 0.7199448347091675, 0.7199298739433289, 0.7199153900146484, 0.7199010252952576, 0.7198864221572876, 0.7198712229728699, 0.7198550701141357, 0.7198377251625061, 0.7198187112808228, 0.7197977900505066, 0.719774603843689, 0.7197489738464355, 0.7197204828262329, 0.719688892364502, 0.7196540236473083, 0.7196155190467834, 0.7195731997489929, 0.7195267677307129, 0.7194761037826538, 0.7194209098815918, 0.7193610072135925, 0.7192962765693665, 0.7192264199256897, 0.7191513776779175, 0.7190709114074707, 0.717685341835022, 0.7172598242759705, 0.716851532459259, 0.7164603471755981, 0.7160858511924744, 0.7157278060913086, 0.715385913848877, 0.7150598168373108, 0.7147492170333862, 0.7144536972045898, 0.714172899723053, 0.7139065265655518, 0.7136541604995728, 0.7134153246879578, 0.7131897211074829, 0.71297687292099, 0.7127764225006104, 0.712587833404541, 0.7124108076095581, 0.7122448086738586, 0.7120894193649292, 0.7119442224502563, 0.7118087410926819, 0.7116825580596924, 0.7115651965141296, 0.7114562392234802, 0.7113551497459412, 0.7112616300582886, 0.7111751437187195, 0.7110952734947205, 0.7110215425491333, 0.7109535336494446, 0.7108908295631409, 0.7108330726623535, 0.7107797265052795, 0.7107304334640503, 0.7106847167015076, 0.7106423377990723, 0.7106027007102966, 0.7105656266212463, 0.7105305790901184, 0.7104972004890442, 0.7104652523994446, 0.7104342579841614, 0.7104038596153259, 0.7103738784790039, 0.7103438377380371, 0.7103134393692017, 0.7102824449539185, 0.7102505564689636, 0.7102174162864685, 0.710182785987854, 0.710146427154541, 0.7101080417633057, 0.7100673913955688, 0.7100242376327515, 0.7099783420562744, 0.7099295258522034, 0.7098775506019592, 0.7098222374916077, 0.7097634077072144, 0.709700882434845, 0.7096344828605652, 0.7095640301704407, 0.7097153067588806, 0.7092341780662537, 0.7087708115577698, 0.7083249688148499, 0.7078964114189148, 0.7074848413467407, 0.7070900201797485, 0.7067115306854248, 0.7063491344451904, 0.7060025334358215, 0.7056713104248047, 0.7053551077842712, 0.7050535678863525, 0.7047663331031799, 0.7044930458068848, 0.7042332887649536, 0.703986644744873, 0.7037527561187744, 0.7035310864448547, 0.7033213973045349, 0.703123152256012, 0.7029359340667725, 0.7027593851089478, 0.7025930285453796, 0.7024363875389099, 0.7022891640663147, 0.702150821685791, 0.7020210027694702, 0.7018992304801941, 0.7017850875854492, 0.7016782164573669, 0.7015781402587891, 0.7014844417572021, 0.7013967633247375, 0.7013146877288818, 0.7012377977371216, 0.7011657357215881, 0.7010980844497681, 0.7010345458984375, 0.7009746432304382, 0.7009180784225464, 0.7008644938468933, 0.7008134722709656, 0.7007648348808289, 0.700718104839325, 0.70067298412323, 0.7006292343139648, 0.7005864977836609, 0.700544536113739, 0.7005029320716858, 0.7004615664482117, 0.700420081615448, 0.7003781795501709, 0.7003357410430908, 0.7002924680709839, 0.7002480626106262, 0.7002024054527283, 0.7001551985740662, 0.7001063227653503, 0.7000555396080017, 0.7000026702880859, 0.6999474763870239, 0.6998898983001709, 0.6998297572135925, 0.7016119956970215, 0.7010734677314758, 0.7005532383918762, 0.7000510096549988, 0.6995665431022644, 0.699099600315094, 0.6986499428749084, 0.6982172727584839, 0.6978012323379517, 0.6974015831947327, 0.6970179080963135, 0.696649968624115, 0.6962974071502686, 0.6959598660469055, 0.6956369280815125, 0.6953282356262207, 0.6950334906578064, 0.6947522163391113, 0.6944840550422668, 0.6942286491394043, 0.6939855813980103, 0.6937544345855713, 0.693534791469574, 0.6933262944221497, 0.6931285262107849, 0.6929410696029663, 0.6927635669708252, 0.6925955414772034, 0.6924366354942322, 0.6922864317893982, 0.692144513130188, 0.6920105814933777, 0.6918841600418091, 0.6917648315429688, 0.6916523575782776, 0.6915462613105774, 0.6914461851119995, 0.6913517713546753, 0.6912627220153809, 0.691178560256958, 0.6910990476608276, 0.6910238265991211, 0.6909525990486145, 0.6908849477767944, 0.6908206939697266, 0.6907594203948975, 0.690700888633728, 0.6906447410583496, 0.6905907988548279, 0.6905387043952942, 0.6904882788658142, 0.6904391646385193, 0.6903912425041199, 0.6903440952301025, 0.6902976632118225, 0.6902516484260559, 0.6902058720588684, 0.6901600360870361, 0.6901140809059143, 0.690067708492279, 0.690020740032196, 0.6899730563163757, 0.6899244785308838, 0.6898748278617859, 0.6933785676956177, 0.6927810907363892, 0.6922022700309753, 0.6916419863700867, 0.6910999417304993, 0.6905759572982788, 0.6900697946548462, 0.6895810961723328, 0.6891096830368042, 0.6886551976203918, 0.6882174015045166, 0.6877959370613098, 0.6873904466629028, 0.6870006918907166, 0.6866263151168823, 0.6862668991088867, 0.6859221458435059, 0.6855916976928711, 0.6852751970291138, 0.6849722266197205, 0.6846824288368225, 0.684405505657196, 0.6841409802436829, 0.6838885545730591, 0.6836477518081665, 0.683418333530426, 0.6831997632980347, 0.6829918026924133, 0.6827939748764038, 0.6826059222221375, 0.6824273467063904, 0.6822578310966492, 0.6820969581604004, 0.6819444894790649, 0.6817999482154846, 0.6816630959510803, 0.6815334558486938, 0.6814107894897461, 0.6812947392463684, 0.6811850070953369, 0.6810811758041382, 0.6809830665588379, 0.6808902621269226, 0.6808024644851685, 0.6807194948196411, 0.6806409358978271, 0.6805665493011475, 0.680496096611023, 0.6804292798042297, 0.6803658604621887, 0.6803056001663208, 0.6802482008934021, 0.6801934838294983, 0.6801411509513855, 0.6800910830497742, 0.6800430417060852, 0.6799967288970947, 0.6799520254135132, 0.6799087524414062, 0.6798666715621948, 0.6798256039619446, 0.679785430431366, 0.6797459721565247, 0.6797069907188416, 0.6850185990333557, 0.6843606233596802, 0.6837217807769775, 0.6831018924713135, 0.6825007796287537, 0.6819181442260742, 0.6813538670539856, 0.6808075904846191, 0.6802791357040405, 0.6797682642936707, 0.6792746186256409, 0.6787979602813721, 0.6783379316329956, 0.6778942942619324, 0.6774667501449585, 0.6770548820495605, 0.6766584515571594, 0.6762771010398865, 0.6759105324745178, 0.67555832862854, 0.6752201318740845, 0.674895703792572, 0.6745845675468445, 0.6742864847183228, 0.6740009784698486, 0.6737278699874878, 0.6734666228294373, 0.6732170581817627, 0.6729786396026611, 0.6727511882781982, 0.6725342869758606, 0.6723276376724243, 0.672130823135376, 0.6719435453414917, 0.6717655062675476, 0.671596348285675, 0.6714357733726501, 0.671283483505249, 0.671139121055603, 0.6710023880004883, 0.6708730459213257, 0.6707506775856018, 0.6706351637840271, 0.6705261468887329, 0.6704232692718506, 0.6703264117240906, 0.670235276222229, 0.6701495051383972, 0.6700689792633057, 0.6699934005737305, 0.6699225306510925, 0.6698561310768127, 0.6697940230369568, 0.6697359681129456, 0.6696817874908447, 0.6696311831474304, 0.6695840954780579, 0.6695402264595032, 0.669499397277832, 0.6694614887237549, 0.6694262623786926, 0.6693935990333557, 0.669363260269165, 0.6693351864814758, 0.6765359044075012, 0.6758161187171936, 0.6751158833503723, 0.6744350790977478, 0.6737734079360962, 0.6731307506561279, 0.6725069284439087, 0.6719016432762146, 0.6713147163391113, 0.6707459092140198, 0.6701949238777161, 0.6696615815162659, 0.6691455245018005, 0.668646514415741, 0.6681642532348633, 0.6676984429359436, 0.6672487854957581, 0.6668149828910828, 0.6663967370986938, 0.6659936904907227, 0.6656055450439453, 0.6652320623397827, 0.6648727655410767, 0.6645274758338928, 0.6641957759857178, 0.6638773679733276, 0.6635719537734985, 0.6632792353630066, 0.6629987955093384, 0.6627303957939148, 0.662473738193512, 0.6622284650802612, 0.661994218826294, 0.6617708206176758, 0.6615578532218933, 0.6613550782203674, 0.661162257194519, 0.6609789729118347, 0.6608049869537354, 0.6606401205062866, 0.6604839563369751, 0.6603363156318665, 0.6601969599723816, 0.6600655317306519, 0.6599419116973877, 0.6598257422447205, 0.6597168445587158, 0.6596149206161499, 0.6595198512077332, 0.6594312787055969, 0.6593491435050964, 0.659273087978363, 0.6592029929161072, 0.6591386795043945, 0.6590798497200012, 0.6590263843536377, 0.6589780449867249, 0.6589347124099731, 0.6588961482048035, 0.6588622331619263, 0.6588327288627625, 0.6588075160980225, 0.6587864756584167, 0.6587693691253662, 0.6679345965385437, 0.6671518683433533, 0.6663890480995178, 0.6656460165977478, 0.6649225950241089, 0.6642186641693115, 0.6635340452194214, 0.6628684997558594, 0.6622217893600464, 0.6615937352180481, 0.6609841585159302, 0.6603927612304688, 0.6598193049430847, 0.659263551235199, 0.6587252616882324, 0.6582040786743164, 0.6576998233795166, 0.6572121977806091, 0.6567409038543701, 0.6562857031822205, 0.6558462381362915, 0.6554222106933594, 0.6550134420394897, 0.6546195149421692, 0.6542401909828186, 0.6538752317428589, 0.6535242199897766, 0.6531869769096375, 0.6528632044792175, 0.6525525450706482, 0.6522547602653503, 0.6519695520401001, 0.6516966223716736, 0.6514357924461365, 0.6511867046356201, 0.6509490609169006, 0.6507226824760437, 0.6505072712898254, 0.6503025889396667, 0.6501083970069885, 0.6499243974685669, 0.6497504115104675, 0.6495861411094666, 0.6494314670562744, 0.6492860317230225, 0.6491496562957764, 0.6490221619606018, 0.6489032506942749, 0.6487928628921509, 0.6486906409263611, 0.6485965251922607, 0.648510217666626, 0.6484315395355225, 0.6483603715896606, 0.6482964754104614, 0.6482396721839905, 0.6481897830963135, 0.6481466889381409, 0.6481101512908936, 0.648080050945282, 0.648056149482727, 0.648038387298584, 0.6480265259742737, 0.6480204463005066, 0.6592190861701965, 0.6583723425865173, 0.6575459241867065, 0.6567397117614746, 0.6559534668922424, 0.65518718957901, 0.654440701007843, 0.6537137031555176, 0.6530061364173889, 0.6523178219795227, 0.6516484618186951, 0.6509978771209717, 0.6503658890724182, 0.6497521996498108, 0.6491566896438599, 0.6485790014266968, 0.6480189561843872, 0.647476315498352, 0.6469507813453674, 0.646442174911499, 0.6459501385688782, 0.6454744935035706, 0.6450149416923523, 0.6445712447166443, 0.6441431045532227, 0.6437302827835083, 0.6433325409889221, 0.642949640750885, 0.6425812244415283, 0.6422271728515625, 0.6418870687484741, 0.6415608525276184, 0.6412481069564819, 0.6409487128257751, 0.6406623125076294, 0.6403887867927551, 0.6401278972625732, 0.6398792862892151, 0.6396428942680359, 0.639418363571167, 0.6392055749893188, 0.6390042304992676, 0.6388142108917236, 0.6386352777481079, 0.6384672522544861, 0.6383098363876343, 0.6381629705429077, 0.6380264163017273, 0.6378999352455139, 0.6377834677696228, 0.6376767158508301, 0.6375795602798462, 0.637491762638092, 0.6374132633209229, 0.6373438835144043, 0.6372833251953125, 0.6372315287590027, 0.6371883749961853, 0.6371535658836365, 0.6371271014213562, 0.6371086835861206, 0.6370981931686401, 0.6370955109596252, 0.6371004581451416, 0.6503940224647522, 0.6494824886322021, 0.6485915780067444, 0.6477212905883789, 0.6468714475631714, 0.6460419297218323, 0.645232617855072, 0.6444433331489563, 0.6436740159988403, 0.6429243683815002, 0.6421943306922913, 0.6414836049079895, 0.6407921314239502, 0.6401196122169495, 0.639465868473053, 0.6388306617736816, 0.6382138729095459, 0.6376152038574219, 0.6370344758033752, 0.6364713907241821, 0.635925829410553, 0.6353975534439087, 0.6348862051963806, 0.634391725063324, 0.6339137554168701, 0.6334521770477295, 0.6330066323280334, 0.6325770616531372, 0.6321630477905273, 0.6317645311355591, 0.6313812732696533, 0.6310129761695862, 0.6306594610214233, 0.6303205490112305, 0.6299960017204285, 0.629685640335083, 0.629389226436615, 0.6291065812110901, 0.628837525844574, 0.6285818815231323, 0.628339409828186, 0.6281099915504456, 0.6278933882713318, 0.6276894807815552, 0.6274980306625366, 0.6273189187049866, 0.6271520256996155, 0.6269970536231995, 0.6268539428710938, 0.626722514629364, 0.6266026496887207, 0.6264940500259399, 0.6263967156410217, 0.6263104677200317, 0.6262350678443909, 0.6261704564094543, 0.6261164546012878, 0.626072883605957, 0.6260396242141724, 0.6260164976119995, 0.6260033249855042, 0.6260000467300415, 0.6260064840316772, 0.6260223984718323, 0.641464352607727, 0.6404873728752136, 0.6395313739776611, 0.6385962963104248, 0.6376821398735046, 0.6367887258529663, 0.6359159350395203, 0.6350637078285217, 0.6342318058013916, 0.6334202289581299, 0.6326287388801575, 0.63185715675354, 0.6311054229736328, 0.6303732395172119, 0.6296605467796326, 0.6289671063423157, 0.6282927989959717, 0.6276373267173767, 0.6270005702972412, 0.6263823509216309, 0.6257824301719666, 0.625200629234314, 0.6246367692947388, 0.6240906715393066, 0.6235620975494385, 0.6230508685112, 0.6225568056106567, 0.6220797300338745, 0.6216194033622742, 0.6211756467819214, 0.6207483410835266, 0.6203372478485107, 0.6199421882629395, 0.6195629835128784, 0.6191994547843933, 0.6188514828681946, 0.6185188889503479, 0.6182014346122742, 0.6178990602493286, 0.6176115274429321, 0.6173387169837952, 0.6170805096626282, 0.616836667060852, 0.616607129573822, 0.6163917183876038, 0.6161903142929077, 0.6160027384757996, 0.6158288717269897, 0.615668535232544, 0.6155216693878174, 0.6153880953788757, 0.6152676939964294, 0.615160346031189, 0.61506587266922, 0.6149841547012329, 0.6149150729179382, 0.6148585081100464, 0.614814281463623, 0.6147822141647339, 0.6147622466087341, 0.6147541999816895, 0.6147579550743103, 0.6147733330726624, 0.6148001551628113, 0.6324353218078613, 0.6313924193382263, 0.6303708553314209, 0.6293706297874451, 0.6283916234970093, 0.6274338364601135, 0.6264970898628235, 0.6255813241004944, 0.6246864795684814, 0.6238123774528503, 0.6229589581489563, 0.622126042842865, 0.6213135123252869, 0.6205211877822876, 0.6197490096092224, 0.6189967393875122, 0.6182643175125122, 0.6175515055656433, 0.616858184337616, 0.6161841750144958, 0.6155293583869934, 0.6148935556411743, 0.6142765283584595, 0.6136782765388489, 0.6130985021591187, 0.6125370860099792, 0.6119938492774963, 0.6114686727523804, 0.6109614372253418, 0.6104719042778015, 0.60999995470047, 0.6095454692840576, 0.6091082692146301, 0.608688235282898, 0.6082851886749268, 0.6078990697860718, 0.6075296401977539, 0.6071768999099731, 0.6068406105041504, 0.6065206527709961, 0.6062169671058655, 0.605929434299469, 0.6056578755378723, 0.6054022312164307, 0.6051623225212097, 0.6049380898475647, 0.6047293543815613, 0.6045361161231995, 0.6043581366539001, 0.6041954159736633, 0.6040477752685547, 0.6039151549339294, 0.6037973761558533, 0.6036943197250366, 0.6036059260368347, 0.603532075881958, 0.6034725904464722, 0.6034274101257324, 0.6033963561058044, 0.6033793091773987, 0.6033761501312256, 0.6033867001533508, 0.6034108400344849, 0.6034484505653381, 0.6233123540878296, 0.6222032904624939, 0.6211159229278564, 0.6200502514839172, 0.6190062165260315, 0.6179837584495544, 0.6169828176498413, 0.6160032749176025, 0.6150451302528381, 0.6141083240509033, 0.613192617893219, 0.6122980117797852, 0.6114243865013123, 0.6105716824531555, 0.6097396612167358, 0.6089282631874084, 0.6081374287605286, 0.6073669791221619, 0.606616735458374, 0.6058866381645203, 0.605176568031311, 0.6044864058494568, 0.6038159728050232, 0.6031651496887207, 0.6025338172912598, 0.6019218564033508, 0.6013291478157043, 0.6007555723190308, 0.6002009510993958, 0.5996652841567993, 0.5991483330726624, 0.5986500978469849, 0.5981703996658325, 0.5977091193199158, 0.5972661972045898, 0.5968415141105652, 0.596435010433197, 0.596046507358551, 0.5956759452819824, 0.5953232049942017, 0.5949882864952087, 0.5946710109710693, 0.5943713784217834, 0.5940892100334167, 0.5938244462013245, 0.5935770273208618, 0.5933468341827393, 0.5931338667869568, 0.5929379463195801, 0.5927589535713196, 0.5925969481468201, 0.5924516916275024, 0.5923231840133667, 0.5922113060951233, 0.5921159386634827, 0.5920370221138, 0.5919743776321411, 0.5919279456138611, 0.5918976068496704, 0.5918832421302795, 0.5918847322463989, 0.5919018983840942, 0.5919346213340759, 0.5919827818870544, 0.614101231098175, 0.6129259467124939, 0.6117727756500244, 0.610641598701477, 0.6095324754714966, 0.6084452867507935, 0.6073800921440125, 0.606336772441864, 0.6053153276443481, 0.6043156385421753, 0.6033377051353455, 0.6023813486099243, 0.6014466285705566, 0.6005333065986633, 0.5996414422988892, 0.5987709164619446, 0.5979215502738953, 0.5970933437347412, 0.5962861180305481, 0.5954998731613159, 0.5947344899177551, 0.5939897894859314, 0.5932657718658447, 0.5925623178482056, 0.5918793082237244, 0.5912166833877563, 0.5905743837356567, 0.5899522304534912, 0.5893502235412598, 0.5887682437896729, 0.5882061719894409, 0.587664008140564, 0.5871416330337524, 0.5866389870643616, 0.5861560106277466, 0.5856926441192627, 0.5852487683296204, 0.5848243832588196, 0.5844193696975708, 0.5840336680412292, 0.5836672782897949, 0.583320140838623, 0.5829921364784241, 0.582683265209198, 0.5823934078216553, 0.5821226239204407, 0.5818707346916199, 0.5816377401351929, 0.5814235806465149, 0.5812281966209412, 0.5810515284538269, 0.5808934569358826, 0.5807539820671082, 0.5806329846382141, 0.5805304050445557, 0.5804461240768433, 0.5803800821304321, 0.5803321599960327, 0.5803022980690002, 0.5802903771400452, 0.5802962183952332, 0.5803197622299194, 0.5803608894348145, 0.5804193615913391, 0.6048079133033752, 0.6035666465759277, 0.6023477911949158, 0.6011513471603394, 0.5999772548675537, 0.5988255739212036, 0.5976963043212891, 0.5965893864631653, 0.5955047607421875, 0.5944424271583557, 0.5934023857116699, 0.5923845171928406, 0.5913887619972229, 0.5904151797294617, 0.5894635915756226, 0.5885339975357056, 0.5876263380050659, 0.5867405533790588, 0.5858765244483948, 0.5850342512130737, 0.5842136740684509, 0.5834146738052368, 0.5826372504234314, 0.5818812847137451, 0.581146776676178, 0.5804336071014404, 0.5797417759895325, 0.5790711641311646, 0.5784218311309814, 0.5777935981750488, 0.5771865248680115, 0.5766005516052246, 0.5760355591773987, 0.5754916071891785, 0.5749686360359192, 0.5744665265083313, 0.5739853382110596, 0.573525071144104, 0.573085606098175, 0.5726669430732727, 0.572269082069397, 0.5718919634819031, 0.5715356469154358, 0.5711999535560608, 0.5708850026130676, 0.5705907344818115, 0.5703170299530029, 0.5700639486312866, 0.5698314905166626, 0.5696194767951965, 0.569428026676178, 0.5692569613456726, 0.5691063404083252, 0.5689760446548462, 0.5688660144805908, 0.5687761902809143, 0.5687065720558167, 0.5686569213867188, 0.5686272978782654, 0.5686175227165222, 0.5686275362968445, 0.5686572194099426, 0.5687063932418823, 0.568774938583374, 0.5954385995864868, 0.5941317081451416, 0.5928475856781006, 0.5915862321853638, 0.5903477072715759, 0.5891318917274475, 0.5879389643669128, 0.5867688059806824, 0.5856214761734009, 0.5844969749450684, 0.5833951830863953, 0.5823162198066711, 0.5812599658966064, 0.5802264213562012, 0.5792155861854553, 0.5782274007797241, 0.5772618055343628, 0.5763188600540161, 0.5753984451293945, 0.574500560760498, 0.5736252069473267, 0.5727722644805908, 0.5719417929649353, 0.5711337327957153, 0.5703480243682861, 0.5695847272872925, 0.5688437223434448, 0.5681250691413879, 0.567428708076477, 0.5667546391487122, 0.5661028027534485, 0.5654733180999756, 0.5648660063743591, 0.5642810463905334, 0.5637182593345642, 0.563177764415741, 0.562659502029419, 0.5621635317802429, 0.5616897940635681, 0.5612383484840393, 0.5608091950416565, 0.5604022741317749, 0.5600177049636841, 0.5596553683280945, 0.5593153834342957, 0.558997631072998, 0.5587022304534912, 0.5584290623664856, 0.558178186416626, 0.5579496026039124, 0.5577433109283447, 0.5575591921806335, 0.5573973059654236, 0.5572575926780701, 0.557140052318573, 0.557044506072998, 0.5569710731506348, 0.5569195747375488, 0.5568899512290955, 0.5568820834159851, 0.5568959712982178, 0.5569314956665039, 0.5569884777069092, 0.557066798210144, 0.5859997272491455, 0.5846279263496399, 0.5832791328430176, 0.5819535255432129, 0.5806511044502258, 0.5793718695640564, 0.5781158804893494, 0.5768831968307495, 0.5756738185882568, 0.5744877457618713, 0.573324978351593, 0.5721855163574219, 0.5710694193840027, 0.5699766278266907, 0.5689071416854858, 0.567861020565033, 0.566838264465332, 0.5658387541770935, 0.5648626089096069, 0.5639097690582275, 0.5629802942276001, 0.5620740652084351, 0.561191201210022, 0.5603316426277161, 0.5594954490661621, 0.5586825609207153, 0.5578930377960205, 0.5571268796920776, 0.5563840866088867, 0.5556647181510925, 0.5549687743186951, 0.5542962551116943, 0.5536471605300903, 0.5530216097831726, 0.5524196028709412, 0.551841139793396, 0.5512862801551819, 0.5507550239562988, 0.5502474308013916, 0.549763560295105, 0.549303412437439, 0.5488670468330383, 0.5484544634819031, 0.5480657815933228, 0.5477009415626526, 0.5473600029945374, 0.547042965888977, 0.5467498898506165, 0.5464808344841003, 0.5462357401847839, 0.5460146069526672, 0.545817494392395, 0.5456443428993225, 0.5454951524734497, 0.5453699231147766, 0.5452686548233032, 0.54519122838974, 0.5451376438140869, 0.5451078414916992, 0.5451017618179321, 0.5451192259788513, 0.5451602935791016, 0.5452247262001038, 0.5453124642372131, 0.5764980316162109, 0.5750620365142822, 0.573649525642395, 0.5722604990005493, 0.5708950161933899, 0.5695532560348511, 0.5682351589202881, 0.5669407844543457, 0.5656701922416687, 0.5644234418869019, 0.5632005929946899, 0.5620015859603882, 0.5608265399932861, 0.559675395488739, 0.5585482716560364, 0.5574451684951782, 0.5563660264015198, 0.5553109645843506, 0.5542799830436707, 0.5532731413841248, 0.5522903800010681, 0.5513318181037903, 0.5503974556922913, 0.5494873523712158, 0.5486014485359192, 0.5477398633956909, 0.5469026565551758, 0.5460898280143738, 0.5453014969825745, 0.5445376038551331, 0.5437982082366943, 0.5430834889411926, 0.5423933863639832, 0.5417280197143555, 0.5410873889923096, 0.5404716730117798, 0.5398808121681213, 0.5393149256706238, 0.5387740731239319, 0.5382583141326904, 0.5377677083015442, 0.5373023748397827, 0.536862313747406, 0.5364476442337036, 0.5360583662986755, 0.5356945395469666, 0.5353562831878662, 0.5350435972213745, 0.5347564816474915, 0.5344950556755066, 0.5342593193054199, 0.5340492725372314, 0.5338649749755859, 0.5337063670158386, 0.5335735082626343, 0.5334663391113281, 0.5333849191665649, 0.5333291292190552, 0.5332989692687988, 0.5332943201065063, 0.5333151817321777, 0.5333614945411682, 0.5334330201148987, 0.5335298180580139, 0.5669403076171875, 0.5654412508010864, 0.5639659762382507, 0.56251460313797, 0.5610872507095337, 0.5596839785575867, 0.5583048462867737, 0.5569499731063843, 0.5556193590164185, 0.5543131232261658, 0.5530312657356262, 0.5517739057540894, 0.5505410432815552, 0.5493327379226685, 0.5481491088867188, 0.546990156173706, 0.5458558797836304, 0.5447463989257812, 0.5436617732048035, 0.542602002620697, 0.5415672063827515, 0.5405573844909668, 0.5395726561546326, 0.5386130809783936, 0.5376786589622498, 0.5367695093154907, 0.5358856916427612, 0.535027265548706, 0.5341943502426147, 0.5333870053291321, 0.5326052904129028, 0.5318493247032166, 0.531119167804718, 0.530414879322052, 0.5297366380691528, 0.5290844440460205, 0.5284584760665894, 0.5278587341308594, 0.5272853970527649, 0.5267385244369507, 0.5262182354927063, 0.5257245898246765, 0.5252576470375061, 0.5248175859451294, 0.5244044065475464, 0.5240182876586914, 0.5236591696739197, 0.5233272314071655, 0.5230225324630737, 0.5227450728416443, 0.5224949717521667, 0.5222722291946411, 0.5220769047737122, 0.5219089984893799, 0.5217685699462891, 0.5216555595397949, 0.5215700268745422, 0.5215119123458862, 0.5214812159538269, 0.5214779376983643, 0.521501898765564, 0.521553099155426, 0.5216314196586609, 0.5217368006706238, 0.5573336482048035, 0.5557727217674255, 0.5542359948158264, 0.5527236461639404, 0.5512356758117676, 0.5497722625732422, 0.5483335256576538, 0.5469194054603577, 0.5455301403999329, 0.5441657900810242, 0.5428263545036316, 0.5415120124816895, 0.5402227640151978, 0.538958728313446, 0.5377199053764343, 0.5365065336227417, 0.5353184938430786, 0.5341560244560242, 0.5330191254615784, 0.5319079160690308, 0.5308224558830261, 0.5297628045082092, 0.5287290811538696, 0.5277214050292969, 0.526739776134491, 0.5257843732833862, 0.5248553156852722, 0.5239526033401489, 0.5230764150619507, 0.522226870059967, 0.5214040279388428, 0.5206080079078674, 0.5198389291763306, 0.5190969705581665, 0.5183821320533752, 0.5176946520805359, 0.5170345902442932, 0.5164020657539368, 0.5157971978187561, 0.5152201652526855, 0.5146710276603699, 0.5141499042510986, 0.5136569142341614, 0.5131921768188477, 0.5127558708190918, 0.5123479962348938, 0.5119686722755432, 0.5116180181503296, 0.5112961530685425, 0.5110031366348267, 0.5107390284538269, 0.510503888130188, 0.5102977752685547, 0.5101207494735718, 0.509972870349884, 0.5098540782928467, 0.5097644925117493, 0.509704053401947, 0.5096727013587952, 0.5096704959869385, 0.5096973180770874, 0.5097531676292419, 0.5098379254341125, 0.5099514722824097, 0.5476852059364319, 0.5460639595985413, 0.5444673299789429, 0.5428954362869263, 0.5413484573364258, 0.5398264527320862, 0.5383296012878418, 0.5368579030036926, 0.5354115962982178, 0.5339906811714172, 0.5325952768325806, 0.5312255620956421, 0.5298815369606018, 0.528563380241394, 0.5272711515426636, 0.5260049104690552, 0.5247648358345032, 0.5235509872436523, 0.5223634839057922, 0.5212023854255676, 0.5200678706169128, 0.5189599990844727, 0.5178789496421814, 0.5168247222900391, 0.5157975554466248, 0.5147974491119385, 0.5138246417045593, 0.5128791928291321, 0.5119612812995911, 0.511070966720581, 0.5102084279060364, 0.5093738436698914, 0.5085672736167908, 0.507788896560669, 0.5070388913154602, 0.5063173770904541, 0.5056244730949402, 0.5049603581428528, 0.5043251514434814, 0.5037189722061157, 0.5031420588493347, 0.5025944709777832, 0.5020763874053955, 0.5015879273414612, 0.5011292695999146, 0.5007004737854004, 0.5003016591072083, 0.4999329745769501, 0.4995945692062378, 0.4992864727973938, 0.49900883436203003, 0.49876168370246887, 0.49854516983032227, 0.4983592927455902, 0.4982041120529175, 0.49807971715927124, 0.4979860782623291, 0.49792322516441345, 0.4978911876678467, 0.497889906167984, 0.49791938066482544, 0.4979795217514038, 0.4980703294277191, 0.4981916546821594, 0.5380023717880249, 0.5363224744796753, 0.5346676707267761, 0.5330380201339722, 0.5314337611198425, 0.529854953289032, 0.5283017158508301, 0.5267742872238159, 0.5252727270126343, 0.5237970948219299, 0.522347629070282, 0.5209243297576904, 0.5195274353027344, 0.5181570053100586, 0.5168131589889526, 0.515496015548706, 0.5142056941986084, 0.512942373752594, 0.5117061138153076, 0.5104970932006836, 0.5093153715133667, 0.5081611275672913, 0.5070344805717468, 0.5059356093406677, 0.5048646330833435, 0.5038216710090637, 0.5028068423271179, 0.5018203854560852, 0.5008624196052551, 0.4999330937862396, 0.49903255701065063, 0.4981609880924225, 0.4973185360431671, 0.49650534987449646, 0.49572163820266724, 0.494967520236969, 0.49424320459365845, 0.4935488700866699, 0.49288463592529297, 0.4922506809234619, 0.4916471838951111, 0.4910743236541748, 0.490532249212265, 0.49002107977867126, 0.48954102396965027, 0.4890922009944916, 0.4886747896671295, 0.48828884959220886, 0.48793458938598633, 0.4876120984554291, 0.48732149600982666, 0.48706284165382385, 0.486836314201355, 0.48664191365242004, 0.4864797294139862, 0.48634985089302063, 0.4862523078918457, 0.4861871004104614, 0.4861542582511902, 0.48615381121635437, 0.4861856698989868, 0.4862498641014099, 0.4863463044166565, 0.4864749312400818, 0.5282925963401794, 0.5265560150146484, 0.5248448848724365, 0.5231595039367676, 0.5214998722076416, 0.5198662281036377, 0.5182587504386902, 0.5166774988174438, 0.5151227116584778, 0.5135944485664368, 0.5120929479598999, 0.5106182098388672, 0.5091704726219177, 0.5077498555183411, 0.5063564777374268, 0.5049905180931091, 0.5036520957946777, 0.5023413300514221, 0.5010583996772766, 0.49980345368385315, 0.4985766112804413, 0.49737805128097534, 0.49620792269706726, 0.49506640434265137, 0.4939536154270172, 0.4928697645664215, 0.4918149709701538, 0.4907894432544708, 0.4897933602333069, 0.4888268709182739, 0.48789018392562866, 0.48698344826698303, 0.48610684275627136, 0.48526060581207275, 0.48444485664367676, 0.4836598336696625, 0.48290571570396423, 0.48218265175819397, 0.4814908802509308, 0.480830579996109, 0.4802019000053406, 0.4796050488948822, 0.4790402352809906, 0.4785075783729553, 0.4780072569847107, 0.4775394797325134, 0.4771043658256531, 0.47670209407806396, 0.47633281350135803, 0.47599664330482483, 0.4756937325000763, 0.4754241704940796, 0.47518810629844666, 0.47498559951782227, 0.47481676936149597, 0.47468164563179016, 0.474580317735672, 0.47451284527778625, 0.47447919845581055, 0.47447943687438965, 0.47451353073120117, 0.47458145022392273, 0.47468316555023193, 0.4748186469078064, 0.5185636281967163, 0.5167723894119263, 0.5150071382522583, 0.513267993927002, 0.5115552544593811, 0.5098689794540405, 0.5082094073295593, 0.506576657295227, 0.5049708485603333, 0.503392219543457, 0.5018408298492432, 0.5003169775009155, 0.4988206923007965, 0.49735215306282043, 0.495911568403244, 0.4944990575313568, 0.49311476945877075, 0.49175888299942017, 0.4904315769672394, 0.48913300037384033, 0.48786330223083496, 0.48662269115448, 0.48541131615638733, 0.48422935605049133, 0.4830769896507263, 0.481954425573349, 0.4808618128299713, 0.4797993302345276, 0.4787672162055969, 0.47776564955711365, 0.4767948091030121, 0.47585490345954895, 0.47494614124298096, 0.47406870126724243, 0.4732228219509125, 0.4724086821079254, 0.47162649035453796, 0.4708764851093292, 0.47015881538391113, 0.4694736897945404, 0.46882137656211853, 0.46820199489593506, 0.4676157832145691, 0.46706292033195496, 0.46654361486434937, 0.46605801582336426, 0.46560633182525635, 0.4651886820793152, 0.4648053050041199, 0.4644562900066376, 0.4641418159008026, 0.46386200189590454, 0.4636169970035553, 0.46340686082839966, 0.46323174238204956, 0.4630917012691498, 0.4629867970943451, 0.46291711926460266, 0.4628826975822449, 0.4628835618495941, 0.462919682264328, 0.4629910886287689, 0.4630977213382721, 0.46323955059051514, 0.5088229775428772, 0.5069794058799744, 0.5051623582839966, 0.5033719539642334, 0.5016083717346191, 0.4998718500137329, 0.49816256761550903, 0.49648064374923706, 0.4948263168334961, 0.4931997060775757, 0.49160102009773254, 0.4900304079055786, 0.4884880483150482, 0.4869741201400757, 0.48548877239227295, 0.48403218388557434, 0.4826045632362366, 0.4812060296535492, 0.4798367917537689, 0.47849705815315247, 0.47718697786331177, 0.47590672969818115, 0.47465652227401733, 0.47343653440475464, 0.4722469747066498, 0.4710880219936371, 0.46995991468429565, 0.4688628017902374, 0.4677969217300415, 0.466762512922287, 0.4657597243785858, 0.4647887945175171, 0.4638499617576599, 0.462943434715271, 0.46206939220428467, 0.4612281024456024, 0.46041974425315857, 0.4596445858478546, 0.4589027762413025, 0.4581946134567261, 0.4575202465057373, 0.4568799138069153, 0.4562738239765167, 0.4557022154331207, 0.45516523718833923, 0.45466309785842896, 0.4541959762573242, 0.4537641108036041, 0.4533676207065582, 0.45300671458244324, 0.4526815116405487, 0.4523921608924866, 0.45213884115219116, 0.45192164182662964, 0.45174068212509155, 0.45159605145454407, 0.45148783922195435, 0.45141613483428955, 0.45138096809387207, 0.4513823986053467, 0.4514204263687134, 0.4514950215816498, 0.45160621404647827, 0.45175397396087646, 0.49907857179641724, 0.4971851408481598, 0.4953187108039856, 0.49347952008247375, 0.4916676878929138, 0.4898834824562073, 0.48812705278396606, 0.4863986372947693, 0.4846983551979065, 0.4830264151096344, 0.48138299584388733, 0.479768306016922, 0.47818252444267273, 0.47662582993507385, 0.4750983715057373, 0.4736003875732422, 0.4721320569515228, 0.47069352865219116, 0.4692850410938263, 0.46790674328804016, 0.46655887365341187, 0.46524161100387573, 0.4639551639556885, 0.4626997113227844, 0.4614754915237427, 0.46028271317481995, 0.45912155508995056, 0.457992285490036, 0.4568950831890106, 0.4558301568031311, 0.45479777455329895, 0.4537981450557709, 0.4528314769268036, 0.4518980085849762, 0.4509979784488678, 0.4501316249370575, 0.4492991268634796, 0.44850078225135803, 0.4477367699146271, 0.44700732827186584, 0.44631272554397583, 0.44565311074256897, 0.44502878189086914, 0.4444398880004883, 0.4438866972923279, 0.44336938858032227, 0.44288814067840576, 0.44244319200515747, 0.4420347213745117, 0.44166287779808044, 0.441327840089798, 0.44102978706359863, 0.44076886773109436, 0.4405452013015747, 0.4403589069843292, 0.44021010398864746, 0.4400988817214966, 0.44002532958984375, 0.43998953700065613, 0.43999147415161133, 0.4400312602519989, 0.44010886549949646, 0.4402242600917816, 0.44037744402885437, 0.4893381595611572, 0.48739752173423767, 0.48548442125320435, 0.48359909653663635, 0.481741726398468, 0.47991254925727844, 0.47811174392700195, 0.47633951902389526, 0.4745960831642151, 0.47288161516189575, 0.4711962938308716, 0.4695403277873993, 0.4679139256477356, 0.4663172662258148, 0.4647505581378937, 0.4632139503955841, 0.4617077112197876, 0.4602320194244385, 0.4587870240211487, 0.4573729932308197, 0.45599010586738586, 0.4546385705471039, 0.45331859588623047, 0.4520304203033447, 0.450774222612381, 0.44955024123191833, 0.4483586847782135, 0.44719982147216797, 0.44607383012771606, 0.4449809491634369, 0.44392141699790955, 0.44289547204971313, 0.44190335273742676, 0.4409452974796295, 0.4400215148925781, 0.4391322731971741, 0.4382777810096741, 0.437458336353302, 0.4366740882396698, 0.43592536449432373, 0.43521231412887573, 0.4345352351665497, 0.4338943362236023, 0.4332898259162903, 0.43272194266319275, 0.4321908950805664, 0.4316968619823456, 0.43124011158943176, 0.4308207929134369, 0.4304391145706177, 0.43009522557258606, 0.42978933453559875, 0.4295215606689453, 0.42929208278656006, 0.42910102009773254, 0.4289484918117523, 0.42883458733558655, 0.4287594258785248, 0.4287230968475342, 0.42872560024261475, 0.42876702547073364, 0.42884737253189087, 0.4289666712284088, 0.4291248917579651, 0.4796096086502075, 0.4776245355606079, 0.4756675660610199, 0.47373896837234497, 0.47183892130851746, 0.46996766328811646, 0.4681254029273987, 0.4663122892379761, 0.4645286202430725, 0.46277451515197754, 0.46105024218559265, 0.4593559503555298, 0.45769190788269043, 0.45605823397636414, 0.4544551968574524, 0.4528829753398895, 0.45134177803993225, 0.4498318135738373, 0.44835329055786133, 0.4469064176082611, 0.4454914331436157, 0.4441084861755371, 0.44275787472724915, 0.44143977761268616, 0.44015440344810486, 0.43890202045440674, 0.4376828074455261, 0.4364970326423645, 0.435344934463501, 0.4342266917228699, 0.43314260244369507, 0.4320928752422333, 0.431077778339386, 0.4300975203514099, 0.42915236949920654, 0.4282425343990326, 0.42736828327178955, 0.4265298545360565, 0.425727516412735, 0.42496147751808167, 0.42423197627067566, 0.42353925108909607, 0.4228835701942444, 0.42226511240005493, 0.4216841459274292, 0.4211408793926239, 0.42063552141189575, 0.42016828060150146, 0.41973933577537537, 0.41934895515441895, 0.41899722814559937, 0.4186844229698181, 0.41841062903404236, 0.4181760549545288, 0.417980819940567, 0.41782504320144653, 0.4177088439464569, 0.4176323413848877, 0.41759559512138367, 0.4175987243652344, 0.41764169931411743, 0.4177246391773224, 0.41784751415252686, 0.41801029443740845, 0.4699007570743561, 0.4678741991519928, 0.46587634086608887, 0.4639074504375458, 0.46196773648262024, 0.4600573778152466, 0.4581766724586487, 0.4563257694244385, 0.45450490713119507, 0.4527142643928528, 0.4509540796279907, 0.4492245614528656, 0.44752591848373413, 0.445858359336853, 0.444222092628479, 0.4426172971725464, 0.4410442113876343, 0.4395030736923218, 0.4379940330982208, 0.4365173578262329, 0.43507325649261475, 0.43366193771362305, 0.4322836101055145, 0.4309385418891907, 0.4296269118785858, 0.42834895849227905, 0.4271049499511719, 0.4258950650691986, 0.42471960186958313, 0.42357873916625977, 0.42247274518013, 0.42140185832977295, 0.4203663170337677, 0.41936635971069336, 0.4184022545814514, 0.4174742102622986, 0.41658249497413635, 0.4157273769378662, 0.4149090647697449, 0.41412779688835144, 0.4133838415145874, 0.4126774072647095, 0.41200879216194153, 0.4113781452178955, 0.4107857644557953, 0.4102318286895752, 0.40971657633781433, 0.4092402458190918, 0.40880298614501953, 0.408405065536499, 0.4080466032028198, 0.40772780776023865, 0.4074488878250122, 0.40720996260643005, 0.4070111811161041, 0.40685269236564636, 0.4067346155643463, 0.40665706992149353, 0.4066201150417328, 0.406623899936676, 0.40666842460632324, 0.4067537486553192, 0.40687990188598633, 0.4070468842983246, 0.4602194130420685, 0.45815443992614746, 0.45611879229545593, 0.454112708568573, 0.45213642716407776, 0.4501901865005493, 0.448274165391922, 0.4463886022567749, 0.44453373551368713, 0.442709743976593, 0.44091686606407166, 0.43915531039237976, 0.43742528557777405, 0.43572700023651123, 0.434060662984848, 0.4324265122413635, 0.4308246970176697, 0.42925551533699036, 0.4277191162109375, 0.4262157678604126, 0.42474564909935, 0.4233090281486511, 0.421906054019928, 0.4205370247364044, 0.41920211911201477, 0.41790157556533813, 0.416635662317276, 0.4154045581817627, 0.4142085611820221, 0.41304782032966614, 0.4119226634502411, 0.4108332693576813, 0.40977993607521057, 0.4087628424167633, 0.40778228640556335, 0.40683844685554504, 0.40593165159225464, 0.40506210923194885, 0.4042300283908844, 0.40343567728996277, 0.40267932415008545, 0.40196117758750916, 0.401281476020813, 0.40064045786857605, 0.40003836154937744, 0.3994753956794739, 0.39895179867744446, 0.3984677791595459, 0.3980235159397125, 0.39761924743652344, 0.39725515246391296, 0.3969314396381378, 0.39664822816848755, 0.39640572667121887, 0.3962040841579437, 0.39604344964027405, 0.3959239423274994, 0.3958456814289093, 0.39580875635147095, 0.3958132863044739, 0.3958593010902405, 0.3959468901157379, 0.3960760831832886, 0.39624688029289246, 0.4505733847618103, 0.448473185300827, 0.4464029371738434, 0.4443628787994385, 0.4423532783985138, 0.44037437438964844, 0.43842631578445435, 0.4365094006061554, 0.4346237778663635, 0.4327697455883026, 0.4309474527835846, 0.4291571080684662, 0.4273989796638489, 0.425673246383667, 0.42398014664649963, 0.42231985926628113, 0.4206925928592682, 0.4190985858440399, 0.4175380766391754, 0.4160112142562866, 0.414518266916275, 0.4130594730377197, 0.41163498163223267, 0.41024506092071533, 0.4088899493217468, 0.4075698256492615, 0.40628498792648315, 0.4050355851650238, 0.4038219153881073, 0.40264415740966797, 0.4015026092529297, 0.4003974497318268, 0.39932894706726074, 0.3982973098754883, 0.3973028361797333, 0.39634570479393005, 0.3954262137413025, 0.3945445418357849, 0.3937009871006012, 0.39289572834968567, 0.3921290636062622, 0.3914012014865875, 0.39071235060691833, 0.3900628089904785, 0.38945272564888, 0.3888823688030243, 0.3883519470691681, 0.3878616690635681, 0.38741177320480347, 0.38700243830680847, 0.38663384318351746, 0.38630619645118713, 0.38601964712142944, 0.3857744038105011, 0.38557061553001404, 0.3854084014892578, 0.38528794050216675, 0.385209321975708, 0.38517263531684875, 0.3851780295372009, 0.3852255642414093, 0.38531529903411865, 0.38544726371765137, 0.3856215178966522, 0.44097039103507996, 0.4388382136821747, 0.4367366433143616, 0.4346659779548645, 0.4326263666152954, 0.4306181073188782, 0.4286413788795471, 0.42669644951820374, 0.42478346824645996, 0.4229027032852173, 0.4210543632507324, 0.4192386269569397, 0.4174557626247406, 0.41570594906806946, 0.4139893651008606, 0.4123062789440155, 0.4106568694114685, 0.4090413749217987, 0.40745994448661804, 0.4059128761291504, 0.4044003188610077, 0.40292251110076904, 0.40147966146469116, 0.40007200837135315, 0.3986997604370117, 0.3973631262779236, 0.39606234431266785, 0.3947976231575012, 0.3935691714286804, 0.3923772871494293, 0.39122211933135986, 0.39010393619537354, 0.38902297616004944, 0.3879794478416443, 0.38697361946105957, 0.3860056698322296, 0.3850758969783783, 0.38418447971343994, 0.38333168625831604, 0.3825177252292633, 0.38174283504486084, 0.38100728392601013, 0.3803112506866455, 0.37965497374534607, 0.37903866171836853, 0.3784625828266144, 0.37792691588401794, 0.3774318993091583, 0.37697768211364746, 0.37656453251838684, 0.3761925995349884, 0.37586209177970886, 0.37557315826416016, 0.375326007604599, 0.37512078881263733, 0.3749576210975647, 0.37483668327331543, 0.3747580647468567, 0.3747219443321228, 0.37472835183143616, 0.3747774064540863, 0.3748691976070404, 0.37500375509262085, 0.37518110871315, 0.43141797184944153, 0.42925721406936646, 0.42712774872779846, 0.42502981424331665, 0.42296361923217773, 0.4209294319152832, 0.41892746090888977, 0.41695788502693176, 0.41502097249031067, 0.4131169319152832, 0.4112459421157837, 0.40940824151039124, 0.40760403871536255, 0.40583351254463196, 0.4040968716144562, 0.4023943543434143, 0.4007261395454407, 0.399092435836792, 0.39749348163604736, 0.3959294259548187, 0.3944005072116852, 0.39290693402290344, 0.3914489150047302, 0.39002665877342224, 0.3886403739452362, 0.3872902989387512, 0.3859765827655792, 0.3846995234489441, 0.38345927000045776, 0.38225606083869934, 0.3810901343822479, 0.37996166944503784, 0.3788709342479706, 0.37781813740730286, 0.37680351734161377, 0.37582725286483765, 0.374889612197876, 0.3739908039569855, 0.37313103675842285, 0.3723105490207672, 0.37152957916259766, 0.3707883358001709, 0.37008705735206604, 0.3694259226322174, 0.3688051700592041, 0.3682250380516052, 0.3676856756210327, 0.36718735098838806, 0.3667302429676056, 0.36631453037261963, 0.3659404516220093, 0.3656081259250641, 0.36531776189804077, 0.36506953835487366, 0.36486363410949707, 0.3647001385688782, 0.3645792305469513, 0.36450105905532837, 0.36446571350097656, 0.3644733428955078, 0.3645239770412445, 0.3646177649497986, 0.36475470662117004, 0.36493489146232605, 0.4219236671924591, 0.4197377562522888, 0.4175838232040405, 0.41546207666397095, 0.4133727550506592, 0.4113161265850067, 0.4092923402786255, 0.407301664352417, 0.40534427762031555, 0.4034203886985779, 0.4015302360057831, 0.3996739983558655, 0.3978518545627594, 0.39606407284736633, 0.39431077241897583, 0.3925922214984894, 0.3909085690975189, 0.38926005363464355, 0.3876468241214752, 0.38606908917427063, 0.3845270872116089, 0.3830209970474243, 0.38155099749565125, 0.380117267370224, 0.37872007489204407, 0.3773595988750458, 0.37603598833084106, 0.3747495114803314, 0.37350037693977356, 0.3722887337207794, 0.3711148202419281, 0.3699788749217987, 0.3688810467720032, 0.367821604013443, 0.3668007254600525, 0.3658186197280884, 0.36487552523612976, 0.36397165060043335, 0.36310717463493347, 0.3622823655605316, 0.3614974021911621, 0.36075249314308167, 0.360047847032547, 0.3593837022781372, 0.358760267496109, 0.35817772150039673, 0.3576362729072571, 0.3571361005306244, 0.35667744278907776, 0.3562604784965515, 0.3558853566646576, 0.3555523157119751, 0.35526150465011597, 0.35501307249069214, 0.35480719804763794, 0.3546440303325653, 0.3545236885547638, 0.3544463515281677, 0.35441213846206665, 0.35442110896110535, 0.35447344183921814, 0.35456913709640503, 0.35470834374427795, 0.3548910915851593, 0.41249483823776245, 0.4102872312068939, 0.4081123173236847, 0.4059702754020691, 0.4038613438606262, 0.4017857313156128, 0.3997436463832855, 0.3977353274822235, 0.39576098322868347, 0.39382076263427734, 0.39191490411758423, 0.39004358649253845, 0.38820701837539673, 0.3864053785800934, 0.38463884592056274, 0.3829076290130615, 0.38121193647384644, 0.37955188751220703, 0.3779277503490448, 0.3763396441936493, 0.3747877776622772, 0.3732723593711853, 0.3717935383319855, 0.37035152316093445, 0.36894649267196655, 0.3675786554813385, 0.36624816060066223, 0.36495524644851685, 0.3637000620365143, 0.36248284578323364, 0.36130374670028687, 0.36016297340393066, 0.35906073451042175, 0.35799720883369446, 0.3569726049900055, 0.3559871017932892, 0.35504093766212463, 0.35413429141044617, 0.3532673716545105, 0.35244035720825195, 0.35165345668792725, 0.3509069085121155, 0.3502008318901062, 0.3495354950428009, 0.3489110767841339, 0.34832775592803955, 0.34778574109077454, 0.3472852110862732, 0.34682634472846985, 0.3464093804359436, 0.346034437417984, 0.3457016944885254, 0.34541136026382446, 0.34516358375549316, 0.34495851397514343, 0.3447962999343872, 0.3446770906448364, 0.344601035118103, 0.3445682227611542, 0.3445788025856018, 0.3446328639984131, 0.34473052620887756, 0.34487184882164, 0.3450568616390228, 0.4031386077404022, 0.4009128510951996, 0.39872047305107117, 0.3965616524219513, 0.3944365978240967, 0.3923455774784088, 0.39028871059417725, 0.38826629519462585, 0.3862784504890442, 0.38432538509368896, 0.3824073076248169, 0.3805243670940399, 0.37867680191993713, 0.3768647313117981, 0.3750883936882019, 0.3733479082584381, 0.3716434836387634, 0.3699752986431122, 0.3683434724807739, 0.36674824357032776, 0.3651897609233856, 0.36366820335388184, 0.36218371987342834, 0.3607364892959595, 0.35932669043540955, 0.3579544723033905, 0.35662007331848145, 0.35532358288764954, 0.3540652394294739, 0.3528452217578888, 0.35166364908218384, 0.3505207598209381, 0.34941667318344116, 0.3483516275882721, 0.34732577204704285, 0.34633931517601013, 0.3453924357891083, 0.3444852828979492, 0.3436180651187897, 0.34279096126556396, 0.3420041799545288, 0.34125787019729614, 0.34055227041244507, 0.33988749980926514, 0.33926379680633545, 0.33868131041526794, 0.33814024925231934, 0.33764079213142395, 0.3371830880641937, 0.336767315864563, 0.33639368414878845, 0.33606234192848206, 0.33577343821525574, 0.3355271518230438, 0.33532363176345825, 0.33516302704811096, 0.3350454568862915, 0.3349711000919342, 0.3349400758743286, 0.3349524736404419, 0.3350084125995636, 0.3351080119609833, 0.3352513313293457, 0.33543846011161804, 0.3938619792461395, 0.3916216194629669, 0.38941529393196106, 0.38724324107170105, 0.38510560989379883, 0.3830026686191559, 0.38093456625938416, 0.37890151143074036, 0.3769036829471588, 0.37494125962257385, 0.3730144202709198, 0.371123343706131, 0.36926817893981934, 0.3674491047859192, 0.3656662702560425, 0.3639198839664459, 0.36221006512641907, 0.36053696274757385, 0.358900785446167, 0.35730162262916565, 0.35573968291282654, 0.3542151153087616, 0.3527280390262604, 0.3512786328792572, 0.3498670756816864, 0.3484934866428375, 0.3471580147743225, 0.3458608388900757, 0.344602108001709, 0.34338200092315674, 0.3422006368637085, 0.34105822443962097, 0.3399548828601837, 0.33889079093933105, 0.3378661274909973, 0.33688101172447205, 0.33593565225601196, 0.3350302278995514, 0.3341648578643799, 0.33333972096443176, 0.33255502581596375, 0.33181092143058777, 0.33110755681991577, 0.3304451107978821, 0.3298237919807434, 0.3292437195777893, 0.3287050724029541, 0.3282080590724945, 0.3277527987957001, 0.3273395001888275, 0.3269682824611664, 0.326639324426651, 0.3263528048992157, 0.32610884308815, 0.3259076178073883, 0.3257492482662201, 0.3256339132785797, 0.3255617022514343, 0.3255327641963959, 0.3255472183227539, 0.3256051540374756, 0.32570672035217285, 0.3258519768714905, 0.32604101300239563, 0.38467174768447876, 0.38242030143737793, 0.38020357489585876, 0.37802180647850037, 0.3758751153945923, 0.373763769865036, 0.3716879189014435, 0.369647741317749, 0.3676433861255646, 0.36567503213882446, 0.363742858171463, 0.36184701323509216, 0.35998764634132385, 0.3581649363040924, 0.356378972530365, 0.3546299338340759, 0.35291799902915955, 0.3512432277202606, 0.34960582852363586, 0.34800592064857483, 0.34644365310668945, 0.3449191153049469, 0.3434324860572815, 0.3419839143753052, 0.3405734896659851, 0.3392013609409332, 0.3378676772117615, 0.3365725874900818, 0.33531618118286133, 0.3340986669063568, 0.332920104265213, 0.33178070187568665, 0.33068057894706726, 0.3296198546886444, 0.32859867811203003, 0.32761722803115845, 0.3266756236553192, 0.32577401399612427, 0.32491254806518555, 0.3240914046764374, 0.3233107030391693, 0.3225705921649933, 0.32187125086784363, 0.3212128281593323, 0.32059547305107117, 0.32001933455467224, 0.31948456168174744, 0.3189913332462311, 0.3185397982597351, 0.31813013553619385, 0.31776243448257446, 0.3174368739128113, 0.3171536326408386, 0.31691282987594604, 0.3167146146297455, 0.3165591359138489, 0.3164465129375458, 0.3163768947124481, 0.31635040044784546, 0.31636714935302734, 0.3164272606372833, 0.31653082370758057, 0.3166779577732086, 0.31686875224113464, 0.3755744397640228, 0.3733154237270355, 0.3710918128490448, 0.3689038157463074, 0.3667515814304352, 0.36463531851768494, 0.3625551462173462, 0.36051127314567566, 0.3585038185119629, 0.3565329611301422, 0.35459885001182556, 0.3527016043663025, 0.35084134340286255, 0.34901824593544006, 0.3472324311733246, 0.3454839885234833, 0.34377309679985046, 0.3420998454093933, 0.34046435356140137, 0.3388667702674866, 0.3373071551322937, 0.3357856869697571, 0.3343024253845215, 0.33285754919052124, 0.3314511179924011, 0.3300832509994507, 0.32875409722328186, 0.3274637460708618, 0.3262123465538025, 0.32499995827674866, 0.32382673025131226, 0.32269278168678284, 0.32159823179244995, 0.32054319977760315, 0.319527804851532, 0.3185521960258484, 0.31761646270751953, 0.31672075390815735, 0.3158651888370514, 0.3150499165058136, 0.3142750561237335, 0.3135407269001007, 0.3128471076488495, 0.312194287776947, 0.31158244609832764, 0.3110117018222809, 0.3104822039604187, 0.30999407172203064, 0.309547483921051, 0.3091425597667694, 0.30877944827079773, 0.30845826864242554, 0.30817919969558716, 0.30794233083724976, 0.30774784088134766, 0.3075958490371704, 0.30748650431632996, 0.30741992592811584, 0.30739620327949524, 0.30741551518440247, 0.3074779510498047, 0.30758360028266907, 0.30773258209228516, 0.3079250156879425, 0.36657634377479553, 0.3643132448196411, 0.3620862364768982, 0.3598954975605011, 0.3577411472797394, 0.35562336444854736, 0.3535423278808594, 0.3514981269836426, 0.3494909703731537, 0.3475209176540375, 0.34558814764022827, 0.3436927795410156, 0.3418349027633667, 0.34001466631889343, 0.3382321298122406, 0.33648747205734253, 0.3347807228565216, 0.33311203122138977, 0.3314814865589142, 0.329889178276062, 0.3283351957798004, 0.32681965827941895, 0.32534265518188477, 0.32390424609184265, 0.32250455021858215, 0.32114365696907043, 0.31982168555259705, 0.3185386657714844, 0.317294716835022, 0.3160899579524994, 0.3149244785308838, 0.31379833817481995, 0.3127116560935974, 0.31166455149650574, 0.3106571137905121, 0.3096894323825836, 0.3087615966796875, 0.30787375569343567, 0.3070259988307953, 0.3062184154987335, 0.3054511845111847, 0.3047243356704712, 0.30403804779052734, 0.3033924102783203, 0.30278754234313965, 0.3022235929965973, 0.3017006516456604, 0.3012188971042633, 0.30077841877937317, 0.30037933588027954, 0.30002179741859436, 0.2997059226036072, 0.29943186044692993, 0.2991997301578522, 0.29900965094566345, 0.2988617718219757, 0.2987562119960785, 0.29869306087493896, 0.29867249727249146, 0.2986946105957031, 0.2987595200538635, 0.2988673448562622, 0.2990182042121887, 0.29921215772628784, 0.3576834797859192, 0.35541975498199463, 0.3531927764415741, 0.35100266337394714, 0.3488495945930481, 0.3467336893081665, 0.3446550965309143, 0.34261393547058105, 0.3406102955341339, 0.3386443555355072, 0.3367161750793457, 0.33482587337493896, 0.33297351002693176, 0.3311592638492584, 0.32938313484191895, 0.3276452422142029, 0.3259456753730774, 0.32428452372550964, 0.3226618468761444, 0.32107770442962646, 0.319532185792923, 0.3180253803730011, 0.3165573179721832, 0.3151281177997589, 0.31373780965805054, 0.3123864531517029, 0.3110741376876831, 0.3098009526729584, 0.3085669279098511, 0.30737215280532837, 0.30621668696403503, 0.30510061979293823, 0.30402401089668274, 0.30298691987991333, 0.30198946595191956, 0.3010317385196686, 0.3001137673854828, 0.29923564195632935, 0.2983974814414978, 0.2975993752479553, 0.2968413829803467, 0.2961236238479614, 0.2954462170600891, 0.2948092222213745, 0.2942127585411072, 0.2936569154262543, 0.29314181208610535, 0.29266759753227234, 0.2922343313694, 0.29184213280677795, 0.29149115085601807, 0.2911814749240875, 0.29091325402259827, 0.29068657755851746, 0.29050156474113464, 0.29035836458206177, 0.2902570962905884, 0.29019784927368164, 0.2901807725429535, 0.2902059555053711, 0.290273517370224, 0.29038360714912415, 0.2905363142490387, 0.2907317280769348, 0.3489015996456146, 0.34664061665534973, 0.344417005777359, 0.342230886220932, 0.34008240699768066, 0.33797165751457214, 0.3358987867832184, 0.33386385440826416, 0.3318670094013214, 0.3299082815647125, 0.3279878497123718, 0.32610568404197693, 0.32426196336746216, 0.3224566876888275, 0.32068994641304016, 0.3189617991447449, 0.3172723054885864, 0.315621554851532, 0.31400951743125916, 0.3124363422393799, 0.3109019994735718, 0.309406578540802, 0.30795010924339294, 0.3065326511859894, 0.3051542043685913, 0.3038148880004883, 0.3025146722793579, 0.30125364661216736, 0.3000318109989166, 0.2988492548465729, 0.2977060079574585, 0.29660212993621826, 0.29553765058517456, 0.29451265931129456, 0.29352715611457825, 0.2925812304019928, 0.2916749119758606, 0.2908083200454712, 0.289981484413147, 0.2891944646835327, 0.2884473204612732, 0.28774017095565796, 0.2870730757713318, 0.28644609451293945, 0.2858593165874481, 0.28531286120414734, 0.2848068177700043, 0.2843412160873413, 0.2839162349700928, 0.28353193402290344, 0.2831884026527405, 0.28288576006889343, 0.28262415528297424, 0.2824036180973053, 0.2822243273258209, 0.2820863723754883, 0.2819898724555969, 0.281934916973114, 0.2819216549396515, 0.2819502055644989, 0.282020628452301, 0.2821331024169922, 0.28228768706321716, 0.2824845016002655, 0.340236097574234, 0.3379811942577362, 0.33576422929763794, 0.33358538150787354, 0.33144471049308777, 0.3293423056602478, 0.3272783160209656, 0.3252527713775635, 0.32326576113700867, 0.3213173747062683, 0.3194076716899872, 0.31753668189048767, 0.31570449471473694, 0.3139111399650574, 0.31215664744377136, 0.3104410767555237, 0.30876442790031433, 0.3071267902851105, 0.3055281341075897, 0.30396851897239685, 0.30244794487953186, 0.30096644163131714, 0.29952406883239746, 0.29812076687812805, 0.2967566251754761, 0.29543161392211914, 0.294145792722702, 0.29289916157722473, 0.29169172048568726, 0.290523499250412, 0.2893945574760437, 0.28830486536026, 0.2872544825077057, 0.28624340891838074, 0.28527167439460754, 0.28433936834335327, 0.28344646096229553, 0.2825930118560791, 0.28177908062934875, 0.2810046970844269, 0.28026989102363586, 0.27957478165626526, 0.2789193391799927, 0.27830368280410767, 0.277727872133255, 0.27719196677207947, 0.2766960561275482, 0.27624017000198364, 0.2758244574069977, 0.2754489481449127, 0.2751137316226959, 0.27481895685195923, 0.274564653635025, 0.27435097098350525, 0.2741779685020447, 0.27404579520225525, 0.2739545404911041, 0.27390429377555847, 0.27389517426490784, 0.2739273011684418, 0.27400079369544983, 0.27411574125289917, 0.27427226305007935, 0.2744704782962799, 0.3316921293735504, 0.3294464945793152, 0.32723939418792725, 0.32507094740867615, 0.32294121384620667, 0.32085028290748596, 0.3187982141971588, 0.3167850971221924, 0.31481093168258667, 0.31287580728530884, 0.3109797537326813, 0.30912280082702637, 0.3073049783706665, 0.3055263161659241, 0.3037868142127991, 0.3020865023136139, 0.30042535066604614, 0.2988034188747406, 0.2972206771373749, 0.29567715525627136, 0.2941727936267853, 0.2927076518535614, 0.29128167033195496, 0.2898949086666107, 0.2885472774505615, 0.28723880648612976, 0.2859695255756378, 0.2847393751144409, 0.28354835510253906, 0.28239646553993225, 0.2812837064266205, 0.28021010756492615, 0.27917560935020447, 0.2781802713871002, 0.277224063873291, 0.27630698680877686, 0.2754290699958801, 0.2745903432369232, 0.27379080653190613, 0.27303048968315125, 0.27230942249298096, 0.27162763476371765, 0.2709851562976837, 0.2703820466995239, 0.26981833577156067, 0.26929405331611633, 0.26880931854248047, 0.2683641314506531, 0.2679585814476013, 0.26759272813796997, 0.2672666609287262, 0.26698046922683716, 0.26673421263694763, 0.2665279507637024, 0.26636186242103577, 0.26623594760894775, 0.2661503553390503, 0.26610520482063293, 0.26610055565834045, 0.2661365270614624, 0.2662132680416107, 0.2663308382034302, 0.2664893567562103, 0.2666889429092407, 0.32327449321746826, 0.3210412561893463, 0.31884709000587463, 0.3166921138763428, 0.31457632780075073, 0.3124998211860657, 0.3104626536369324, 0.30846482515335083, 0.3065063953399658, 0.30458736419677734, 0.3027077615261078, 0.30086755752563477, 0.29906681180000305, 0.29730546474456787, 0.295583575963974, 0.2939010560512543, 0.2922579348087311, 0.29065418243408203, 0.28908976912498474, 0.2875646650791168, 0.28607887029647827, 0.2846323251724243, 0.28322499990463257, 0.28185686469078064, 0.28052785992622375, 0.2792380154132843, 0.2779872417449951, 0.2767755389213562, 0.2756028473377228, 0.27446916699409485, 0.27337440848350525, 0.27231863141059875, 0.2713017463684082, 0.2703237533569336, 0.26938462257385254, 0.26848435401916504, 0.2676229178905487, 0.2668003439903259, 0.26601657271385193, 0.2652716636657715, 0.2645655870437622, 0.2638983428478241, 0.26326999068260193, 0.2626805305480957, 0.2621299624443054, 0.26161837577819824, 0.2611457407474518, 0.2607121765613556, 0.2603176534175873, 0.259962260723114, 0.25964608788490295, 0.2593691647052765, 0.2591315507888794, 0.2589333653450012, 0.25877463817596436, 0.25865551829338074, 0.25857603549957275, 0.25853630900382996, 0.2585364282131195, 0.25857654213905334, 0.25865668058395386, 0.258776992559433, 0.2589375972747803, 0.2591385841369629, 0.3149876594543457, 0.3127698004245758, 0.31059154868125916, 0.3084529638290405, 0.3063540458679199, 0.3042948544025421, 0.3022753596305847, 0.3002956211566925, 0.2983556389808655, 0.29645538330078125, 0.2945948541164398, 0.2927740216255188, 0.2909928858280182, 0.2892514169216156, 0.28754955530166626, 0.2858872711658478, 0.2842645049095154, 0.28268125653266907, 0.2811374366283417, 0.2796330153942108, 0.2781679034233093, 0.2767420709133148, 0.2753554582595825, 0.27400797605514526, 0.27269962430000305, 0.27143028378486633, 0.27019989490509033, 0.26900845766067505, 0.26785585284233093, 0.2667420506477356, 0.26566699147224426, 0.26463061571121216, 0.2636328935623169, 0.2626737654209137, 0.2617532014846802, 0.26087114214897156, 0.26002755761146545, 0.25922244787216187, 0.25845572352409363, 0.25772741436958313, 0.25703752040863037, 0.2563859522342682, 0.25577279925346375, 0.25519800186157227, 0.25466158986091614, 0.25416356325149536, 0.2537039816379547, 0.2532828152179718, 0.2529001235961914, 0.2525559365749359, 0.2522503137588501, 0.25198331475257874, 0.2517549693584442, 0.2515653371810913, 0.25141453742980957, 0.2513025999069214, 0.2512296140193939, 0.25119566917419434, 0.2512008845806122, 0.25124531984329224, 0.25132909417152405, 0.2514522969722748, 0.251615047454834, 0.2518174648284912, 0.30683568120002747, 0.304636150598526, 0.3024766445159912, 0.30035725235939026, 0.29827797412872314, 0.2962387800216675, 0.29423969984054565, 0.2922806739807129, 0.2903617322444916, 0.28848278522491455, 0.2866438627243042, 0.28484484553337097, 0.28308573365211487, 0.2813664674758911, 0.27968695759773254, 0.2780471742153168, 0.27644699811935425, 0.2748863995075226, 0.2733652889728546, 0.27188360691070557, 0.2704412043094635, 0.26903805136680603, 0.267674058675766, 0.2663491368293762, 0.26506319642066956, 0.26381611824035645, 0.2626078724861145, 0.2614383399486542, 0.2603074610233307, 0.2592151463031769, 0.2581612765789032, 0.25714582204818726, 0.2561686933040619, 0.2552298307418823, 0.2543291449546814, 0.25346657633781433, 0.25264209508895874, 0.25185561180114746, 0.2511070966720581, 0.2503965198993683, 0.24972383677959442, 0.24908900260925293, 0.24849198758602142, 0.2479328066110611, 0.24741141498088837, 0.24692782759666443, 0.24648204445838928, 0.24607406556606293, 0.24570390582084656, 0.24537159502506256, 0.24507717788219452, 0.24482066929340363, 0.24460211396217346, 0.24442158639431, 0.24427913129329681, 0.2441748082637787, 0.24410869181156158, 0.24408087134361267, 0.24409142136573792, 0.24414043128490448, 0.24422800540924072, 0.244354248046875, 0.24451926350593567, 0.2447231411933899, 0.29882240295410156, 0.2966439127922058, 0.29450589418411255, 0.29240837693214417, 0.29035136103630066, 0.28833475708961487, 0.2863585948944092, 0.28442278504371643, 0.2825273275375366, 0.2806721329689026, 0.27885711193084717, 0.277082234621048, 0.27534741163253784, 0.2736525535583496, 0.2719975709915161, 0.2703823745250702, 0.2688068449497223, 0.26727092266082764, 0.2657744884490967, 0.26431742310523987, 0.26289960741996765, 0.26152095198631287, 0.26018133759498596, 0.2588806748390198, 0.25761881470680237, 0.2563956677913666, 0.25521111488342285, 0.25406503677368164, 0.2529573440551758, 0.2518879175186157, 0.2508566379547119, 0.24986344575881958, 0.24890820682048798, 0.24799083173274994, 0.2471112459897995, 0.2462693601846695, 0.24546508491039276, 0.24469834566116333, 0.24396908283233643, 0.24327722191810608, 0.24262270331382751, 0.24200549721717834, 0.2414255291223526, 0.2408827841281891, 0.24037723243236542, 0.239908829331398, 0.23947757482528687, 0.23908346891403198, 0.23872648179531097, 0.23840665817260742, 0.23812399804592133, 0.23787853121757507, 0.23767027258872986, 0.23749926686286926, 0.23736557364463806, 0.23726925253868103, 0.23721034824848175, 0.2371889352798462, 0.23720510303974152, 0.2372589260339737, 0.2373504936695099, 0.23747989535331726, 0.23764726519584656, 0.23785267770290375, 0.2909511923789978, 0.2887963652610779, 0.2866824269294739, 0.284609317779541, 0.2825770378112793, 0.28058549761772156, 0.278634637594223, 0.2767243981361389, 0.2748546898365021, 0.2730254530906677, 0.2712365984916687, 0.26948797702789307, 0.26777952909469604, 0.2661111354827881, 0.26448267698287964, 0.26289403438568115, 0.2613450586795807, 0.2598356604576111, 0.2583656907081604, 0.2569350004196167, 0.2555435001850128, 0.25419101119041443, 0.2528773844242096, 0.25160253047943115, 0.25036630034446716, 0.2491685152053833, 0.2480090856552124, 0.24688784778118134, 0.24580468237400055, 0.2447594404220581, 0.24375201761722565, 0.24278227984905243, 0.2418500930070877, 0.24095535278320312, 0.24009795486927032, 0.23927778005599976, 0.23849472403526306, 0.23774868249893188, 0.23703959584236145, 0.2363673448562622, 0.23573186993598938, 0.2351330816745758, 0.2345709502696991, 0.23404540121555328, 0.2335563749074936, 0.23310384154319763, 0.23268777132034302, 0.23230813443660736, 0.23196491599082947, 0.23165811598300934, 0.23138771951198578, 0.23115375638008118, 0.23095624148845673, 0.23079518973827362, 0.23067064583301544, 0.23058265447616577, 0.23053127527236938, 0.23051655292510986, 0.23053857684135437, 0.23059740662574768, 0.23069314658641815, 0.23082587122917175, 0.23099568486213684, 0.23120270669460297, 0.2832251489162445, 0.281096488237381, 0.27900901436805725, 0.2769627273082733, 0.2749575078487396, 0.272993266582489, 0.2710699737071991, 0.26918748021125793, 0.26734569668769836, 0.2655445337295532, 0.26378384232521057, 0.26206350326538086, 0.26038336753845215, 0.2587433159351349, 0.2571432292461395, 0.25558289885520935, 0.2540622353553772, 0.25258105993270874, 0.25113919377326965, 0.24973651766777039, 0.24837283790111542, 0.2470480054616928, 0.24576187133789062, 0.24451424181461334, 0.24330498278141022, 0.24213390052318573, 0.24100087583065033, 0.2399057149887085, 0.2388482689857483, 0.23782838881015778, 0.23684591054916382, 0.23590070009231567, 0.2349926233291626, 0.23412151634693146, 0.2332872599363327, 0.2324897050857544, 0.23172876238822937, 0.2310042828321457, 0.2303161770105362, 0.22966432571411133, 0.22904863953590393, 0.22846901416778564, 0.2279253900051117, 0.22741767764091492, 0.22694581747055054, 0.22650974988937378, 0.22610943019390106, 0.2257447987794876, 0.22541585564613342, 0.22512255609035492, 0.2248648852109909, 0.22464285790920258, 0.22445647418498993, 0.22430573403835297, 0.22419068217277527, 0.22411134839057922, 0.22406776249408722, 0.22405999898910522, 0.22408810257911682, 0.22415214776992798, 0.22425222396850586, 0.22438840568065643, 0.22456079721450806, 0.2247695028781891, 0.27564695477485657, 0.2735467851161957, 0.2714880704879761, 0.26947081089019775, 0.2674948275089264, 0.2655600905418396, 0.26366642117500305, 0.2618137001991272, 0.2600018382072449, 0.25823065638542175, 0.2565000057220459, 0.25480973720550537, 0.2531597316265106, 0.25154978036880493, 0.24997970461845398, 0.24844937026500702, 0.24695856869220734, 0.245507150888443, 0.24409490823745728, 0.24272166192531586, 0.24138721823692322, 0.24009141325950623, 0.23883403837680817, 0.23761491477489471, 0.23643384873867035, 0.23529066145420074, 0.23418515920639038, 0.23311717808246613, 0.23208650946617126, 0.23109300434589386, 0.2301364690065384, 0.22921675443649292, 0.22833366692066193, 0.22748707234859467, 0.2266767919063568, 0.22590269148349762, 0.22516462206840515, 0.22446244955062866, 0.2237960398197174, 0.22316527366638184, 0.2225700169801712, 0.22201019525527954, 0.2214856743812561, 0.22099637985229492, 0.22054222226142883, 0.22012312710285187, 0.21973904967308044, 0.2193899005651474, 0.21907566487789154, 0.2187962830066681, 0.21855174005031586, 0.21834200620651245, 0.21816709637641907, 0.2180270105600357, 0.21792174875736237, 0.21785134077072144, 0.2178158164024353, 0.21781523525714874, 0.21784964203834534, 0.21791908144950867, 0.2180236577987671, 0.21816343069076538, 0.2183385044336319, 0.218548983335495, 0.268218994140625, 0.26614946126937866, 0.2641216814517975, 0.26213550567626953, 0.26019081473350525, 0.2582874894142151, 0.2564253509044647, 0.254604309797287, 0.25282415747642517, 0.25108471512794495, 0.24938586354255676, 0.24772736430168152, 0.24610906839370728, 0.24453076720237732, 0.24299226701259613, 0.24149338901042938, 0.24003389477729797, 0.23861360549926758, 0.23723231256008148, 0.23588979244232178, 0.23458583652973175, 0.23332025110721588, 0.23209281265735626, 0.23090329766273499, 0.22975151240825653, 0.22863724827766418, 0.22756029665470123, 0.22652044892311096, 0.22551749646663666, 0.2245512455701828, 0.22362150251865387, 0.22272808849811554, 0.2218707948923111, 0.2210494577884674, 0.22026388347148895, 0.2195139229297638, 0.2187993973493576, 0.21812015771865845, 0.21747605502605438, 0.21686695516109467, 0.21629270911216736, 0.21575318276882172, 0.21524828672409058, 0.2147778868675232, 0.2143419086933136, 0.21394024789333344, 0.21357281506061554, 0.21323955059051514, 0.21294039487838745, 0.2126752883195877, 0.21244420111179352, 0.2122471183538437, 0.21208398044109344, 0.21195481717586517, 0.21185961365699768, 0.21179838478565216, 0.2117711454629898, 0.2117779552936554, 0.2118188440799713, 0.21189387142658234, 0.21200309693813324, 0.21214659512043, 0.21232447028160095, 0.2125367969274521, 0.2609432637691498, 0.25890639424324036, 0.2569114863872528, 0.25495830178260803, 0.25304678082466125, 0.2511766850948334, 0.24934789538383484, 0.24756020307540894, 0.24581341445446014, 0.24410736560821533, 0.2424418330192566, 0.2408166080713272, 0.23923149704933167, 0.23768626153469086, 0.23618070781230927, 0.2347145825624466, 0.23328767716884613, 0.23189975321292877, 0.23055057227611542, 0.22923992574214935, 0.22796756029129028, 0.2267332524061203, 0.2255367487668991, 0.22437784075737, 0.22325627505779266, 0.2221718281507492, 0.22112427651882172, 0.2201133817434311, 0.21913893520832062, 0.2182007133960724, 0.2172984927892685, 0.21643207967281342, 0.21560125052928925, 0.21480579674243927, 0.21404555439949036, 0.2133203148841858, 0.21262989938259125, 0.2119741290807724, 0.21135284006595612, 0.21076588332653046, 0.21021310985088348, 0.20969434082508087, 0.20920948684215546, 0.2087583988904953, 0.20834097266197205, 0.20795708894729614, 0.20760667324066162, 0.20728962123394012, 0.20700587332248688, 0.2067553550004959, 0.20653802156448364, 0.2063538134098053, 0.20620273053646088, 0.2060847282409668, 0.20599980652332306, 0.20594796538352966, 0.205929234623909, 0.20594361424446106, 0.20599116384983063, 0.2060719132423401, 0.2061859369277954, 0.20633329451084137, 0.20651407539844513, 0.20672836899757385, 0.25382140278816223, 0.25181910395622253, 0.24985884130001068, 0.24794045090675354, 0.24606375396251678, 0.2442285716533661, 0.24243469536304474, 0.24068191647529602, 0.23897001147270203, 0.23729877173900604, 0.23566797375679016, 0.23407737910747528, 0.2325267493724823, 0.23101584613323212, 0.22954443097114563, 0.22811223566532135, 0.22671903669834137, 0.225364550948143, 0.22404853999614716, 0.22277075052261353, 0.22153092920780182, 0.22032880783081055, 0.21916411817073822, 0.21803663671016693, 0.2169460952281952, 0.2158922404050827, 0.21487481892108917, 0.2138935923576355, 0.21294832229614258, 0.21203875541687012, 0.2111646682024002, 0.21032582223415375, 0.20952200889587402, 0.20875298976898193, 0.20801857113838196, 0.2073185294866562, 0.2066526710987091, 0.20602081716060638, 0.2054227590560913, 0.20485834777355194, 0.20432740449905396, 0.20382976531982422, 0.203365296125412, 0.20293383300304413, 0.20253527164459229, 0.2021694779396057, 0.20183634757995605, 0.20153579115867615, 0.20126770436763763, 0.20103201270103455, 0.2008286565542221, 0.20065759122371674, 0.20051874220371246, 0.20041212439537048, 0.20033767819404602, 0.20029541850090027, 0.20028534531593323, 0.2003074735403061, 0.20036183297634125, 0.20044846832752228, 0.20056742429733276, 0.20071876049041748, 0.2009025663137436, 0.20111890137195587, 0.2468547821044922, 0.24488870799541473, 0.24296477437019348, 0.24108275771141052, 0.23924246430397034, 0.23744367063045502, 0.23568613827228546, 0.23396965861320496, 0.2322939783334732, 0.2306588590145111, 0.22906403243541718, 0.2275092601776123, 0.2259942591190338, 0.2245187908411026, 0.22308257222175598, 0.22168533504009247, 0.22032679617404938, 0.21900668740272522, 0.2177247256040573, 0.21648065745830536, 0.2152741700410843, 0.2141050100326538, 0.21297289431095123, 0.21187753975391388, 0.21081869304180145, 0.20979605615139008, 0.20880937576293945, 0.2078583836555481, 0.2069428265094757, 0.206062451004982, 0.20521698892116547, 0.20440618693828583, 0.20362982153892517, 0.2028876543045044, 0.2021794468164444, 0.20150499045848846, 0.2008640617132187, 0.20025645196437836, 0.19968196749687195, 0.19914039969444275, 0.19863158464431763, 0.19815535843372345, 0.1977115273475647, 0.19729995727539062, 0.1969205141067505, 0.19657303392887115, 0.19625741243362427, 0.19597354531288147, 0.1957213282585144, 0.1955006718635559, 0.19531148672103882, 0.19515371322631836, 0.19502730667591095, 0.19493220746517181, 0.19486840069293976, 0.1948358714580536, 0.1948346197605133, 0.19486463069915771, 0.1949259340763092, 0.19501857459545135, 0.19514258205890656, 0.19529801607131958, 0.1954849511384964, 0.19570347666740417, 0.24004435539245605, 0.2381160706281662, 0.23622994124889374, 0.23438574373722076, 0.23258322477340698, 0.23082216084003448, 0.2291022688150406, 0.22742332518100739, 0.2257850617170334, 0.22418718039989471, 0.22262942790985107, 0.2211115062236786, 0.21963314712047577, 0.21819406747817993, 0.216793954372406, 0.2154325246810913, 0.21410948038101196, 0.21282453835010529, 0.2115773856639862, 0.21036772429943085, 0.20919527113437653, 0.20805971324443817, 0.2069607526063919, 0.20589812099933624, 0.20487149059772491, 0.20388059318065643, 0.20292514562606812, 0.20200484991073608, 0.20111945271492004, 0.20026865601539612, 0.19945219159126282, 0.19866980612277985, 0.1979212462902069, 0.19720624387264252, 0.19652456045150757, 0.19587597250938416, 0.19526022672653198, 0.19467711448669434, 0.1941264271736145, 0.19360794126987457, 0.19312147796154022, 0.19266684353351593, 0.19224387407302856, 0.1918523758649826, 0.1914921998977661, 0.19116321206092834, 0.19086527824401855, 0.190598264336586, 0.19036205112934113, 0.19015656411647797, 0.18998168408870697, 0.18983735144138336, 0.18972350656986237, 0.1896400898694992, 0.1895870566368103, 0.18956437706947327, 0.1895720511674881, 0.1896100789308548, 0.18967844545841217, 0.189777210354805, 0.18990638852119446, 0.19006602466106415, 0.19025620818138123, 0.19047698378562927, 0.23339082300662994, 0.2315017133951187, 0.2296547293663025, 0.22784960269927979, 0.2260860949754715, 0.22436393797397614, 0.22268283367156982, 0.22104249894618988, 0.21944266557693481, 0.21788302063941956, 0.21636326611042023, 0.21488310396671295, 0.21344223618507385, 0.21204033493995667, 0.2106771022081375, 0.20935222506523132, 0.20806536078453064, 0.20681622624397278, 0.20560449361801147, 0.20442983508110046, 0.20329193770885468, 0.20219050347805023, 0.20112518966197968, 0.20009571313858032, 0.19910173118114471, 0.19814297556877136, 0.197219118475914, 0.19632987678050995, 0.19547493755817413, 0.19465403258800507, 0.19386684894561768, 0.19311313331127167, 0.19239261746406555, 0.19170500338077545, 0.19105006754398346, 0.1904275268316269, 0.18983715772628784, 0.1892787218093872, 0.1887519806623459, 0.18825674057006836, 0.18779276311397552, 0.18735986948013306, 0.18695785105228424, 0.18658654391765594, 0.18624576926231384, 0.185935378074646, 0.18565522134304047, 0.1854051649570465, 0.18518507480621338, 0.18499484658241272, 0.18483439087867737, 0.18470358848571777, 0.18460239470005035, 0.18453074991703033, 0.18448857963085175, 0.1844758540391922, 0.1844925582408905, 0.18453869223594666, 0.18461422622203827, 0.18471921980381012, 0.184853658080101, 0.18501761555671692, 0.18521113693714142, 0.1854342818260193, 0.2268945276737213, 0.2250458151102066, 0.22323915362358093, 0.22147426009178162, 0.21975083649158478, 0.21806859970092773, 0.21642723679542542, 0.21482644975185394, 0.21326592564582825, 0.21174535155296326, 0.2102643996477127, 0.20882275700569153, 0.20742006599903107, 0.20605602860450745, 0.2047303020954132, 0.2034425586462021, 0.20219244062900543, 0.20097963511943817, 0.19980378448963165, 0.198664590716362, 0.19756168127059937, 0.19649474322795868, 0.19546344876289368, 0.1944674551486969, 0.19350644946098328, 0.19258010387420654, 0.19168810546398163, 0.19083014130592346, 0.19000589847564697, 0.18921507894992828, 0.18845736980438232, 0.1877324879169464, 0.18704015016555786, 0.186380073428154, 0.1857519894838333, 0.18515563011169434, 0.18459072709083557, 0.1840570569038391, 0.18355436623096466, 0.18308241665363312, 0.18264099955558777, 0.1822299063205719, 0.1818489283323288, 0.18149785697460175, 0.1811765432357788, 0.18088480830192566, 0.18062248826026917, 0.1803894340991974, 0.18018551170825958, 0.1800106018781662, 0.17986461520195007, 0.17974741756916046, 0.1796589344739914, 0.1795991063117981, 0.1795678734779358, 0.1795651763677597, 0.17959100008010864, 0.17964531481266022, 0.17972810566425323, 0.17983940243721008, 0.17997920513153076, 0.18014757335186005, 0.18034452199935913, 0.18057015538215637, 0.22055554389953613, 0.2187483161687851, 0.21698300540447235, 0.21525928378105164, 0.21357686817646027, 0.21193543076515198, 0.2103346586227417, 0.20877419412136078, 0.20725373923778534, 0.20577292144298553, 0.2043314278125763, 0.20292888581752777, 0.2015649825334549, 0.20023933053016663, 0.1989516019821167, 0.19770143926143646, 0.19648849964141846, 0.19531241059303284, 0.19417282938957214, 0.19306939840316772, 0.19200178980827332, 0.19096963107585907, 0.18997259438037872, 0.18901032209396362, 0.1880824714899063, 0.18718872964382172, 0.18632875382900238, 0.18550220131874084, 0.18470877408981323, 0.18394814431667328, 0.18321998417377472, 0.18252402544021606, 0.18185994029045105, 0.181227445602417, 0.1806262582540512, 0.18005609512329102, 0.17951670289039612, 0.17900781333446503, 0.17852917313575745, 0.17808054387569427, 0.1776616871356964, 0.17727239429950714, 0.17691244184970856, 0.17658163607120514, 0.17627978324890137, 0.17600668966770172, 0.17576220631599426, 0.17554616928100586, 0.17535842955112457, 0.17519886791706085, 0.17506735026836395, 0.17496377229690552, 0.17488804459571838, 0.1748400777578354, 0.17481981217861176, 0.1748271882534027, 0.17486217617988586, 0.17492473125457764, 0.17501485347747803, 0.17513252794742584, 0.17527776956558228, 0.1754506230354309, 0.17565113306045532, 0.1758793294429779, 0.2143736630678177, 0.21260882914066315, 0.21088573336601257, 0.2092040330171585, 0.2075634002685547, 0.20596350729465485, 0.20440398156642914, 0.2028844952583313, 0.20140469074249268, 0.19996419548988342, 0.19856266677379608, 0.1971997320652008, 0.19587501883506775, 0.19458816945552826, 0.1933388113975525, 0.1921265870332718, 0.19095110893249512, 0.18981201946735382, 0.18870893120765686, 0.18764151632785797, 0.1866093873977661, 0.18561217188835144, 0.1846495270729065, 0.18372109532356262, 0.18282651901245117, 0.1819654405117035, 0.1811375468969345, 0.18034246563911438, 0.17957986891269684, 0.17884944379329681, 0.17815086245536804, 0.17748379707336426, 0.17684794962406158, 0.17624300718307495, 0.17566868662834167, 0.17512470483779907, 0.17461076378822327, 0.17412661015987396, 0.17367197573184967, 0.1732466071844101, 0.17285026609897614, 0.1724827140569687, 0.17214374244213104, 0.1718331277370453, 0.1715506762266159, 0.17129619419574738, 0.17106950283050537, 0.17087045311927795, 0.1706988662481308, 0.17055460810661316, 0.17043755948543549, 0.17034758627414703, 0.17028461396694183, 0.17024852335453033, 0.17023923993110657, 0.17025671899318695, 0.1703009009361267, 0.17037174105644226, 0.1704692244529724, 0.17059335112571716, 0.17074410617351532, 0.17092153429985046, 0.17112566530704498, 0.17135652899742126, 0.2083483636379242, 0.20662671327590942, 0.20494656264781952, 0.20330755412578583, 0.20170935988426208, 0.20015157759189606, 0.19863387942314148, 0.19715586304664612, 0.19571717083454132, 0.19431743025779724, 0.19295625388622284, 0.19163326919078827, 0.1903480738401413, 0.18910031020641327, 0.18788957595825195, 0.1867154836654663, 0.18557767570018768, 0.18447573482990265, 0.18340928852558136, 0.18237797915935516, 0.18138141930103302, 0.1804192066192627, 0.17949101328849792, 0.17859643697738647, 0.1777351349592209, 0.17690671980381012, 0.1761108785867691, 0.1753472238779068, 0.17461541295051575, 0.17391513288021088, 0.17324604094028473, 0.17260779440402985, 0.17200009524822235, 0.17142261564731598, 0.17087505757808685, 0.1703571230173111, 0.16986852884292603, 0.16940900683403015, 0.1689782589673996, 0.16857604682445526, 0.16820210218429565, 0.16785620152950287, 0.1675380915403366, 0.16724757850170135, 0.1669844388961792, 0.16674847900867462, 0.1665395051240921, 0.1663573533296585, 0.16620184481143951, 0.16607284545898438, 0.16597020626068115, 0.1658938229084015, 0.16584354639053345, 0.16581930220127106, 0.16582101583480835, 0.16584858298301697, 0.16590197384357452, 0.16598112881183624, 0.16608601808547974, 0.166216641664505, 0.16637298464775085, 0.16655504703521729, 0.16676288843154907, 0.16699650883674622, 0.20247891545295715, 0.2008010745048523, 0.19916445016860962, 0.19756869971752167, 0.196013405919075, 0.1944982260465622, 0.1930227428674698, 0.19158658385276794, 0.19018936157226562, 0.1888306587934494, 0.18751010298728943, 0.18622729182243347, 0.1849818229675293, 0.18377329409122467, 0.18260131776332855, 0.1814654916524887, 0.1803654134273529, 0.1793006807565689, 0.17827092111110687, 0.17727571725845337, 0.17631471157073975, 0.17538748681545258, 0.174493670463562, 0.1736328899860382, 0.17280477285385132, 0.1720089316368103, 0.1712450236082077, 0.17051267623901367, 0.16981153190135956, 0.1691412478685379, 0.16850149631500244, 0.16789191961288452, 0.16731220483779907, 0.16676202416419983, 0.16624106466770172, 0.16574902832508087, 0.1652856171131134, 0.16485053300857544, 0.1644435077905655, 0.16406428813934326, 0.16371259093284607, 0.16338816285133362, 0.1630907952785492, 0.1628202348947525, 0.16257627308368683, 0.16235868632793427, 0.16216731071472168, 0.16200195252895355, 0.16186243295669556, 0.16174858808517456, 0.161660298705101, 0.16159740090370178, 0.1615597903728485, 0.16154734790325165, 0.16155999898910522, 0.16159765422344208, 0.16166025400161743, 0.16174772381782532, 0.16186004877090454, 0.16199719905853271, 0.16215915977954865, 0.16234593093395233, 0.16255755722522736, 0.16279403865337372, 0.1967642903327942, 0.19513076543807983, 0.1935381442308426, 0.19198602437973022, 0.1904740333557129, 0.18900176882743835, 0.187568798661232, 0.18617475032806396, 0.18481919169425964, 0.183501735329628, 0.18222194910049438, 0.1809794306755066, 0.17977376282215118, 0.17860454320907593, 0.1774713546037674, 0.17637377977371216, 0.17531141638755798, 0.17428386211395264, 0.1732906997203827, 0.1723315417766571, 0.17140597105026245, 0.1705136001110077, 0.16965404152870178, 0.1688269078731537, 0.16803181171417236, 0.16726836562156677, 0.16653621196746826, 0.1658349633216858, 0.1651642769575119, 0.16452379524707794, 0.16391316056251526, 0.1633320301771164, 0.16278007626533508, 0.16225697100162506, 0.16176238656044006, 0.1612960249185562, 0.16085755825042725, 0.16044671833515167, 0.16006320714950562, 0.15970675647258759, 0.1593770831823349, 0.15907396376132965, 0.15879711508750916, 0.1585463285446167, 0.158321350812912, 0.1581220030784607, 0.1579480618238449, 0.1577993482351303, 0.15767568349838257, 0.15757688879966736, 0.15750281512737274, 0.15745332837104797, 0.1574282944202423, 0.1574275940656662, 0.15745113790035248, 0.15749883651733398, 0.15757061541080475, 0.1576664000749588, 0.15778616070747375, 0.15792986750602722, 0.15809749066829681, 0.15828903019428253, 0.15850451588630676, 0.1587439477443695])

# Y_RAND = np.array([0.9228929281234741, 0.9210525751113892, 0.919157087802887, 0.9172049760818481, 0.9151946902275085, 0.9131246209144592, 0.9109931588172913, 0.9087986946105957, 0.9065396189689636, 0.9042143225669861, 0.9018210172653198, 0.8993582129478455, 0.8968241214752197, 0.8942171335220337, 0.8915355205535889, 0.8887777328491211, 0.8859419822692871, 0.8830267786979675, 0.8800304532051086, 0.8769513964653015, 0.8737881183624268, 0.8705390691757202, 0.8672026991844177, 0.8637776970863342, 0.8602626323699951, 0.8566561937332153, 0.8529571890830994, 0.8491643071174622, 0.8452765941619873, 0.8412929773330688, 0.8372125625610352, 0.8330345749855042, 0.8287582397460938, 0.824383020401001, 0.8199084997177124, 0.8153342008590698, 0.8106600642204285, 0.8058860301971436, 0.8010120987892151, 0.7960386276245117, 0.7909659743309021, 0.785794734954834, 0.7805256247520447, 0.7751596570014954, 0.7696978449821472, 0.7641414999961853, 0.758492112159729, 0.7527512907981873, 0.7469209432601929, 0.7410030364990234, 0.7349998354911804, 0.7289136648178101, 0.7227471470832825, 0.7165030837059021, 0.7101842761039734, 0.7037939429283142, 0.6973353028297424, 0.6908117532730103, 0.6842268109321594, 0.6775842905044556, 0.6708879470825195, 0.6641417741775513, 0.6573497653007507, 0.6505162119865417, 0.9232547283172607, 0.9214298725128174, 0.9195504188537598, 0.9176146984100342, 0.9156211614608765, 0.9135683178901672, 0.9114544987678528, 0.9092780351638794, 0.9070373773574829, 0.9047308564186096, 0.902356743812561, 0.8999134302139282, 0.8973991870880127, 0.8948123455047607, 0.8921512365341187, 0.8894141912460327, 0.8865995407104492, 0.8837056159973145, 0.8807307481765747, 0.8776733875274658, 0.8745319247245789, 0.8713048100471497, 0.8679905533790588, 0.864587664604187, 0.8610947132110596, 0.8575103282928467, 0.8538333177566528, 0.850062370300293, 0.8461963534355164, 0.8422342538833618, 0.8381750583648682, 0.8340179324150085, 0.8297621011734009, 0.8254069685935974, 0.8209519386291504, 0.8163967132568359, 0.8117409348487854, 0.8069846034049988, 0.8021276593208313, 0.7971703410148621, 0.7921130061149597, 0.7869561314582825, 0.7817004323005676, 0.7763468027114868, 0.7708961963653564, 0.7653499841690063, 0.7597094774246216, 0.7539762854576111, 0.7481522560119629, 0.7422392964363098, 0.7362396717071533, 0.7301556468009949, 0.7239897847175598, 0.7177448868751526, 0.7114237546920776, 0.7050295472145081, 0.6985654830932617, 0.6920349597930908, 0.6854414939880371, 0.6787889003753662, 0.672080934047699, 0.6653215289115906, 0.6585148572921753, 0.6516650915145874, 0.923588752746582, 0.9217789769172668, 0.9199149012565613, 0.917995035648346, 0.9160177707672119, 0.913981556892395, 0.9118848443031311, 0.9097259044647217, 0.9075031280517578, 0.9052148461341858, 0.9028593897819519, 0.9004350900650024, 0.8979402780532837, 0.8953731656074524, 0.8927321434020996, 0.8900154829025269, 0.8872215151786804, 0.8843485116958618, 0.8813948631286621, 0.8783589005470276, 0.8752390742301941, 0.8720337152481079, 0.8687413334846497, 0.8653604388237, 0.8618895411491394, 0.8583272695541382, 0.8546723127365112, 0.8509233593940735, 0.8470792174339294, 0.8431388735771179, 0.8391011953353882, 0.8349653482437134, 0.8307304978370667, 0.8263959288597107, 0.8219611048698425, 0.817425549030304, 0.8127889037132263, 0.8080511093139648, 0.80321204662323, 0.798271894454956, 0.7932309508323669, 0.7880896329879761, 0.7828485369682312, 0.777508556842804, 0.7720707058906555, 0.7665359973907471, 0.7609059810638428, 0.7551820874214172, 0.7493661046028137, 0.743459939956665, 0.7374657392501831, 0.7313858270645142, 0.7252227067947388, 0.7189790606498718, 0.7126577496528625, 0.7062618732452393, 0.6997946500778198, 0.6932594180107117, 0.6866598129272461, 0.6799994707107544, 0.673282265663147, 0.666512131690979, 0.6596932411193848, 0.6528297662734985, 0.9238953590393066, 0.9220999479293823, 0.9202507734298706, 0.9183461666107178, 0.9163845777511597, 0.9143645167350769, 0.9122843146324158, 0.9101423025131226, 0.9079368710517883, 0.9056664109230042, 0.903329074382782, 0.9009233713150024, 0.8984474539756775, 0.8958995938301086, 0.8932782411575317, 0.8905815482139587, 0.8878078460693359, 0.8849554657936096, 0.8820226788520813, 0.879007875919342, 0.8759093880653381, 0.8727256059646606, 0.8694549798965454, 0.8660959601402283, 0.8626470565795898, 0.8591068387031555, 0.8554739952087402, 0.8517470955848694, 0.8479250073432922, 0.8440065979957581, 0.8399907350540161, 0.8358765244483948, 0.8316630125045776, 0.8273495435714722, 0.8229354619979858, 0.8184202313423157, 0.813803493976593, 0.8090850114822388, 0.8042647242546082, 0.799342691898346, 0.7943191528320312, 0.7891945242881775, 0.7839693427085876, 0.778644323348999, 0.7732204794883728, 0.7676988840103149, 0.7620809078216553, 0.7563679218292236, 0.7505617141723633, 0.7446641325950623, 0.7386772632598877, 0.7326033711433411, 0.7264449596405029, 0.7202047109603882, 0.7138853669166565, 0.7074900269508362, 0.7010219097137451, 0.6944843530654907, 0.6878809332847595, 0.6812152862548828, 0.674491286277771, 0.6677128672599792, 0.6608842611312866, 0.6540095210075378, 0.9241744875907898, 0.9223930239677429, 0.9205580949783325, 0.9186682105064392, 0.9167218208312988, 0.9147173166275024, 0.9126530289649963, 0.9105274677276611, 0.9083388447761536, 0.9060856103897095, 0.9037659764289856, 0.9013782739639282, 0.8989207744598389, 0.8963918089866638, 0.8937895894050598, 0.8911124467849731, 0.8883586525917053, 0.8855264782905579, 0.882614254951477, 0.8796202540397644, 0.876542866230011, 0.8733804225921631, 0.8701313734054565, 0.8667941093444824, 0.8633670806884766, 0.8598489165306091, 0.8562381267547607, 0.8525334000587463, 0.8487334847450256, 0.8448371887207031, 0.8408433794975281, 0.8367511034011841, 0.83255934715271, 0.8282673954963684, 0.8238745927810669, 0.8193802833557129, 0.8147841095924377, 0.8100857734680176, 0.8052850961685181, 0.800382137298584, 0.7953770756721497, 0.790270209312439, 0.7850620746612549, 0.7797533869743347, 0.7743449211120605, 0.7688378691673279, 0.7632334232330322, 0.7575330138206482, 0.7517382502555847, 0.7458510398864746, 0.7398734092712402, 0.7338075637817383, 0.7276558876037598, 0.7214210033416748, 0.7151058316230774, 0.7087132930755615, 0.7022465467453003, 0.6957089304924011, 0.6891040205955505, 0.6824355125427246, 0.6757071614265442, 0.668923020362854, 0.6620871424674988, 0.6552037596702576, 0.9244264364242554, 0.9226582646369934, 0.9208371043205261, 0.9189613461494446, 0.9170295596122742, 0.9150400161743164, 0.9129912257194519, 0.9108814597129822, 0.9087091684341431, 0.9064726233482361, 0.9041700959205627, 0.9017999172210693, 0.8993603587150574, 0.8968497514724731, 0.8942663073539734, 0.8916082978248596, 0.8888739943504333, 0.8860616683959961, 0.8831695914268494, 0.8801961541175842, 0.8771395683288574, 0.87399822473526, 0.8707704544067383, 0.8674547672271729, 0.8640495538711548, 0.8605533242225647, 0.856964647769928, 0.85328209400177, 0.8495044708251953, 0.8456304669380188, 0.841658890247345, 0.8375888466835022, 0.8334192037582397, 0.8291492462158203, 0.824778139591217, 0.8203054070472717, 0.8157304525375366, 0.8110529184341431, 0.8062727451324463, 0.801389753818512, 0.7964041233062744, 0.7913161516189575, 0.7861261963844299, 0.7808350324630737, 0.7754433751106262, 0.7699522972106934, 0.7643629312515259, 0.7586766481399536, 0.7528950572013855, 0.7470200061798096, 0.7410534024238586, 0.7349974513053894, 0.7288545370101929, 0.7226272225379944, 0.7163183093070984, 0.7099307179450989, 0.703467607498169, 0.6969323754310608, 0.6903283596038818, 0.6836593747138977, 0.6769291758537292, 0.6701417565345764, 0.6633011698722839, 0.6564117074012756, 0.9246513247489929, 0.9228959083557129, 0.921087920665741, 0.9192257523536682, 0.9173079133033752, 0.9153328537940979, 0.9132989645004272, 0.9112045168876648, 0.9090479612350464, 0.9068275094032288, 0.9045416116714478, 0.9021884799003601, 0.8997663855552673, 0.897273600101471, 0.8947084546089172, 0.8920691013336182, 0.88935387134552, 0.8865610361099243, 0.8836888074874878, 0.8807355165481567, 0.8776994347572327, 0.8745788931846619, 0.8713722825050354, 0.8680779933929443, 0.8646944165229797, 0.8612200021743774, 0.8576533794403076, 0.8539930582046509, 0.8502377867698669, 0.8463861346244812, 0.8424370884895325, 0.83838951587677, 0.8342423439025879, 0.8299947381019592, 0.8256458640098572, 0.8211951851844788, 0.8166420459747314, 0.8119861483573914, 0.8072271347045898, 0.8023650050163269, 0.7973998188972473, 0.7923317551612854, 0.7871612310409546, 0.7818887829780579, 0.7765151858329773, 0.7710414528846741, 0.7654686570167542, 0.7597981095314026, 0.7540314197540283, 0.7481702566146851, 0.7422165274620056, 0.7361723780632019, 0.7300402522087097, 0.7238225340843201, 0.7175220251083374, 0.7111415863037109, 0.7046844363212585, 0.6981537938117981, 0.6915531754493713, 0.6848861575126648, 0.6781565546989441, 0.6713683605194092, 0.6645256876945496, 0.6576327085494995, 0.9248493313789368, 0.9231060743331909, 0.9213106632232666, 0.9194615483283997, 0.9175571799278259, 0.9155960083007812, 0.9135763645172119, 0.9114966988563538, 0.9093552827835083, 0.9071505069732666, 0.904880702495575, 0.9025440812110901, 0.9001389145851135, 0.8976635336875916, 0.8951161503791809, 0.8924950361251831, 0.8897984623908997, 0.8870246410369873, 0.8841719031333923, 0.8812384009361267, 0.8782225251197815, 0.8751225471496582, 0.8719368577003479, 0.8686637282371521, 0.8653016090393066, 0.8618490099906921, 0.8583043217658997, 0.8546662330627441, 0.850933313369751, 0.8471042513847351, 0.8431777954101562, 0.8391529321670532, 0.8350284695625305, 0.830803632736206, 0.8264774680137634, 0.8220493197441101, 0.8175186514854431, 0.8128849864006042, 0.8081480264663696, 0.8033075928688049, 0.7983637452125549, 0.7933166027069092, 0.7881665229797363, 0.7829140424728394, 0.7775598168373108, 0.772104799747467, 0.7665500640869141, 0.7608968019485474, 0.7551465630531311, 0.749301016330719, 0.7433620095252991, 0.7373316287994385, 0.7312121987342834, 0.725006103515625, 0.7187161445617676, 0.7123451232910156, 0.7058961987495422, 0.6993725299835205, 0.6927775740623474, 0.6861149668693542, 0.6793885231018066, 0.672602117061615, 0.6657598614692688, 0.6588659286499023, 0.9250205159187317, 0.923288881778717, 0.9215055108070374, 0.9196688532829285, 0.9177773594856262, 0.9158294796943665, 0.913823664188385, 0.9117581844329834, 0.9096314311027527, 0.9074417352676392, 0.9051874279975891, 0.902866780757904, 0.9004781246185303, 0.8980196118354797, 0.8954895734786987, 0.892886221408844, 0.8902078866958618, 0.8874527215957642, 0.8846189975738525, 0.8817049860954285, 0.8787089586257935, 0.8756292462348938, 0.872464120388031, 0.8692119717597961, 0.8658711910247803, 0.8624401688575745, 0.8589174747467041, 0.8553014993667603, 0.8515909910202026, 0.8477845191955566, 0.8438808917999268, 0.8398789167404175, 0.8357775211334229, 0.8315757513046265, 0.8272727131843567, 0.8228676319122314, 0.8183599710464478, 0.8137491941452026, 0.8090349435806274, 0.8042170405387878, 0.7992954254150391, 0.7942702174186707, 0.7891417145729065, 0.78391033411026, 0.7785767316818237, 0.7731418013572693, 0.7676064968109131, 0.7619721293449402, 0.7562399506568909, 0.7504117488861084, 0.7444891929626465, 0.7384744882583618, 0.7323696613311768, 0.7261773347854614, 0.7199000120162964, 0.7135405540466309, 0.7071020603179932, 0.7005876898765564, 0.6940008401870728, 0.6873450875282288, 0.68062424659729, 0.6738421320915222, 0.6670029163360596, 0.6601106524467468, 0.9251651167869568, 0.9234445691108704, 0.9216726422309875, 0.919847846031189, 0.91796875, 0.9160336256027222, 0.9140409231185913, 0.9119890928268433, 0.9098764657974243, 0.907701313495636, 0.9054620265960693, 0.9031568765640259, 0.9007840752601624, 0.8983420133590698, 0.8958287835121155, 0.8932427763938904, 0.8905822038650513, 0.8878452181816101, 0.8850301504135132, 0.882135272026062, 0.8791587948799133, 0.8760990500450134, 0.872954249382019, 0.8697229027748108, 0.8664032220840454, 0.8629936575889587, 0.8594927787780762, 0.8558989763259888, 0.8522108197212219, 0.8484270572662354, 0.8445463180541992, 0.8405674695968628, 0.8364893198013306, 0.8323109149932861, 0.8280313611030579, 0.8236498832702637, 0.8191657662391663, 0.8145785331726074, 0.809887707233429, 0.8050930500030518, 0.8001945614814758, 0.7951922416687012, 0.7900863289833069, 0.7848771810531616, 0.7795655131340027, 0.7741519212722778, 0.7686375379562378, 0.7630234360694885, 0.7573109865188599, 0.751501739025116, 0.7455974817276001, 0.7396001815795898, 0.7335120439529419, 0.7273353934288025, 0.721072793006897, 0.7147271633148193, 0.708301305770874, 0.7017985582351685, 0.6952221989631653, 0.6885757446289062, 0.6818629503250122, 0.6750877499580383, 0.6682540774345398, 0.6613661050796509, 0.9252832531929016, 0.9235731363296509, 0.9218121767044067, 0.9199987649917603, 0.9181313514709473, 0.9162084460258484, 0.9142284393310547, 0.9121896624565125, 0.9100905656814575, 0.9079294800758362, 0.9057046175003052, 0.9034144282341003, 0.9010570645332336, 0.8986308574676514, 0.8961340188980103, 0.8935648798942566, 0.8909215927124023, 0.888202428817749, 0.8854056000709534, 0.8825294375419617, 0.8795721530914307, 0.8765320181846619, 0.8734073638916016, 0.8701964616775513, 0.866897702217102, 0.863509476184845, 0.8600302934646606, 0.8564585447311401, 0.8527927994728088, 0.8490317463874817, 0.8451740145683289, 0.8412184119224548, 0.8371638059616089, 0.8330091238021851, 0.8287534117698669, 0.8243958950042725, 0.8199358582496643, 0.8153727054595947, 0.8107059597969055, 0.8059354424476624, 0.8010608553886414, 0.7960823774337769, 0.7910000085830688, 0.7858142852783203, 0.7805256247520447, 0.7751347422599792, 0.7696425914764404, 0.7640502452850342, 0.7583590745925903, 0.7525704503059387, 0.7466862201690674, 0.7407082319259644, 0.7346385717391968, 0.7284796237945557, 0.7222338914871216, 0.7159041166305542, 0.7094932198524475, 0.7030043601989746, 0.6964408159255981, 0.6898061037063599, 0.6831038594245911, 0.6763380169868469, 0.6695125102996826, 0.6626315116882324, 0.9253749847412109, 0.9236748814582825, 0.9219242334365845, 0.9201216101646423, 0.9182654619216919, 0.9163541793823242, 0.9143862128257751, 0.9123600125312805, 0.9102739691734314, 0.9081262946128845, 0.905915379524231, 0.9036396145820618, 0.9012971520423889, 0.8988863229751587, 0.8964053988456726, 0.8938525915145874, 0.8912261724472046, 0.8885243535041809, 0.8857454657554626, 0.882887601852417, 0.879949152469635, 0.8769283294677734, 0.8738234639167786, 0.8706328272819519, 0.8673548102378845, 0.8639877438545227, 0.8605300784111023, 0.8569803237915039, 0.8533369898796082, 0.8495986461639404, 0.8457640409469604, 0.8418318629264832, 0.837800920009613, 0.8336701989173889, 0.8294386863708496, 0.825105607509613, 0.8206700682640076, 0.816131591796875, 0.8114896416664124, 0.806743860244751, 0.8018940687179565, 0.796940267086029, 0.7918825149536133, 0.7867211699485779, 0.781456708908081, 0.7760897874832153, 0.7706212401390076, 0.7650521397590637, 0.7593836784362793, 0.7536173462867737, 0.7477548122406006, 0.7417978644371033, 0.7357486486434937, 0.7296093702316284, 0.7233825325965881, 0.7170707583427429, 0.7106770277023315, 0.7042043209075928, 0.697655975818634, 0.6910353302955627, 0.6843461990356445, 0.6775922179222107, 0.670777440071106, 0.6639059782028198, 0.9254405498504639, 0.9237498641014099, 0.9220090508460999, 0.9202166199684143, 0.9183711409568787, 0.9164709448814392, 0.9145145416259766, 0.9125003218650818, 0.9104267358779907, 0.9082919955253601, 0.906094491481781, 0.9038326144218445, 0.9015045762062073, 0.8991085886955261, 0.8966431021690369, 0.8941062092781067, 0.8914961814880371, 0.8888113498687744, 0.8860498666763306, 0.8832100033760071, 0.8802900314331055, 0.8772881627082825, 0.8742027878761292, 0.8710321187973022, 0.8677745461463928, 0.8644284605979919, 0.8609922528266907, 0.8574643731117249, 0.8538433909416199, 0.8501278162002563, 0.8463163375854492, 0.8424077033996582, 0.838400661945343, 0.8342941999435425, 0.8300871849060059, 0.8257788419723511, 0.821368396282196, 0.8168550729751587, 0.8122385144233704, 0.8075181841850281, 0.8026939630508423, 0.7977656722068787, 0.7927334904670715, 0.7875975966453552, 0.7823584675788879, 0.7770166993141174, 0.7715730667114258, 0.7660285830497742, 0.7603843808174133, 0.7546419501304626, 0.7488027811050415, 0.7428686618804932, 0.7368416786193848, 0.730724036693573, 0.7245180606842041, 0.718226432800293, 0.7118520140647888, 0.7053977251052856, 0.6988668441772461, 0.6922627687454224, 0.685589075088501, 0.6788495182991028, 0.672048032283783, 0.6651886701583862, 0.92548006772995, 0.9237982630729675, 0.9220666885375977, 0.9202839732170105, 0.9184485673904419, 0.9165589213371277, 0.9146135449409485, 0.9126107692718506, 0.9105490446090698, 0.9084267616271973, 0.9062421321868896, 0.9039936065673828, 0.9016794562339783, 0.8992978930473328, 0.8968473076820374, 0.894325852394104, 0.8917317986488342, 0.8890634179115295, 0.8863189816474915, 0.8834967017173767, 0.8805948495864868, 0.8776116371154785, 0.8745454549789429, 0.8713945150375366, 0.8681572079658508, 0.8648319244384766, 0.8614169955253601, 0.8579108715057373, 0.8543121814727783, 0.850619375705719, 0.8468310832977295, 0.8429460525512695, 0.8389630913734436, 0.834881067276001, 0.8306989073753357, 0.8264157176017761, 0.8220306634902954, 0.817543089389801, 0.81295245885849, 0.8082582950592041, 0.8034603595733643, 0.7985584735870361, 0.7935527563095093, 0.7884433269500732, 0.7832306623458862, 0.7779151797294617, 0.7724977135658264, 0.7669792175292969, 0.761360764503479, 0.7556437253952026, 0.7498295307159424, 0.7439200282096863, 0.7379171252250671, 0.7318229675292969, 0.7256399393081665, 0.7193704843521118, 0.7130175232887268, 0.7065839171409607, 0.7000728249549866, 0.6934876441955566, 0.6868318319320679, 0.6801092028617859, 0.6733235120773315, 0.6664788722991943, 0.9254936575889587, 0.9238201379776001, 0.9220973253250122, 0.9203237295150757, 0.9184979200363159, 0.916618287563324, 0.9146833419799805, 0.9126915335655212, 0.9106411933898926, 0.9085307121276855, 0.9063584804534912, 0.9041228294372559, 0.901822030544281, 0.8994544148445129, 0.8970181941986084, 0.8945116996765137, 0.8919332027435303, 0.8892809152603149, 0.8865531086921692, 0.8837479948997498, 0.8808638453483582, 0.8778989911079407, 0.8748517036437988, 0.8717201948165894, 0.8685029149055481, 0.8651981949806213, 0.8618043661117554, 0.8583199977874756, 0.8547434210777283, 0.8510733842849731, 0.847308337688446, 0.8434470891952515, 0.8394883275032043, 0.835430920124054, 0.8312738537788391, 0.8270161151885986, 0.8226569294929504, 0.8181955814361572, 0.8136314749717712, 0.8089640736579895, 0.8041931390762329, 0.7993184924125671, 0.7943400740623474, 0.7892581224441528, 0.7840729355812073, 0.7787849307060242, 0.7733949422836304, 0.767903745174408, 0.7623124122619629, 0.7566222548484802, 0.7508347034454346, 0.7449514865875244, 0.7389744520187378, 0.7329056859016418, 0.726747453212738, 0.7205022573471069, 0.7141728401184082, 0.7077620625495911, 0.7012730836868286, 0.694709062576294, 0.6880736351013184, 0.6813703179359436, 0.6746029853820801, 0.6677756309509277, 0.9254814386367798, 0.9238157272338867, 0.9221011400222778, 0.920336127281189, 0.9185193181037903, 0.9166491627693176, 0.9147241711616516, 0.9127427339553833, 0.9107032418251038, 0.9086041450500488, 0.9064437747001648, 0.904220461845398, 0.9019325375556946, 0.8995782732963562, 0.8971560597419739, 0.8946640491485596, 0.8921006321907043, 0.8894639611244202, 0.8867523670196533, 0.8839640617370605, 0.8810973167419434, 0.878150463104248, 0.8751217126846313, 0.8720093965530396, 0.8688119053840637, 0.8655275106430054, 0.8621546626091003, 0.8586918115615845, 0.8551374077796936, 0.8514900207519531, 0.8477482795715332, 0.843910813331604, 0.8399763703346252, 0.8359438180923462, 0.8318120837211609, 0.8275802135467529, 0.8232473134994507, 0.8188126087188721, 0.8142755031585693, 0.8096355199813843, 0.8048923015594482, 0.8000456094741821, 0.7950953841209412, 0.7900418043136597, 0.7848851084709167, 0.7796257734298706, 0.774264395236969, 0.768801748752594, 0.7632389664649963, 0.7575772404670715, 0.7518178820610046, 0.7459625601768494, 0.7400131225585938, 0.7339715361595154, 0.7278400659561157, 0.7216211557388306, 0.7153173685073853, 0.7089316248893738, 0.7024669051170349, 0.6959264278411865, 0.6893136501312256, 0.6826321482658386, 0.6758856177330017, 0.6690781116485596, 0.9254435896873474, 0.9237851500511169, 0.9220781922340393, 0.9203212857246399, 0.9185130000114441, 0.9166517853736877, 0.9147361516952515, 0.9127645492553711, 0.9107354283332825, 0.9086471199989319, 0.9064980745315552, 0.9042866230010986, 0.9020110964775085, 0.8996697664260864, 0.8972610235214233, 0.894783079624176, 0.8922342658042908, 0.8896127939224243, 0.886916995048523, 0.8841450810432434, 0.881295382976532, 0.8783661723136902, 0.8753557205200195, 0.8722622990608215, 0.8690842986106873, 0.865820050239563, 0.8624680042266846, 0.8590266108512878, 0.8554942607879639, 0.851869523525238, 0.8481510281562805, 0.8443374037742615, 0.8404274582862854, 0.836419939994812, 0.8323137760162354, 0.828108012676239, 0.8238017559051514, 0.8193941116333008, 0.814884603023529, 0.8102725744247437, 0.8055577278137207, 0.8007397651672363, 0.7958186268806458, 0.790794312953949, 0.7856671214103699, 0.7804374098777771, 0.7751058340072632, 0.7696731090545654, 0.7641401886940002, 0.7585082650184631, 0.7527785897254944, 0.7469528317451477, 0.7410327196121216, 0.735020101070404, 0.7289173007011414, 0.7227265238761902, 0.7164504528045654, 0.7100918292999268, 0.7036536335945129, 0.6971389055252075, 0.6905511617660522, 0.6838938593864441, 0.6771706342697144, 0.670385479927063, 0.9253801107406616, 0.9237284660339355, 0.922028660774231, 0.920279324054718, 0.9184790253639221, 0.9166262149810791, 0.9147194623947144, 0.9127572178840637, 0.910737931728363, 0.9086599349975586, 0.906521737575531, 0.9043216109275818, 0.902057945728302, 0.8997291326522827, 0.8973333835601807, 0.8948690295219421, 0.8923343420028687, 0.8897277116775513, 0.887047290802002, 0.884291410446167, 0.8814584016799927, 0.8785464763641357, 0.8755539655685425, 0.8724791407585144, 0.8693204522132874, 0.8660761713981628, 0.8627447485923767, 0.85932457447052, 0.8558141589164734, 0.8522120714187622, 0.8485168814659119, 0.8447271585464478, 0.8408417701721191, 0.8368594646453857, 0.8327791094779968, 0.828599750995636, 0.824320375919342, 0.8199403285980225, 0.8154588341712952, 0.8108753561973572, 0.8061895370483398, 0.8014009594917297, 0.7965096235275269, 0.7915155291557312, 0.7864187955856323, 0.7812198400497437, 0.7759191393852234, 0.7705175280570984, 0.7650157809257507, 0.7594150304794312, 0.7537165880203247, 0.7479219436645508, 0.7420327663421631, 0.7360509037971497, 0.7299785614013672, 0.7238179445266724, 0.7175715565681458, 0.711242139339447, 0.7048325538635254, 0.6983458995819092, 0.691785454750061, 0.6851546764373779, 0.6784572005271912, 0.6716968417167664, 0.9252912402153015, 0.9236457943916321, 0.9219526648521423, 0.9202104210853577, 0.9184175729751587, 0.9165726900100708, 0.9146742820739746, 0.9127208590507507, 0.910710871219635, 0.9086427688598633, 0.9065148830413818, 0.9043256044387817, 0.9020733833312988, 0.8997564911842346, 0.8973733186721802, 0.8949220776557922, 0.8924012184143066, 0.8898088932037354, 0.8871434926986694, 0.8844032883644104, 0.8815865516662598, 0.8786916136741638, 0.8757167458534241, 0.8726602792739868, 0.8695206046104431, 0.8662960529327393, 0.8629850745201111, 0.8595860600471497, 0.8560974597930908, 0.8525179624557495, 0.8488459587097168, 0.8450802564620972, 0.8412195444107056, 0.8372625708580017, 0.8332082033157349, 0.8290554881095886, 0.8248034715652466, 0.8204513192176819, 0.8159983158111572, 0.8114439249038696, 0.8067876696586609, 0.8020292520523071, 0.797168493270874, 0.7922053933143616, 0.7871400713920593, 0.7819728255271912, 0.7767041921615601, 0.7713347673416138, 0.7658655047416687, 0.7602973580360413, 0.754631519317627, 0.7488695383071899, 0.7430129051208496, 0.7370635271072388, 0.7310234308242798, 0.7248947620391846, 0.7186800837516785, 0.7123818397521973, 0.7060030102729797, 0.6995465755462646, 0.6930157542228699, 0.6864138245582581, 0.6797444820404053, 0.6730113625526428, 0.9251770377159119, 0.9235373735427856, 0.9218503832817078, 0.9201146364212036, 0.9183287620544434, 0.9164913296699524, 0.9146007895469666, 0.9126557111740112, 0.9106545448303223, 0.9085957407951355, 0.906477689743042, 0.9042988419532776, 0.9020575881004333, 0.8997521996498108, 0.897381067276001, 0.894942581653595, 0.8924350142478943, 0.8898566961288452, 0.8872058987617493, 0.8844809532165527, 0.8816801309585571, 0.8788018226623535, 0.8758442997932434, 0.8728059530258179, 0.8696850538253784, 0.8664799928665161, 0.8631892800331116, 0.8598112463951111, 0.85634446144104, 0.8527873754501343, 0.8491386771202087, 0.8453969955444336, 0.841560959815979, 0.8376294374465942, 0.8336013555526733, 0.8294755220413208, 0.8252511024475098, 0.8209272623062134, 0.8165032267570496, 0.8119783997535706, 0.8073523640632629, 0.8026247024536133, 0.7977952361106873, 0.7928639650344849, 0.7878308892250061, 0.7826963663101196, 0.7774608135223389, 0.7721248269081116, 0.7666892409324646, 0.7611549496650696, 0.7555232048034668, 0.7497952580451965, 0.7439728379249573, 0.738057553768158, 0.732051432132721, 0.7259566187858582, 0.7197754383087158, 0.7135104537010193, 0.707164466381073, 0.7007403373718262, 0.6942412853240967, 0.6876705288887024, 0.6810316443443298, 0.6743282079696655, 0.9250376224517822, 0.923403263092041, 0.9217219352722168, 0.9199922680854797, 0.918212890625, 0.9163823127746582, 0.9144991636276245, 0.9125619530677795, 0.9105691313743591, 0.9085191488265991, 0.9064105153083801, 0.9042415618896484, 0.9020107388496399, 0.8997164368629456, 0.8973569869995117, 0.8949307203292847, 0.8924360871315002, 0.8898712992668152, 0.8872347474098206, 0.8845247030258179, 0.8817394971847534, 0.8788775205612183, 0.8759370446205139, 0.8729164600372314, 0.8698140978813171, 0.8666283488273621, 0.863357663154602, 0.8600005507469177, 0.8565554022789001, 0.8530207872390747, 0.8493952751159668, 0.8456776142120361, 0.8418663740158081, 0.8379604816436768, 0.8339587450027466, 0.8298600912094116, 0.8256635665893555, 0.821368396282196, 0.8169737458229065, 0.8124790191650391, 0.8078836798667908, 0.8031874299049377, 0.7983900308609009, 0.7934913039207458, 0.788491427898407, 0.7833905220031738, 0.7781890630722046, 0.7728875875473022, 0.7674868106842041, 0.7619876861572266, 0.7563912868499756, 0.750698983669281, 0.7449121475219727, 0.7390326261520386, 0.7330621480941772, 0.7270029187202454, 0.7208572030067444, 0.7146273851394653, 0.7083162069320679, 0.7019265294075012, 0.6954613924026489, 0.6889240741729736, 0.6823179125785828, 0.6756464838981628, 0.9248731732368469, 0.9232435822486877, 0.9215673804283142, 0.919843316078186, 0.9180698990821838, 0.9162458181381226, 0.9143695831298828, 0.9124397039413452, 0.9104547500610352, 0.9084131717681885, 0.906313419342041, 0.9041539430618286, 0.9019331336021423, 0.899649441242218, 0.897301197052002, 0.89488685131073, 0.8924046754837036, 0.8898530602455139, 0.8872303366661072, 0.8845348358154297, 0.8817648887634277, 0.8789189457893372, 0.8759952187538147, 0.8729920983314514, 0.8699080944061279, 0.866741418838501, 0.8634907007217407, 0.8601542115211487, 0.8567306399345398, 0.8532183766365051, 0.8496161103248596, 0.8459224700927734, 0.8421361446380615, 0.8382559418678284, 0.8342807292938232, 0.8302094340324402, 0.8260411620140076, 0.8217748999595642, 0.8174100518226624, 0.8129458427429199, 0.8083818554878235, 0.8037176132202148, 0.7989528775215149, 0.7940875291824341, 0.7891215682029724, 0.784055233001709, 0.7788888216018677, 0.7736229300498962, 0.7682581543922424, 0.7627953886985779, 0.7572357058525085, 0.7515802979469299, 0.7458306550979614, 0.739988386631012, 0.7340552806854248, 0.7280333638191223, 0.7219247817993164, 0.7157320380210876, 0.7094576954841614, 0.7031044960021973, 0.6966754198074341, 0.6901736259460449, 0.6836024522781372, 0.6769653558731079, 0.924683690071106, 0.9230584502220154, 0.9213869571685791, 0.9196679592132568, 0.9179001450538635, 0.9160820245742798, 0.914212167263031, 0.9122892618179321, 0.9103116989135742, 0.9082779884338379, 0.9061866998672485, 0.9040362238883972, 0.9018250107765198, 0.899551510810852, 0.8972141146659851, 0.8948111534118652, 0.8923410773277283, 0.8898022174835205, 0.8871930241584778, 0.8845117092132568, 0.8817567229270935, 0.8789263963699341, 0.8760191798210144, 0.873033344745636, 0.8699673414230347, 0.8668196201324463, 0.8635885715484619, 0.8602727055549622, 0.8568705320358276, 0.853380560874939, 0.8498014807701111, 0.8461319208145142, 0.8423705101013184, 0.8385161757469177, 0.834567666053772, 0.8305239677429199, 0.8263840675354004, 0.8221471905708313, 0.817812442779541, 0.8133792877197266, 0.8088470697402954, 0.8042153716087341, 0.7994840145111084, 0.7946527004241943, 0.7897214889526367, 0.7846905589103699, 0.7795602083206177, 0.7743308544158936, 0.7690032124519348, 0.7635779976844788, 0.7580562829971313, 0.7524392008781433, 0.7467281222343445, 0.740924596786499, 0.7350304126739502, 0.7290474772453308, 0.722977876663208, 0.7168239951133728, 0.7105883359909058, 0.7042735815048218, 0.6978825926780701, 0.6914185285568237, 0.6848844885826111, 0.6782840490341187, 0.9244693517684937, 0.9228479862213135, 0.9211807250976562, 0.9194664359092712, 0.9177036285400391, 0.9158910512924194, 0.914027214050293, 0.9121106863021851, 0.9101400971412659, 0.9081138968467712, 0.9060305953025818, 0.9038887023925781, 0.9016866683959961, 0.8994228839874268, 0.8970958590507507, 0.8947039842605591, 0.8922456502914429, 0.8897191882133484, 0.8871230483055115, 0.8844556212425232, 0.8817152976989746, 0.8789003491401672, 0.8760092854499817, 0.873040497303009, 0.8699923157691956, 0.8668632507324219, 0.8636518120765686, 0.860356330871582, 0.8569754958152771, 0.8535078167915344, 0.8499518632888794, 0.846306324005127, 0.8425699472427368, 0.8387414813041687, 0.8348198533058167, 0.8308039307594299, 0.8266927599906921, 0.8224855065345764, 0.8181812763214111, 0.8137795329093933, 0.8092795610427856, 0.8046810626983643, 0.7999836206436157, 0.7951870560646057, 0.790291428565979, 0.7852967381477356, 0.7802032828330994, 0.7750114798545837, 0.7697219252586365, 0.7643354535102844, 0.7588528990745544, 0.7532753944396973, 0.747604250907898, 0.7418410181999207, 0.7359872460365295, 0.7300449013710022, 0.7240159511566162, 0.7179027199745178, 0.711707592010498, 0.705433189868927, 0.6990823745727539, 0.692658007144928, 0.6861633062362671, 0.6796016097068787, 0.9242302775382996, 0.9226123094558716, 0.92094886302948, 0.9192387461662292, 0.9174805879592896, 0.9156730771064758, 0.9138147830963135, 0.9119042754173279, 0.9099401831626892, 0.9079210162162781, 0.9058452844619751, 0.9037115573883057, 0.9015182852745056, 0.899263858795166, 0.8969468474388123, 0.8945656418800354, 0.8921185731887817, 0.8896042108535767, 0.8870208859443665, 0.8843669891357422, 0.8816409111022949, 0.8788411021232605, 0.87596595287323, 0.8730139136314392, 0.8699833750724792, 0.8668728470802307, 0.8636807203292847, 0.8604055643081665, 0.8570459485054016, 0.8536004424095154, 0.8500675559043884, 0.84644615650177, 0.8427348136901855, 0.8389323949813843, 0.8350377082824707, 0.8310497999191284, 0.8269675970077515, 0.8227902054786682, 0.8185169100761414, 0.8141469359397888, 0.8096797466278076, 0.8051148653030396, 0.8004519939422607, 0.7956908345222473, 0.7908313870429993, 0.7858737707138062, 0.7808181047439575, 0.7756648063659668, 0.7704144716262817, 0.7650676965713501, 0.7596254944801331, 0.754088819026947, 0.748458981513977, 0.742737352848053, 0.7369255423545837, 0.7310253381729126, 0.7250387072563171, 0.7189677953720093, 0.7128150463104248, 0.7065828442573547, 0.7002740502357483, 0.6938914060592651, 0.6874381303787231, 0.6809173226356506, 0.9239665865898132, 0.9223515391349792, 0.9206914901733398, 0.91898512840271, 0.9172312021255493, 0.9154282808303833, 0.9135750532150269, 0.9116701483726501, 0.9097121357917786, 0.9076995849609375, 0.9056310653686523, 0.9035050868988037, 0.9013201594352722, 0.8990747332572937, 0.8967673182487488, 0.8943963646888733, 0.8919603228569031, 0.889457643032074, 0.8868867754936218, 0.8842461109161377, 0.8815340399742126, 0.8787490725517273, 0.8758895993232727, 0.8729540109634399, 0.8699409365653992, 0.8668486475944519, 0.8636757731437683, 0.860420823097229, 0.8570823073387146, 0.85365891456604, 0.8501491546630859, 0.846551775932312, 0.842865526676178, 0.8390892148017883, 0.8352217078208923, 0.831261932849884, 0.8272088766098022, 0.8230617046356201, 0.8188195824623108, 0.8144818544387817, 0.8100478649139404, 0.8055171370506287, 0.8008893728256226, 0.7961642742156982, 0.7913417816162109, 0.7864218950271606, 0.7814048528671265, 0.776291012763977, 0.7710807919502258, 0.7657748460769653, 0.7603740692138672, 0.7548794746398926, 0.7492921352386475, 0.7436134815216064, 0.7378450036048889, 0.7319884300231934, 0.7260456681251526, 0.7200188040733337, 0.7139101028442383, 0.7077219486236572, 0.7014570236206055, 0.6951181292533875, 0.6887081861495972, 0.6822304129600525, 0.9236782789230347, 0.9220658540725708, 0.9204087257385254, 0.9187057614326477, 0.9169555306434631, 0.9151568412780762, 0.913308322429657, 0.9114086031913757, 0.9094562530517578, 0.9074499607086182, 0.9053881764411926, 0.9032695293426514, 0.9010925889015198, 0.8988557457923889, 0.8965575695037842, 0.8941965699195862, 0.8917711973190308, 0.889279842376709, 0.886721134185791, 0.8840933442115784, 0.8813949823379517, 0.8786245584487915, 0.8757805228233337, 0.8728612661361694, 0.869865357875824, 0.8667912483215332, 0.8636374473571777, 0.860402524471283, 0.8570850491523743, 0.8536836504936218, 0.8501969575881958, 0.8466237187385559, 0.8429626226425171, 0.8392124772071838, 0.835372269153595, 0.8314407467842102, 0.8274171352386475, 0.8233004212379456, 0.8190898299217224, 0.8147846460342407, 0.8103842735290527, 0.8058882355690002, 0.801296055316925, 0.7966076135635376, 0.7918227314949036, 0.786941409111023, 0.7819637656211853, 0.7768901586532593, 0.7717210054397583, 0.7664569020271301, 0.7610986828804016, 0.7556471824645996, 0.7501036524772644, 0.7444692254066467, 0.7387455105781555, 0.7329339981079102, 0.7270366549491882, 0.7210553288459778, 0.714992344379425, 0.7088499665260315, 0.7026306986808777, 0.6963374018669128, 0.6899727582931519, 0.6835399866104126, 0.923365592956543, 0.9217552542686462, 0.920100748538971, 0.9184006452560425, 0.9166538715362549, 0.9148589968681335, 0.9130147099494934, 0.9111197590827942, 0.9091727137565613, 0.9071722030639648, 0.905116856098175, 0.9030051827430725, 0.9008358120918274, 0.8986071944236755, 0.8963179588317871, 0.893966555595398, 0.8915514349937439, 0.8890712261199951, 0.8865242600440979, 0.8839091062545776, 0.8812242746353149, 0.8784681558609009, 0.8756392598152161, 0.8727360963821411, 0.8697571754455566, 0.8667010068893433, 0.8635661602020264, 0.8603512048721313, 0.8570546507835388, 0.8536752462387085, 0.8502116203308105, 0.8466624617576599, 0.8430265188217163, 0.8393027186393738, 0.835489809513092, 0.8315868377685547, 0.8275927901268005, 0.823506772518158, 0.8193280100822449, 0.815055787563324, 0.810689389705658, 0.8062284588813782, 0.8016725182533264, 0.7970212697982788, 0.7922745943069458, 0.7874324917793274, 0.7824950218200684, 0.7774624824523926, 0.7723352909088135, 0.767113983631134, 0.7617992758750916, 0.7563920617103577, 0.7508934140205383, 0.745304524898529, 0.7396268248558044, 0.7338618040084839, 0.7280112504959106, 0.7220770716667175, 0.7160613536834717, 0.7099664211273193, 0.7037945985794067, 0.6975486278533936, 0.6912312507629395, 0.6848453283309937, 0.9230285286903381, 0.9214199781417847, 0.9197676181793213, 0.9180700778961182, 0.9163262248039246, 0.9145348072052002, 0.9126944541931152, 0.9108038544654846, 0.9088617563247681, 0.9068666696548462, 0.9048173427581787, 0.902712345123291, 0.9005501866340637, 0.8983294367790222, 0.8960487842559814, 0.8937066197395325, 0.8913015127182007, 0.8888320326805115, 0.8862966299057007, 0.8836938738822937, 0.8810221552848816, 0.8782801032066345, 0.8754661679267883, 0.8725789189338684, 0.8696168065071106, 0.8665784597396851, 0.8634624481201172, 0.8602672815322876, 0.8569915890693665, 0.8536341190338135, 0.8501935005187988, 0.8466684818267822, 0.8430577516555786, 0.8393602967262268, 0.8355749249458313, 0.8317005634307861, 0.8277363777160645, 0.8236812949180603, 0.8195346593856812, 0.8152956366539001, 0.8109637498855591, 0.8065382838249207, 0.8020190000534058, 0.7974055409431458, 0.7926976680755615, 0.7878954410552979, 0.7829988598823547, 0.7780081629753113, 0.7729237675666809, 0.7677461504936218, 0.7624759674072266, 0.7571141123771667, 0.751661479473114, 0.7461192607879639, 0.7404887676239014, 0.7347715497016907, 0.7289692163467407, 0.7230836153030396, 0.7171167731285095, 0.7110708355903625, 0.7049481272697449, 0.6987512707710266, 0.6924828290939331, 0.6861457824707031, 0.9226673245429993, 0.9210601449012756, 0.9194095134735107, 0.9177141785621643, 0.9159729480743408, 0.9141845107078552, 0.912347674369812, 0.9104611277580261, 0.9085235595703125, 0.9065335988998413, 0.9044899344444275, 0.9023911952972412, 0.9002359509468079, 0.8980228304862976, 0.8957503437995911, 0.8934171795845032, 0.8910217881202698, 0.8885627388954163, 0.886038601398468, 0.8834479451179504, 0.8807892203330994, 0.8780609965324402, 0.8752617835998535, 0.8723902106285095, 0.8694447875022888, 0.8664240837097168, 0.8633267283439636, 0.8601512908935547, 0.8568964600563049, 0.8535608649253845, 0.8501431941986084, 0.8466423153877258, 0.8430569171905518, 0.8393858671188354, 0.835628092288971, 0.8317825794219971, 0.8278483152389526, 0.8238244652748108, 0.8197101950645447, 0.815504789352417, 0.81120765209198, 0.8068181872367859, 0.8023359775543213, 0.7977607846260071, 0.7930923700332642, 0.788330614566803, 0.7834755778312683, 0.7785274982452393, 0.7734867334365845, 0.7683536410331726, 0.763128936290741, 0.7578133344650269, 0.7524077892303467, 0.7469133734703064, 0.7413313388824463, 0.7356631755828857, 0.7299103736877441, 0.7240747213363647, 0.7181581854820251, 0.7121627926826477, 0.7060908079147339, 0.6999446749687195, 0.6937269568443298, 0.6874404549598694, 0.9222819805145264, 0.9206758141517639, 0.9190265536308289, 0.9173330664634705, 0.9155940413475037, 0.9138083457946777, 0.9119746685028076, 0.9100918173789978, 0.9081584215164185, 0.9061732292175293, 0.9041349291801453, 0.9020420908927917, 0.8998934626579285, 0.897687554359436, 0.8954230546951294, 0.8930985331535339, 0.890712559223175, 0.8882637023925781, 0.8857505917549133, 0.883171796798706, 0.8805257678031921, 0.8778111934661865, 0.8750265836715698, 0.8721705079078674, 0.8692415952682495, 0.8662384152412415, 0.8631595969200134, 0.8600038290023804, 0.8567696809768677, 0.8534559607505798, 0.850061297416687, 0.8465844988822937, 0.8430244326591492, 0.8393799066543579, 0.8356499075889587, 0.8318333029747009, 0.8279292583465576, 0.8239368200302124, 0.8198552131652832, 0.8156837224960327, 0.8114216923713684, 0.8070685863494873, 0.8026239275932312, 0.798087477684021, 0.7934589982032776, 0.7887383103370667, 0.7839255332946777, 0.7790207862854004, 0.7740243673324585, 0.7689366340637207, 0.7637582421302795, 0.7584898471832275, 0.7531324028968811, 0.7476868629455566, 0.7421544790267944, 0.7365365028381348, 0.7308344841003418, 0.725050151348114, 0.7191852331161499, 0.7132418751716614, 0.7072221040725708, 0.7011283040046692, 0.6949629783630371, 0.6887287497520447, 0.9218726754188538, 0.9202671647071838, 0.9186189770698547, 0.916926920413971, 0.915189802646637, 0.9134064316749573, 0.9115756154060364, 0.909696102142334, 0.907766580581665, 0.905785858631134, 0.9037525057792664, 0.9016653299331665, 0.8995229601860046, 0.8973240256309509, 0.8950672149658203, 0.8927510976791382, 0.8903742432594299, 0.8879354000091553, 0.88543301820755, 0.882865846157074, 0.8802323341369629, 0.8775311708450317, 0.8747609853744507, 0.8719202876091003, 0.8690077066421509, 0.8660219311714172, 0.8629615902900696, 0.8598253726959229, 0.8566119074821472, 0.8533199429512024, 0.8499482870101929, 0.8464956879615784, 0.8429610133171082, 0.8393430709838867, 0.8356409072875977, 0.831853449344635, 0.8279797434806824, 0.8240189552307129, 0.8199702501296997, 0.8158329129219055, 0.8116062879562378, 0.8072899580001831, 0.8028833270072937, 0.7983860969543457, 0.79379802942276, 0.7891190052032471, 0.7843490839004517, 0.7794882655143738, 0.7745369076728821, 0.7694953680038452, 0.7643641233444214, 0.7591438889503479, 0.7538354396820068, 0.7484397888183594, 0.7429580092430115, 0.7373914122581482, 0.7317414283752441, 0.7260096073150635, 0.7201977372169495, 0.7143076658248901, 0.7083415985107422, 0.7023016810417175, 0.6961902976036072, 0.6900100111961365, 0.9214394688606262, 0.919834315776825, 0.9181868433952332, 0.9164959192276001, 0.9147603511810303, 0.912979006767273, 0.9111506938934326, 0.9092742204666138, 0.9073482751846313, 0.9053716659545898, 0.9033430814743042, 0.9012612700462341, 0.8991248607635498, 0.8969326615333557, 0.8946831822395325, 0.8923751711845398, 0.8900073170661926, 0.8875781297683716, 0.8850863575935364, 0.8825305700302124, 0.8799093961715698, 0.8772215247154236, 0.8744655251502991, 0.8716400265693665, 0.8687437176704407, 0.8657752275466919, 0.8627332448959351, 0.8596165180206299, 0.8564236760139465, 0.8531535267829895, 0.8498048782348633, 0.8463764786720276, 0.8428671956062317, 0.8392759561538696, 0.8356017470359802, 0.8318434953689575, 0.8280003070831299, 0.8240713477134705, 0.8200557827949524, 0.8159529566764832, 0.8117621541023254, 0.8074828386306763, 0.8031145930290222, 0.7986570596694946, 0.7941099405288696, 0.7894731163978577, 0.7847465872764587, 0.7799303531646729, 0.7750247716903687, 0.7700300812721252, 0.7649467587471008, 0.7597754597663879, 0.7545169591903687, 0.7491721510887146, 0.743742048740387, 0.7382278442382812, 0.7326309680938721, 0.726952850818634, 0.7211952805519104, 0.7153599262237549, 0.7094488739967346, 0.7034642100334167, 0.6974083185195923, 0.6912835240364075, 0.9209825992584229, 0.9193774461746216, 0.9177303314208984, 0.9160401821136475, 0.9143059253692627, 0.9125263094902039, 0.9107002019882202, 0.9088264107704163, 0.9069038033485413, 0.9049310088157654, 0.9029068946838379, 0.9008301496505737, 0.8986995220184326, 0.8965137004852295, 0.8942713737487793, 0.8919712901115417, 0.889612078666687, 0.88719242811203, 0.8847110271453857, 0.8821664452552795, 0.8795574307441711, 0.8768826723098755, 0.8741406798362732, 0.8713303208351135, 0.8684501051902771, 0.8654988408088684, 0.8624751567840576, 0.8593778610229492, 0.8562055826187134, 0.8529572486877441, 0.8496315479278564, 0.8462273478507996, 0.8427435755729675, 0.8391791582107544, 0.8355329632759094, 0.831804096698761, 0.8279916048049927, 0.8240947127342224, 0.8201125264167786, 0.8160443902015686, 0.8118896484375, 0.8076478242874146, 0.8033183217048645, 0.798900842666626, 0.7943951487541199, 0.7898010015487671, 0.7851184010505676, 0.7803474068641663, 0.7754881978034973, 0.7705410718917847, 0.7655064463615417, 0.7603848576545715, 0.7551771402359009, 0.7498840093612671, 0.7445064783096313, 0.7390457391738892, 0.7335030436515808, 0.7278798222541809, 0.7221776247024536, 0.7163982391357422, 0.7105435132980347, 0.7046155333518982, 0.6986164450645447, 0.6925486326217651, 0.9205020666122437, 0.9188966155052185, 0.9172496199607849, 0.9155600070953369, 0.9138267040252686, 0.9120485186576843, 0.9102243185043335, 0.9083530306816101, 0.9064333438873291, 0.9044641256332397, 0.9024442434310913, 0.900372326374054, 0.8982471823692322, 0.8960675597190857, 0.8938321471214294, 0.8915397524833679, 0.8891890645027161, 0.886778712272644, 0.8843074440956116, 0.8817739486694336, 0.8791769742965698, 0.8765150904655457, 0.8737871050834656, 0.8709916472434998, 0.8681274652481079, 0.8651933073997498, 0.862187922000885, 0.8591099381446838, 0.8559582829475403, 0.8527317047119141, 0.8494290113449097, 0.8460491299629211, 0.842590868473053, 0.8390532732009888, 0.8354352712631226, 0.8317359089851379, 0.8279542922973633, 0.8240896463394165, 0.820141077041626, 0.8161079287528992, 0.8119895458221436, 0.8077853918075562, 0.8034950494766235, 0.7991180419921875, 0.794654130935669, 0.7901031970977783, 0.7854650616645813, 0.7807397842407227, 0.7759275436401367, 0.7710286378860474, 0.7660433650016785, 0.760972261428833, 0.7558161020278931, 0.7505755424499512, 0.7452515363693237, 0.7398451566696167, 0.7343575954437256, 0.728790283203125, 0.7231446504592896, 0.717422366142273, 0.7116252183914185, 0.7057551145553589, 0.6998142600059509, 0.6938048005104065, 0.9199981093406677, 0.9183920621871948, 0.9167448878288269, 0.915055513381958, 0.9133228659629822, 0.9115458130836487, 0.9097233414649963, 0.9078541994094849, 0.9059372544288635, 0.9039714336395264, 0.9019554257392883, 0.8998880982398987, 0.8977682590484619, 0.895594596862793, 0.8933659195899963, 0.8910810351371765, 0.8887386322021484, 0.886337399482727, 0.8838761448860168, 0.8813535571098328, 0.8787684440612793, 0.8761193752288818, 0.8734052181243896, 0.8706246614456177, 0.8677764534950256, 0.8648593425750732, 0.8618720769882202, 0.858813464641571, 0.855682373046875, 0.8524775505065918, 0.8491979241371155, 0.8458423018455505, 0.8424096703529358, 0.8388990163803101, 0.8353092670440674, 0.8316395878791809, 0.8278890252113342, 0.8240567445755005, 0.8201419711112976, 0.8161440491676331, 0.8120623230934143, 0.8078962564468384, 0.8036453127861023, 0.7993091940879822, 0.7948874831199646, 0.7903801202774048, 0.7857869267463684, 0.7811079621315002, 0.7763432264328003, 0.7714930772781372, 0.7665578126907349, 0.7615379691123962, 0.7564340233802795, 0.7512468099594116, 0.7459771037101746, 0.7406259775161743, 0.7351945638656616, 0.7296841144561768, 0.7240960597991943, 0.7184320092201233, 0.7126935720443726, 0.7068825960159302, 0.7010011672973633, 0.6950514316558838, 0.9194708466529846, 0.9178639054298401, 0.916216254234314, 0.9145268797874451, 0.9127946496009827, 0.9110184907913208, 0.9091973900794983, 0.9073302149772644, 0.9054158329963684, 0.9034531116485596, 0.9014407992362976, 0.8993778824806213, 0.8972631096839905, 0.8950952291488647, 0.8928731083869934, 0.8905954957008362, 0.8882611989974976, 0.885869026184082, 0.8834176063537598, 0.88090580701828, 0.8783323764801025, 0.875696063041687, 0.8729956150054932, 0.8702298402786255, 0.8673975467681885, 0.8644974231719971, 0.8615282773971558, 0.8584890365600586, 0.8553784489631653, 0.8521954417228699, 0.8489388823509216, 0.8456076383590698, 0.8422006964683533, 0.8387170433998108, 0.835155725479126, 0.8315157890319824, 0.8277963995933533, 0.8239967226982117, 0.8201159834861755, 0.8161535263061523, 0.8121086955070496, 0.8079808950424194, 0.8037697672843933, 0.7994747757911682, 0.7950957417488098, 0.7906323671340942, 0.7860845923423767, 0.7814522981643677, 0.7767356634140015, 0.7719348669052124, 0.7670502066612244, 0.7620821595191956, 0.7570311427116394, 0.7518979907035828, 0.7466834187507629, 0.7413883805274963, 0.7360139489173889, 0.7305613160133362, 0.7250317931175232, 0.7194268703460693, 0.7137482166290283, 0.7079975605010986, 0.7021768093109131, 0.6962879300117493, 0.9189203977584839, 0.9173123836517334, 0.91566401720047, 0.9139742851257324, 0.9122422933578491, 0.9104667901992798, 0.9086468815803528, 0.9067814350128174, 0.9048693180084229, 0.902909517288208, 0.9009007811546326, 0.8988420367240906, 0.8967320919036865, 0.8945698738098145, 0.8923541307449341, 0.8900836706161499, 0.8877573609352112, 0.8853740096092224, 0.8829323053359985, 0.8804312348365784, 0.8778694272041321, 0.8752456903457642, 0.8725589513778687, 0.8698078989982605, 0.8669913411140442, 0.8641082048416138, 0.861157238483429, 0.858137309551239, 0.8550472855567932, 0.8518860340118408, 0.8486525416374207, 0.8453457355499268, 0.8419645428657532, 0.8385080099105835, 0.8349751830101013, 0.831365168094635, 0.8276771306991577, 0.8239102363586426, 0.8200637102127075, 0.8161369562149048, 0.8121291995048523, 0.8080400824546814, 0.8038689494132996, 0.7996155023574829, 0.7952794432640076, 0.7908604145050049, 0.7863584160804749, 0.7817733287811279, 0.7771052718162537, 0.7723543047904968, 0.7675208449363708, 0.7626051306724548, 0.7576077580451965, 0.7525293231010437, 0.7473705410957336, 0.7421323657035828, 0.7368156909942627, 0.731421709060669, 0.725951611995697, 0.7204068899154663, 0.7147890329360962, 0.7090996503829956, 0.7033406496047974, 0.6975138187408447, 0.9183469414710999, 0.9167375564575195, 0.9150882363319397, 0.913398027420044, 0.9116659760475159, 0.9098909497261047, 0.9080720543861389, 0.9062080979347229, 0.9042981266975403, 0.9023409485816956, 0.9003356099128723, 0.8982808589935303, 0.8961756825447083, 0.8940188884735107, 0.891809344291687, 0.8895459175109863, 0.887227475643158, 0.8848528265953064, 0.8824208378791809, 0.8799302577972412, 0.8773800134658813, 0.874768853187561, 0.8720957040786743, 0.8693593144416809, 0.8665586113929749, 0.8636923432350159, 0.8607594966888428, 0.8577588796615601, 0.8546894788742065, 0.8515501022338867, 0.8483397364616394, 0.8450573682785034, 0.8417020440101624, 0.8382726907730103, 0.834768533706665, 0.8311885595321655, 0.8275319933891296, 0.8237980008125305, 0.8199858665466309, 0.8160949945449829, 0.8121246695518494, 0.8080743551254272, 0.8039435744285583, 0.799731969833374, 0.7954391241073608, 0.7910648584365845, 0.7866090536117554, 0.782071590423584, 0.7774524688720703, 0.7727518677711487, 0.7679700255393982, 0.7631072998046875, 0.7581640481948853, 0.7531409859657288, 0.748038649559021, 0.7428579926490784, 0.7375998497009277, 0.732265293598175, 0.7268555164337158, 0.7213717699050903, 0.7158156037330627, 0.7101885676383972, 0.7044922709465027, 0.698728621006012, 0.9177506566047668, 0.9161396622657776, 0.914489209651947, 0.912798285484314, 0.9110659956932068, 0.9092912673950195, 0.907473087310791, 0.9056105017662048, 0.903702437877655, 0.9017478227615356, 0.8997456431388855, 0.8976948261260986, 0.8955941796302795, 0.8934427499771118, 0.8912392854690552, 0.8889827728271484, 0.8866720795631409, 0.8843060731887817, 0.8818836212158203, 0.8794035315513611, 0.8768647909164429, 0.8742661476135254, 0.8716065287590027, 0.868884801864624, 0.8660998344421387, 0.8632505536079407, 0.8603357672691345, 0.8573545217514038, 0.8543056845664978, 0.8511881828308105, 0.8480010628700256, 0.8447432518005371, 0.8414137959480286, 0.8380118012428284, 0.8345363140106201, 0.8309865593910217, 0.8273615837097168, 0.8236607313156128, 0.8198832273483276, 0.816028356552124, 0.8120956420898438, 0.8080843687057495, 0.8039942383766174, 0.7998246550559998, 0.7955754399299622, 0.7912462949752808, 0.7868369817733765, 0.7823474407196045, 0.7777777314186096, 0.7731279134750366, 0.7683981657028198, 0.7635888457298279, 0.7587003707885742, 0.7537331581115723, 0.7486879229545593, 0.7435654401779175, 0.7383664846420288, 0.7330920696258545, 0.7277433276176453, 0.7223214507102966, 0.7168278098106384, 0.71126389503479, 0.7056313753128052, 0.6999318599700928, 0.9171317219734192, 0.9155189394950867, 0.9138671159744263, 0.9121752977371216, 0.910442590713501, 0.9086679220199585, 0.9068503975868225, 0.9049890041351318, 0.9030827283859253, 0.9011304974555969, 0.8991313576698303, 0.8970842361450195, 0.8949881196022034, 0.8928418159484863, 0.8906444311141968, 0.8883947134017944, 0.8860917091369629, 0.8837342262268066, 0.8813212513923645, 0.8788516521453857, 0.8763242959976196, 0.8737381100654602, 0.8710920214653015, 0.8683849573135376, 0.865615725517273, 0.8627833724021912, 0.8598867654800415, 0.8569248914718628, 0.8538966774940491, 0.8508011102676392, 0.8476372361183167, 0.8444040417671204, 0.8411006331443787, 0.8377260565757751, 0.8342794179916382, 0.8307598829269409, 0.827166736125946, 0.8234990835189819, 0.8197563290596008, 0.8159378170967102, 0.8120428323745728, 0.8080708980560303, 0.8040215373039246, 0.799894392490387, 0.7956889867782593, 0.791405200958252, 0.7870427370071411, 0.782601535320282, 0.7780815958976746, 0.7734829187393188, 0.768805742263794, 0.764050304889679, 0.7592169642448425, 0.7543061971664429, 0.7493185997009277, 0.7442547678947449, 0.7391156554222107, 0.7339020371437073, 0.7286149859428406, 0.7232556939125061, 0.7178254127502441, 0.7123255133628845, 0.7067574858665466, 0.701123058795929, 0.9164903163909912, 0.9148755669593811, 0.9132221937179565, 0.9115293622016907, 0.9097959995269775, 0.9080212712287903, 0.9062042236328125, 0.904343843460083, 0.9024392366409302, 0.9004892706871033, 0.8984931111335754, 0.896449625492096, 0.8943578004837036, 0.892216682434082, 0.8900251388549805, 0.8877822160720825, 0.8854867815971375, 0.8831378221511841, 0.8807342648506165, 0.8782750964164734, 0.8757591247558594, 0.873185396194458, 0.8705528378486633, 0.8678603768348694, 0.8651069402694702, 0.8622915148735046, 0.8594130873680115, 0.8564706444740295, 0.8534631133079529, 0.8503895401954651, 0.8472490310668945, 0.8440405130386353, 0.8407632112503052, 0.8374161124229431, 0.8339985013008118, 0.8305094242095947, 0.8269481062889099, 0.8233139514923096, 0.8196060657501221, 0.8158239722251892, 0.8119669556617737, 0.8080345988273621, 0.8040263056755066, 0.7999417185783386, 0.7957804203033447, 0.7915422320365906, 0.7872269153594971, 0.7828343510627747, 0.7783644795417786, 0.7738173604011536, 0.7691931128501892, 0.7644919157028198, 0.7597141861915588, 0.7548602819442749, 0.7499307990074158, 0.7449262738227844, 0.7398474216461182, 0.7346952557563782, 0.7294705510139465, 0.7241744995117188, 0.7188082337379456, 0.713373064994812, 0.7078704237937927, 0.7023018598556519, 0.9158266186714172, 0.9142097234725952, 0.9125546813011169, 0.9108605980873108, 0.9091265201568604, 0.9073516130447388, 0.9055349230766296, 0.9036754369735718, 0.9017723202705383, 0.8998245596885681, 0.8978312015533447, 0.8957912921905518, 0.8937037587165833, 0.8915677070617676, 0.889382004737854, 0.8871457576751709, 0.8848578929901123, 0.882517397403717, 0.8801232576370239, 0.8776744604110718, 0.8751699328422546, 0.8726086616516113, 0.8699896335601807, 0.8673117756843567, 0.8645741939544678, 0.8617757558822632, 0.8589155077934265, 0.8559924960136414, 0.8530057668685913, 0.8499542474746704, 0.8468371629714966, 0.8436534404754639, 0.8404023051261902, 0.8370828628540039, 0.8336942791938782, 0.8302357792854309, 0.8267065286636353, 0.8231058716773987, 0.8194331526756287, 0.8156876564025879, 0.8118687868118286, 0.8079761266708374, 0.8040090799331665, 0.7999672889709473, 0.7958503365516663, 0.7916580438613892, 0.7873901128768921, 0.7830464839935303, 0.7786269783973694, 0.7741316556930542, 0.769560694694519, 0.7649141550064087, 0.7601923942565918, 0.755395770072937, 0.7505248188972473, 0.7455799579620361, 0.7405620217323303, 0.7354717254638672, 0.7303099632263184, 0.725077748298645, 0.7197760939598083, 0.7144063711166382, 0.70896977186203, 0.703467845916748, 0.9151408076286316, 0.9135217070579529, 0.9118648171424866, 0.9101694226264954, 0.9084345102310181, 0.9066592454910278, 0.904842734336853, 0.9029841423034668, 0.901082456111908, 0.8991367816925049, 0.8971462249755859, 0.8951097130775452, 0.8930264115333557, 0.8908953666687012, 0.8887155055999756, 0.8864859342575073, 0.8842055797576904, 0.8818735480308533, 0.8794888257980347, 0.8770503997802734, 0.8745572566986084, 0.8720084428787231, 0.8694029450416565, 0.866739809513092, 0.8640180826187134, 0.8612366914749146, 0.8583947420120239, 0.8554912209510803, 0.8525252938270569, 0.8494960069656372, 0.8464023470878601, 0.8432435393333435, 0.8400187492370605, 0.8367270231246948, 0.833367645740509, 0.8299397826194763, 0.8264427185058594, 0.8228757977485657, 0.8192382454872131, 0.8155295252799988, 0.8117489814758301, 0.8078961968421936, 0.8039705753326416, 0.7999718189239502, 0.7958994507789612, 0.7917532324790955, 0.7875329256057739, 0.783238410949707, 0.7788695693016052, 0.7744264006614685, 0.7699089050292969, 0.7653173804283142, 0.7606519460678101, 0.7559129595756531, 0.7511008977890015, 0.7462161779403687, 0.7412595152854919, 0.7362316250801086, 0.731133222579956, 0.7259653210639954, 0.7207289338111877, 0.7154251933097839, 0.7100553512573242, 0.7046207189559937, 0.9144332408905029, 0.9128116965293884, 0.9111528992652893, 0.9094560146331787, 0.907720148563385, 0.9059444665908813, 0.9041281342506409, 0.9022702574729919, 0.9003699421882629, 0.8984262943267822, 0.8964384198188782, 0.8944054245948792, 0.892326295375824, 0.8902001976966858, 0.8880261182785034, 0.8858031630516052, 0.8835303783416748, 0.8812068104743958, 0.8788315057754517, 0.8764034509658813, 0.8739217519760132, 0.8713854551315308, 0.8687936067581177, 0.8661451935768127, 0.8634393215179443, 0.860675036907196, 0.8578514456748962, 0.8549675941467285, 0.8520225286483765, 0.849015474319458, 0.845945417881012, 0.842811644077301, 0.8396131992340088, 0.836349368095398, 0.8330192565917969, 0.829622209072113, 0.8261575102806091, 0.8226243257522583, 0.8190221786499023, 0.815350353717804, 0.8116083145141602, 0.807795524597168, 0.803911566734314, 0.7999559640884399, 0.7959283590316772, 0.7918283939361572, 0.7876559495925903, 0.7834107875823975, 0.7790928483009338, 0.7747020125389099, 0.7702383399009705, 0.7657020092010498, 0.7610931992530823, 0.756412148475647, 0.7516593337059021, 0.7468351125717163, 0.7419401407241821, 0.7369749546051025, 0.7319404482841492, 0.7268373370170593, 0.721666693687439, 0.7164294719696045, 0.7111268639564514, 0.7057601809501648, 0.9137040376663208, 0.9120799899101257, 0.9104191660881042, 0.9087206721305847, 0.9069837927818298, 0.905207633972168, 0.903391420841217, 0.9015341997146606, 0.8996351957321167, 0.8976935744285583, 0.8957083821296692, 0.8936787247657776, 0.8916038274765015, 0.8894826769828796, 0.8873143792152405, 0.8850980997085571, 0.8828328847885132, 0.8805177807807922, 0.8781518936157227, 0.8757343292236328, 0.8732641339302063, 0.8707404136657715, 0.8681621551513672, 0.8655285835266113, 0.8628386855125427, 0.8600915670394897, 0.8572863936424255, 0.8544222116470337, 0.8514981865882874, 0.8485134243965149, 0.8454671502113342, 0.842358410358429, 0.8391864895820618, 0.8359506130218506, 0.8326500058174133, 0.8292838335037231, 0.825851559638977, 0.8223523497581482, 0.8187857270240784, 0.8151509165763855, 0.8114475011825562, 0.8076749444007874, 0.8038327097892761, 0.7999204397201538, 0.795937716960907, 0.7918842434883118, 0.7877598404884338, 0.7835642099380493, 0.7792972922325134, 0.7749590277671814, 0.7705494165420532, 0.7660685181617737, 0.7615165710449219, 0.7568937540054321, 0.7522004246711731, 0.7474370002746582, 0.7426039576530457, 0.737701952457428, 0.7327315807342529, 0.7276936769485474, 0.7225891351699829, 0.7174189686775208, 0.7121841311454773, 0.7068858742713928, 0.9129534363746643, 0.9113268256187439, 0.9096639156341553, 0.9079638123512268, 0.9062258005142212, 0.9044491052627563, 0.9026329517364502, 0.9007763862609863, 0.8988786935806274, 0.8969389796257019, 0.8949564695358276, 0.892930269241333, 0.8908594846725464, 0.8887433409690857, 0.8865808844566345, 0.884371280670166, 0.8821136355400085, 0.8798070549964905, 0.877450704574585, 0.8750436305999756, 0.8725849986076355, 0.8700738549232483, 0.8675093650817871, 0.8648906350135803, 0.8622168302536011, 0.859486997127533, 0.8567003607749939, 0.8538559675216675, 0.8509530425071716, 0.8479907512664795, 0.8449682593345642, 0.8418847322463989, 0.8387394547462463, 0.8355316519737244, 0.8322605490684509, 0.8289254903793335, 0.82552570104599, 0.8220606446266174, 0.8185296058654785, 0.8149319887161255, 0.8112673163414001, 0.8075350522994995, 0.8037347197532654, 0.7998659014701843, 0.7959282398223877, 0.7919213771820068, 0.7878451347351074, 0.7836992144584656, 0.7794834971427917, 0.7751979827880859, 0.7708425521850586, 0.7664173245429993, 0.7619224190711975, 0.7573580145835876, 0.7527244091033936, 0.7480220198631287, 0.7432512640953064, 0.738412618637085, 0.7335067987442017, 0.728534460067749, 0.7234963774681091, 0.7183935642242432, 0.7132269144058228, 0.7079975605010986, 0.9121817946434021, 0.9105525612831116, 0.9088873863220215, 0.9071856737136841, 0.9054465293884277, 0.9036692976951599, 0.901853084564209, 0.8999971747398376, 0.8981007933616638, 0.8961630463600159, 0.8941832184791565, 0.8921604156494141, 0.8900938630104065, 0.8879827260971069, 0.8858261108398438, 0.8836232423782349, 0.8813732862472534, 0.879075288772583, 0.8767284750938416, 0.8743320107460022, 0.8718849420547485, 0.8693865537643433, 0.86683589220047, 0.8642321825027466, 0.8615744709968567, 0.858862042427063, 0.8560940027236938, 0.8532695174217224, 0.8503878116607666, 0.8474481105804443, 0.844449520111084, 0.8413913249969482, 0.8382728099822998, 0.8350932002067566, 0.8318517804145813, 0.8285478353500366, 0.8251807689666748, 0.8217498660087585, 0.8182545900344849, 0.8146942853927612, 0.8110684752464294, 0.8073766231536865, 0.803618311882019, 0.7997931241989136, 0.7959005832672119, 0.791940450668335, 0.7879124879837036, 0.783816397190094, 0.7796520590782166, 0.7754194140434265, 0.7711183428764343, 0.7667489051818848, 0.7623111605644226, 0.7578054070472717, 0.7532317638397217, 0.7485905289649963, 0.7438822388648987, 0.7391072511672974, 0.7342661619186401, 0.7293596267700195, 0.7243883609771729, 0.719353199005127, 0.7142550945281982, 0.7090950012207031, 0.9113893508911133, 0.9097573757171631, 0.9080900549888611, 0.9063866138458252, 0.9046463370323181, 0.9028684496879578, 0.9010522961616516, 0.8991970419883728, 0.897301971912384, 0.8953661918640137, 0.893389105796814, 0.8913697600364685, 0.8893074989318848, 0.8872013688087463, 0.8850507140159607, 0.8828546404838562, 0.8806123733520508, 0.8783230781555176, 0.8759859204292297, 0.8736000657081604, 0.8711647987365723, 0.8686792254447937, 0.8661424517631531, 0.8635538220405579, 0.8609124422073364, 0.8582174777984619, 0.855468213558197, 0.8526637554168701, 0.8498033881187439, 0.8468863368034363, 0.8439117670059204, 0.8408790826797485, 0.837787389755249, 0.8346360921859741, 0.8314244747161865, 0.8281518220901489, 0.8248175382614136, 0.8214209079742432, 0.8179615139961243, 0.8144385814666748, 0.8108517527580261, 0.8072004914283752, 0.8034842610359192, 0.7997027039527893, 0.7958554625511169, 0.7919421792030334, 0.7879625558853149, 0.7839164137840271, 0.7798035740852356, 0.7756238579750061, 0.7713772058486938, 0.7670636773109436, 0.7626833319664001, 0.7582362294197083, 0.7537226676940918, 0.7491428256034851, 0.7444971203804016, 0.73978590965271, 0.7350097894668579, 0.7301692366600037, 0.7252650260925293, 0.7202978134155273, 0.7152685523033142, 0.710178017616272, 0.9105764031410217, 0.9089416861534119, 0.9072721004486084, 0.9055669903755188, 0.9038255214691162, 0.9020470976829529, 0.9002310037612915, 0.8983764052391052, 0.8964826464653015, 0.8945489525794983, 0.8925746083259583, 0.8905587792396545, 0.8885008096694946, 0.8863998651504517, 0.8842551708221436, 0.882066011428833, 0.8798315525054932, 0.8775510191917419, 0.8752236366271973, 0.8728485703468323, 0.8704251646995544, 0.8679524660110474, 0.8654298186302185, 0.862856388092041, 0.8602313995361328, 0.857554018497467, 0.8548235893249512, 0.8520393371582031, 0.849200427532196, 0.8463061451911926, 0.8433558344841003, 0.8403486609458923, 0.8372839689254761, 0.834161102771759, 0.8309794068336487, 0.8277381062507629, 0.8244367241859436, 0.8210745453834534, 0.817651093006134, 0.8141657114028931, 0.8106179237365723, 0.8070072531700134, 0.8033332228660583, 0.7995954751968384, 0.7957935333251953, 0.79192715883255, 0.7879959940910339, 0.7839998602867126, 0.7799385190010071, 0.7758118510246277, 0.7716197371482849, 0.7673621773719788, 0.763039231300354, 0.7586509585380554, 0.7541975378990173, 0.7496791481971741, 0.7450961470603943, 0.7404488921165466, 0.7357378602027893, 0.7309634685516357, 0.7261264324188232, 0.7212274074554443, 0.7162671089172363, 0.7112464308738708, 0.9097431898117065, 0.9081057906150818, 0.9064339995384216, 0.9047271609306335, 0.902984619140625, 0.9012056589126587, 0.8993895649909973, 0.8975356817245483, 0.8956433534622192, 0.8937117457389832, 0.8917402029037476, 0.8897280693054199, 0.8876744508743286, 0.8855787515640259, 0.8834401965141296, 0.8812580108642578, 0.8790314793586731, 0.8767598271369934, 0.8744422793388367, 0.8720781803131104, 0.8696666955947876, 0.8672071099281311, 0.8646986484527588, 0.8621405363082886, 0.8595320582389832, 0.8568724989891052, 0.8541610836982727, 0.8513970971107483, 0.8485797643661499, 0.8457084894180298, 0.8427824378013611, 0.8398009538650513, 0.8367634415626526, 0.833669126033783, 0.8305174112319946, 0.8273076415061951, 0.8240392208099365, 0.8207115530967712, 0.8173241019248962, 0.8138763308525085, 0.8103677034378052, 0.8067977428436279, 0.8031659722328186, 0.7994720339775085, 0.7957155108451843, 0.7918961048126221, 0.7880134582519531, 0.7840673327445984, 0.7800575494766235, 0.7759839296340942, 0.7718464136123657, 0.7676448822021484, 0.7633793354034424, 0.7590498924255371, 0.7546566724777222, 0.7501997947692871, 0.7456796169281006, 0.7410963773727417, 0.7364504337310791, 0.7317423820495605, 0.7269726395606995, 0.7221418619155884, 0.7172507643699646, 0.7123000621795654, 0.9088901281356812, 0.9072499871253967, 0.9055760502815247, 0.9038675427436829, 0.9021238684654236, 0.9003444314002991, 0.898528516292572, 0.8966754078865051, 0.8947845101356506, 0.8928551077842712, 0.8908865451812744, 0.8888780474662781, 0.8868290185928345, 0.8847386240959167, 0.8826062679290771, 0.8804312348365784, 0.8782127499580383, 0.8759500980377197, 0.87364262342453, 0.8712895512580872, 0.8688901662826538, 0.8664438128471375, 0.8639496564865112, 0.8614070415496826, 0.8588152527809143, 0.8561736345291138, 0.8534813523292542, 0.8507378101348877, 0.8479422330856323, 0.8450939655303955, 0.8421924114227295, 0.8392367362976074, 0.8362264633178711, 0.8331608176231384, 0.8300392031669617, 0.8268610835075378, 0.8236258029937744, 0.8203327655792236, 0.8169814348220825, 0.8135713338851929, 0.8101018667221069, 0.806572675704956, 0.8029832243919373, 0.7993331551551819, 0.7956220507621765, 0.7918496131896973, 0.7880154848098755, 0.7841194272041321, 0.7801612615585327, 0.7761407494544983, 0.7720577716827393, 0.7679122090339661, 0.7637041807174683, 0.7594335675239563, 0.7551005482673645, 0.7507051825523376, 0.7462477684020996, 0.7417285442352295, 0.7371478080749512, 0.7325060367584229, 0.727803647518158, 0.7230412364006042, 0.7182193994522095, 0.7133387923240662, 0.9080175161361694, 0.9063747525215149, 0.9046986103057861, 0.9029884934425354, 0.9012438654899597, 0.899463951587677, 0.897648274898529, 0.8957960605621338, 0.8939067125320435, 0.8919796347618103, 0.8900140523910522, 0.8880094289779663, 0.8859649896621704, 0.8838801383972168, 0.8817541003227234, 0.8795863389968872, 0.8773760199546814, 0.875122606754303, 0.8728252649307251, 0.8704833984375, 0.8680962920188904, 0.8656632900238037, 0.8631836175918579, 0.8606566786766052, 0.8580817580223083, 0.85545814037323, 0.8527852296829224, 0.8500622510910034, 0.8472885489463806, 0.8444635272026062, 0.8415865302085876, 0.8386568427085876, 0.8356739282608032, 0.8326370716094971, 0.8295457363128662, 0.8263993263244629, 0.8231971859931946, 0.8199388384819031, 0.8166237473487854, 0.8132513761520386, 0.8098211884498596, 0.8063327670097351, 0.8027856945991516, 0.7991795539855957, 0.7955138683319092, 0.7917883992195129, 0.7880028486251831, 0.784156858921051, 0.7802501916885376, 0.7762827277183533, 0.7722542881965637, 0.7681647539138794, 0.7640140652656555, 0.7598022818565369, 0.7555294036865234, 0.7511955499649048, 0.7468008995056152, 0.7423456311225891, 0.7378301024436951, 0.7332545518875122, 0.7286195755004883, 0.7239254713058472, 0.7191729545593262, 0.7143625617027283, 0.9071257710456848, 0.9054803848266602, 0.9038021564483643, 0.9020904898643494, 0.9003449082374573, 0.8985646963119507, 0.8967493176460266, 0.8948981165885925, 0.8930104374885559, 0.8910857439041138, 0.8891233801841736, 0.8871226906776428, 0.8850830793380737, 0.883003830909729, 0.8808843493461609, 0.8787239193916321, 0.8765220046043396, 0.8742778897285461, 0.8719909191131592, 0.8696604371070862, 0.8672857880592346, 0.864866316318512, 0.8624013662338257, 0.8598902225494385, 0.8573322892189026, 0.8547269105911255, 0.8520734310150146, 0.8493712544441223, 0.8466196060180664, 0.843817949295044, 0.8409656882286072, 0.8380621075630188, 0.8351066708564758, 0.8320987224578857, 0.8290377259254456, 0.8259230852127075, 0.8227542638778687, 0.8195306658744812, 0.8162518739700317, 0.8129172325134277, 0.8095263838768005, 0.8060788512229919, 0.8025741577148438, 0.7990118861198425, 0.7953916788101196, 0.7917131781578064, 0.787976086139679, 0.7841801047325134, 0.7803249955177307, 0.7764105200767517, 0.7724364995956421, 0.7684029340744019, 0.7643095850944519, 0.760156512260437, 0.7559437155723572, 0.7516712546348572, 0.7473392486572266, 0.7429479360580444, 0.7384974956512451, 0.7339882254600525, 0.7294204831123352, 0.7247947454452515, 0.7201114892959595, 0.7153712511062622, 0.9062151908874512, 0.9045672416687012, 0.9028870463371277, 0.901174008846283, 0.8994275331497192, 0.8976471424102783, 0.8958321809768677, 0.8939820528030396, 0.8920962810516357, 0.890174150466919, 0.8882150650024414, 0.8862184882164001, 0.8841837644577026, 0.8821103572845459, 0.8799975514411926, 0.8778447508811951, 0.8756513595581055, 0.8734167814254761, 0.8711403608322144, 0.868821382522583, 0.8664593696594238, 0.8640536069869995, 0.8616034984588623, 0.8591083884239197, 0.8565676808357239, 0.8539807200431824, 0.8513468503952026, 0.8486655354499817, 0.845936119556427, 0.8431580066680908, 0.8403305411338806, 0.8374532461166382, 0.8345254063606262, 0.8315464854240417, 0.8285159468650818, 0.8254331946372986, 0.8222977519035339, 0.8191090226173401, 0.8158665299415588, 0.8125697374343872, 0.8092182278633118, 0.8058115243911743, 0.8023492097854614, 0.7988308668136597, 0.7952560782432556, 0.7916244864463806, 0.7879358530044556, 0.7841898202896118, 0.7803861498832703, 0.7765246033668518, 0.7726050019264221, 0.7686272263526917, 0.7645911574363708, 0.7604966759681702, 0.7563438415527344, 0.7521326541900635, 0.747863233089447, 0.7435356974601746, 0.7391502261161804, 0.7347070574760437, 0.7302065491676331, 0.7256489992141724, 0.7210349440574646, 0.7163648009300232, 0.9052862524986267, 0.9036358594894409, 0.9019536972045898, 0.9002393484115601, 0.8984922170639038, 0.8967117071151733, 0.8948973417282104, 0.8930485248565674, 0.8911646604537964, 0.8892452716827393, 0.8872897028923035, 0.885297417640686, 0.8832678198814392, 0.88120037317276, 0.8790944218635559, 0.8769494295120239, 0.8747648000717163, 0.8725398778915405, 0.8702741861343384, 0.8679670095443726, 0.8656178116798401, 0.8632259964942932, 0.8607909083366394, 0.8583120107650757, 0.8557886481285095, 0.8532202243804932, 0.8506062030792236, 0.8479459881782532, 0.8452389240264893, 0.8424844741821289, 0.8396820425987244, 0.8368310928344727, 0.8339309692382812, 0.8309812545776367, 0.8279812335968018, 0.8249304890632629, 0.8218284845352173, 0.8186746835708618, 0.8154685497283936, 0.8122096061706543, 0.8088974952697754, 0.8055316209793091, 0.8021116852760315, 0.7986371517181396, 0.7951077818870544, 0.7915230989456177, 0.7878828644752502, 0.7841867208480835, 0.7804343700408936, 0.7766256332397461, 0.7727603316307068, 0.7688382267951965, 0.7648591995239258, 0.7608231902122498, 0.7567301988601685, 0.7525801062583923, 0.7483730912208557, 0.7441091537475586, 0.7397884726524353, 0.7354112863540649, 0.7309778332710266, 0.7264884114265442, 0.7219433188438416, 0.7173431515693665, 0.9043392539024353, 0.9026865363121033, 0.9010026454925537, 0.8992871046066284, 0.8975393772125244, 0.8957589864730835, 0.8939453363418579, 0.8920979499816895, 0.8902162313461304, 0.8882997632026672, 0.8863478302955627, 0.8843600749969482, 0.8823357820510864, 0.8802744746208191, 0.8781756162643433, 0.876038670539856, 0.8738629221916199, 0.8716480135917664, 0.8693931698799133, 0.8670979738235474, 0.8647618293762207, 0.8623841404914856, 0.8599642515182495, 0.857501745223999, 0.8549959659576416, 0.8524463772773743, 0.8498523235321045, 0.8472133874893188, 0.84452885389328, 0.8417982459068298, 0.8390209674835205, 0.8361964821815491, 0.8333242535591125, 0.830403745174408, 0.8274344205856323, 0.8244157433509827, 0.8213472366333008, 0.8182283639907837, 0.8150586485862732, 0.8118376135826111, 0.8085648417472839, 0.8052398562431335, 0.8018622398376465, 0.7984315752983093, 0.7949475049972534, 0.7914096117019653, 0.787817656993866, 0.7841712832450867, 0.7804701328277588, 0.7767140865325928, 0.7729029059410095, 0.7690362930297852, 0.7651141881942749, 0.761136531829834, 0.7571030855178833, 0.7530139684677124, 0.7488691210746765, 0.7446686029434204, 0.7404125928878784, 0.7361011505126953, 0.7317345142364502, 0.7273129224777222, 0.7228367924690247, 0.7183063626289368, 0.9033747315406799, 0.9017198085784912, 0.9000343084335327, 0.8983177542686462, 0.8965696096420288, 0.894789457321167, 0.892976701259613, 0.8911309242248535, 0.8892515897750854, 0.8873381614685059, 0.8853901624679565, 0.8834070563316345, 0.8813883662223816, 0.8793334364891052, 0.8772419095039368, 0.8751130700111389, 0.872946560382843, 0.8707417249679565, 0.8684980869293213, 0.8662151098251343, 0.8638921976089478, 0.8615288138389587, 0.8591244220733643, 0.8566784858703613, 0.854190468788147, 0.8516598343849182, 0.8490859866142273, 0.846468448638916, 0.8438066244125366, 0.8410999774932861, 0.8383480310440063, 0.8355501890182495, 0.8327059149742126, 0.8298147916793823, 0.826876163482666, 0.8238896727561951, 0.8208547234535217, 0.8177708387374878, 0.8146376013755798, 0.8114544749259949, 0.8082210421562195, 0.8049368858337402, 0.8016015291213989, 0.7982146739959717, 0.7947758436203003, 0.7912846803665161, 0.7877408862113953, 0.7841441035270691, 0.7804940938949585, 0.7767905592918396, 0.7730333209037781, 0.7692220211029053, 0.7653566598892212, 0.7614370584487915, 0.7574630379676819, 0.7534345984458923, 0.7493516802787781, 0.7452143430709839, 0.7410227060317993, 0.7367767691612244, 0.7324767112731934, 0.7281227707862854, 0.7237152457237244, 0.7192543745040894, 0.902393102645874, 0.9007361531257629, 0.8990492224693298, 0.8973317742347717, 0.89558345079422, 0.8938036561012268, 0.8919920921325684, 0.8901481032371521, 0.8882713317871094, 0.8863611817359924, 0.8844172954559326, 0.882439136505127, 0.8804261088371277, 0.8783778548240662, 0.8762938380241394, 0.8741734623908997, 0.8720163702964783, 0.8698219060897827, 0.8675896525382996, 0.8653190732002258, 0.8630096316337585, 0.8606607913970947, 0.8582720756530762, 0.8558430075645447, 0.8533729314804077, 0.8508614897727966, 0.8483080267906189, 0.8457120656967163, 0.8430730700492859, 0.8403906226158142, 0.8376640677452087, 0.834892988204956, 0.8320768475532532, 0.8292151689529419, 0.8263074159622192, 0.823353111743927, 0.820351779460907, 0.817302942276001, 0.8142061233520508, 0.811060905456543, 0.8078668117523193, 0.8046234250068665, 0.8013303279876709, 0.797987163066864, 0.7945934534072876, 0.7911489009857178, 0.7876531481742859, 0.7841058969497681, 0.7805067896842957, 0.7768555879592896, 0.7731520533561707, 0.7693959474563599, 0.7655870318412781, 0.7617251873016357, 0.7578103542327881, 0.753842294216156, 0.749821126461029, 0.7457466721534729, 0.7416191101074219, 0.7374383807182312, 0.7332046031951904, 0.7289180755615234, 0.724578857421875, 0.7201873064041138, 0.901394784450531, 0.8997359871864319, 0.8980478048324585, 0.8963297009468079, 0.8945813179016113, 0.8928022384643555, 0.8909919261932373, 0.8891500234603882, 0.8872759938240051, 0.8853694200515747, 0.8834298253059387, 0.8814567923545837, 0.8794498443603516, 0.8774084448814392, 0.8753321766853333, 0.8732205033302307, 0.8710730075836182, 0.8688892126083374, 0.8666685819625854, 0.8644106388092041, 0.8621149063110352, 0.8597809076309204, 0.8574081063270569, 0.8549960255622864, 0.8525441288948059, 0.850051999092102, 0.8475190997123718, 0.844944953918457, 0.8423290252685547, 0.8396708369255066, 0.8369699120521545, 0.8342257142066956, 0.8314377665519714, 0.8286056518554688, 0.8257287740707397, 0.8228067755699158, 0.8198391199111938, 0.8168253302574158, 0.8137649893760681, 0.8106576800346375, 0.8075028657913208, 0.8043002486228943, 0.8010493516921997, 0.7977497577667236, 0.7944010496139526, 0.7910029888153076, 0.7875551581382751, 0.7840571999549866, 0.780508816242218, 0.7769097089767456, 0.7732596397399902, 0.7695584297180176, 0.765805721282959, 0.7620014548301697, 0.7581454515457153, 0.7542375922203064, 0.7502776980400085, 0.7462658882141113, 0.7422020435333252, 0.7380861639976501, 0.7339184284210205, 0.729698896408081, 0.7254276871681213, 0.7211050987243652, 0.9003803730010986, 0.8987199068069458, 0.8970305919647217, 0.8953120708465576, 0.8935638666152954, 0.8917856216430664, 0.8899769186973572, 0.8881372213363647, 0.8862662315368652, 0.8843634724617004, 0.8824284672737122, 0.8804608583450317, 0.8784601092338562, 0.8764258623123169, 0.8743575811386108, 0.8722549080848694, 0.8701173067092896, 0.8679443597793579, 0.865735650062561, 0.8634905815124512, 0.8612087965011597, 0.8588898181915283, 0.8565331697463989, 0.8541383147239685, 0.8517048358917236, 0.8492323160171509, 0.8467201590538025, 0.844167947769165, 0.8415752053260803, 0.8389415144920349, 0.836266279220581, 0.8335490822792053, 0.8307895064353943, 0.827987015247345, 0.8251411318778992, 0.8222514390945435, 0.8193175196647644, 0.816338837146759, 0.8133149743080139, 0.8102454543113708, 0.807129979133606, 0.8039679527282715, 0.8007591366767883, 0.7975030541419983, 0.7941992878913879, 0.7908475399017334, 0.7874473929405212, 0.7839985489845276, 0.7805006504058838, 0.7769534587860107, 0.7733566761016846, 0.7697100043296814, 0.7660132646560669, 0.7622662782669067, 0.758468747138977, 0.7546206712722778, 0.7507218718528748, 0.7467722296714783, 0.7427718043327332, 0.7387204766273499, 0.7346183061599731, 0.7304654121398926, 0.7262619137763977, 0.7220078706741333, 0.8993502259254456, 0.8976883292198181, 0.8959981799125671, 0.894279420375824, 0.8925316333770752, 0.8907545208930969, 0.8889475464820862, 0.8871104717254639, 0.8852427005767822, 0.8833439946174622, 0.8814138770103455, 0.8794519305229187, 0.8774576783180237, 0.8754308223724365, 0.873370885848999, 0.871277391910553, 0.8691499829292297, 0.8669881820678711, 0.8647915720939636, 0.8625596761703491, 0.8602920770645142, 0.8579883575439453, 0.8556480407714844, 0.8532707095146179, 0.8508558869361877, 0.8484030961990356, 0.8459118604660034, 0.8433818221092224, 0.8408124446868896, 0.8382033705711365, 0.8355540037155151, 0.8328639268875122, 0.830132782459259, 0.8273600339889526, 0.8245452046394348, 0.8216879367828369, 0.818787693977356, 0.8158440589904785, 0.8128566741943359, 0.8098250031471252, 0.8067486882209778, 0.8036273121833801, 0.8004604578018188, 0.7972477078437805, 0.7939887046813965, 0.7906830906867981, 0.7873304486274719, 0.7839305400848389, 0.7804829478263855, 0.7769873738288879, 0.7734435200691223, 0.7698512077331543, 0.7662100791931152, 0.7625199556350708, 0.7587806582450867, 0.7549920082092285, 0.7511538863182068, 0.7472661137580872, 0.7433286309242249, 0.7393414378166199, 0.7353044748306274, 0.7312178611755371, 0.7270815372467041, 0.7228957414627075, 0.8983049392700195, 0.8966417908668518, 0.8949510455131531, 0.8932322859764099, 0.8914852142333984, 0.88970947265625, 0.8879045844078064, 0.8860702514648438, 0.8842060565948486, 0.8823117017745972, 0.8803866505622864, 0.8784306645393372, 0.8764432668685913, 0.8744240403175354, 0.8723726868629456, 0.8702886700630188, 0.8681716918945312, 0.8660213351249695, 0.8638371229171753, 0.8616186380386353, 0.8593655228614807, 0.8570773005485535, 0.8547536134719849, 0.8523939251899719, 0.8499979376792908, 0.8475651144981384, 0.8450950980186462, 0.8425873517990112, 0.8400415182113647, 0.8374571800231934, 0.8348338007926941, 0.8321710228919983, 0.8294683694839478, 0.826725423336029, 0.8239417672157288, 0.8211168646812439, 0.8182504177093506, 0.8153418898582458, 0.8123908638954163, 0.8093969821929932, 0.8063598275184631, 0.803278923034668, 0.800153911113739, 0.7969844341278076, 0.7937700152397156, 0.790510356426239, 0.7872050404548645, 0.7838537693023682, 0.7804561853408813, 0.7770119309425354, 0.773520827293396, 0.7699824571609497, 0.7663966417312622, 0.7627630829811096, 0.7590816020965576, 0.7553519606590271, 0.7515740990638733, 0.7477477192878723, 0.7438728213310242, 0.7399493455886841, 0.7359771728515625, 0.7319563031196594, 0.7278867959976196, 0.7237687110900879, 0.8972450494766235, 0.8955809473991394, 0.8938897848129272, 0.8921713829040527, 0.8904252052307129, 0.8886510729789734, 0.8868485689163208, 0.8850172758102417, 0.883156955242157, 0.8812671899795532, 0.879347562789917, 0.8773978352546692, 0.8754174709320068, 0.8734062910079956, 0.8713637590408325, 0.8692895174026489, 0.867183268070221, 0.8650445938110352, 0.8628730177879333, 0.8606682419776917, 0.8584298491477966, 0.8561574220657349, 0.8538505434989929, 0.8515088558197021, 0.8491318821907043, 0.8467192649841309, 0.8442705273628235, 0.8417853116989136, 0.8392632007598877, 0.8367037177085876, 0.8341065049171448, 0.8314710855484009, 0.8287970423698425, 0.8260840177536011, 0.8233314752578735, 0.8205390572547913, 0.8177063465118408, 0.814832866191864, 0.8119182586669922, 0.8089621067047119, 0.8059639930725098, 0.8029235005378723, 0.7998402118682861, 0.7967137694358826, 0.7935437560081482, 0.7903298139572144, 0.7870716452598572, 0.7837687730789185, 0.7804208993911743, 0.7770277261734009, 0.7735889554023743, 0.7701042294502258, 0.7665733695030212, 0.7629959583282471, 0.7593719363212585, 0.755700945854187, 0.7519828677177429, 0.7482174634933472, 0.7444047331809998, 0.7405444383621216, 0.7366365194320679, 0.7326809763908386, 0.7286778092384338, 0.7246269583702087])

# union_volume_64 = 0.014487585984170437


In [ ]:
# @title Visualize configurations of tubes

# Recreate tubes_params_np if not available
N_grid = 16
i_vals_grid = np.arange(N_grid)
j_vals_grid = np.arange(N_grid)
i_grid_grid, j_grid_grid = np.meshgrid(i_vals_grid, j_vals_grid, indexing='ij')
i_flat_grid = i_grid_grid.flatten()
j_flat_grid = j_grid_grid.flatten()
num_tubes_grid = N_grid * N_grid
# x_rand_grid = np.random.uniform(0, 5, size=num_tubes_grid).astype(np.float32)
# y_rand_grid = np.random.uniform(0, 5, size=num_tubes_grid).astype(np.float32)
tubes_params_np_grid = np.stack([X_RAND, Y_RAND, i_flat_grid, j_flat_grid], axis=-1)

# Convert numpy array rows to list of tuples for the function input
tubes_data_grid = [(row[0], row[1], int(row[2]), int(row[3]), N_grid) for row in tubes_params_np_grid]

print(f"\nPlotting {len(tubes_data_grid)} tubes from N=10 grid example...")
# Limit number of tubes plotted for performance if necessary
limit_plot = 4096

if len(tubes_data_grid) > limit_plot:
    print(f"(Plotting only the first {limit_plot} tubes for performance)")
    tubes_data_grid = tubes_data_grid[:limit_plot]


fig_grid = plot_multiple_tubes_plotly(tubes_data_grid, title=f"Plotting Tubes from N={N_grid} Grid", opacity=0.5)
fig_grid.show()

tmp_filename = "kakeya_fs_64.html"

# Export/download figures
# fig_grid.write_html("fig_grid.html", full_html=True, include_plotlyjs='inline')
# fig_grid.write_html(tmp_filename, include_plotlyjs="cdn")
# %download_file $tmp_filename
# display.display(display.HTML(filename=tmp_filename))
# fig_grid.show()